In [1]:
import pandas as pd
import numpy as np
import chart_studio.plotly as py
import cufflinks as cf
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import collections
from collections import OrderedDict
%matplotlib inline
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
cf.go_offline()

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
####### Macro Cases ######
dfn4_macro_fused_probempi = pd.read_csv("n4_fused_macro_probmpi.csv")
dfn4_macro_fused_nonprobempi = pd.read_csv("n4_fused_macro_nonprobmpi.csv")
dfn4_macro_newgeo_probempi = pd.read_csv("n4_newgeo_macro_probmpi.csv")
dfn4_macro_newgeo_nonprobempi = pd.read_csv("n4_newgeo_macro_nonprobmpi.csv")
dfn4_macro_nonfused_probempi = pd.read_csv("n4_nonfused_macro_probmpi.csv")
dfn4_macro_nonfused_nonprobempi = pd.read_csv("n4_nonfused_macro_nonprobmpi.csv")

####### AVX Cases ######
dfn4_avx_fused_probmpi = pd.read_csv("n4_fused_avx_probmpi.csv")
# dfn4_avx_fused_probmpi = pd.read_csv("n4_fused_avx_probmpi_updated.csv")
dfn4_avx_fused_nonprobmpi = pd.read_csv("n4_fused_avx_nonprobmpi.csv")
dfn4_avx_newgeo_probempi = pd.read_csv("n4_newgeo_avx_probmpi.csv")
dfn4_avx_newgeo_nonprobempi = pd.read_csv("n4_newgeo_avx_nonprobmpi.csv")
dfn4_avx_nonfused_probempi = pd.read_csv("n4_nonfused_avx_probmpi.csv")
dfn4_avx_nonfused_nonprobempi = pd.read_csv("n4_nonfused_avx_nonprobmpi.csv")

####### VECT Cases ######
dfn4_vect_fused_probmpi = pd.read_csv("n4_fused_vect_probmpi.csv")
dfn4_vect_fused_nonprobmpi = pd.read_csv("n4_fused_vect_nonprobmpi.csv")
dfn4_vect_newgeo_probempi = pd.read_csv("n4_newgeo_vect_probmpi.csv")
dfn4_vect_newgeo_nonprobempi = pd.read_csv("n4_newgeo_vect_nonprobmpi.csv")
dfn4_vect_nonfused_probempi = pd.read_csv("n4_nonfused_vect_probmpi.csv")
dfn4_vect_nonfused_nonprobempi = pd.read_csv("n4_nonfused_vect_nonprobmpi.csv")




# MACRO CASES

## 1) macro_fused_probempi

### Adding Columns

In [3]:
dfn4_macro_fused_probempi["N_Nodes"] = 4
dfn4_macro_fused_probempi['NPX'] = (dfn4_macro_fused_probempi.GLB_X//dfn4_macro_fused_probempi.LOC_X)
dfn4_macro_fused_probempi['NPY'] = (dfn4_macro_fused_probempi.GLB_Y//dfn4_macro_fused_probempi.LOC_Y)
dfn4_macro_fused_probempi['NPZ'] = (dfn4_macro_fused_probempi.GLB_Z//dfn4_macro_fused_probempi.LOC_Z)
dfn4_macro_fused_probempi['NPT'] = (dfn4_macro_fused_probempi.GLB_T//dfn4_macro_fused_probempi.LOC_T)
dfn4_macro_fused_probempi["Total_proc"] = dfn4_macro_fused_probempi["omp_thrd"]* dfn4_macro_fused_probempi.MPI_size
# dfn4_macro_fused_probempi.columns

### Data Summary

In [4]:
dfn4_macro_fused_probempi_summary = dfn4_macro_fused_probempi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn4_macro_fused_probempi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,32,16,4,2,2,2,512,137
1,56,9,7,2,2,2,504,40
2,64,8,8,2,2,2,512,69
3,72,7,9,2,2,2,504,31
4,128,4,16,2,2,2,512,35
5,168,3,21,2,2,2,504,14
6,248,2,31,2,2,2,496,9
7,256,2,32,2,2,2,512,16
8,488,1,61,2,2,2,488,4
9,496,1,62,2,2,2,496,7


### Cases Setup

In [5]:
dfn4MacFusProb_32mpi_16omp = dfn4_macro_fused_probempi[(dfn4_macro_fused_probempi.omp_thrd == 16) & (dfn4_macro_fused_probempi.GLB_T/dfn4_macro_fused_probempi.LOC_T == 4) & (dfn4_macro_fused_probempi.GLB_X/dfn4_macro_fused_probempi.LOC_X == 2) & (dfn4_macro_fused_probempi.GLB_Y/dfn4_macro_fused_probempi.LOC_Y == 2)& (dfn4_macro_fused_probempi.GLB_Z/dfn4_macro_fused_probempi.LOC_Z == 2)]
dfn4MacFusProb_64mpi_8omp = dfn4_macro_fused_probempi[(dfn4_macro_fused_probempi.omp_thrd == 8) & (dfn4_macro_fused_probempi.GLB_T/dfn4_macro_fused_probempi.LOC_T == 8) & (dfn4_macro_fused_probempi.GLB_X/dfn4_macro_fused_probempi.LOC_X == 2) & (dfn4_macro_fused_probempi.GLB_Y/dfn4_macro_fused_probempi.LOC_Y == 2)& (dfn4_macro_fused_probempi.GLB_Z/dfn4_macro_fused_probempi.LOC_Z == 2)]
dfn4MacFusProb_128mpi_4omp = dfn4_macro_fused_probempi[(dfn4_macro_fused_probempi.omp_thrd == 4) & (dfn4_macro_fused_probempi.GLB_T/dfn4_macro_fused_probempi.LOC_T == 16) & (dfn4_macro_fused_probempi.GLB_X/dfn4_macro_fused_probempi.LOC_X == 2) & (dfn4_macro_fused_probempi.GLB_Y/dfn4_macro_fused_probempi.LOC_Y == 2)& (dfn4_macro_fused_probempi.GLB_Z/dfn4_macro_fused_probempi.LOC_Z == 2)]
dfn4MacFusProb_256mpi_2omp = dfn4_macro_fused_probempi[(dfn4_macro_fused_probempi.omp_thrd == 2) & (dfn4_macro_fused_probempi.GLB_T/dfn4_macro_fused_probempi.LOC_T == 32) & (dfn4_macro_fused_probempi.GLB_X/dfn4_macro_fused_probempi.LOC_X == 2) & (dfn4_macro_fused_probempi.GLB_Y/dfn4_macro_fused_probempi.LOC_Y == 2)& (dfn4_macro_fused_probempi.GLB_Z/dfn4_macro_fused_probempi.LOC_Z == 2)]
dfn4MacFusProb_512mpi_1omp = dfn4_macro_fused_probempi[(dfn4_macro_fused_probempi.omp_thrd == 1) & (dfn4_macro_fused_probempi.GLB_T/dfn4_macro_fused_probempi.LOC_T == 64) & (dfn4_macro_fused_probempi.GLB_X/dfn4_macro_fused_probempi.LOC_X == 2) & (dfn4_macro_fused_probempi.GLB_Y/dfn4_macro_fused_probempi.LOC_Y == 2)& (dfn4_macro_fused_probempi.GLB_Z/dfn4_macro_fused_probempi.LOC_Z == 2)]
dfn4MacFusProb_56mpi_9omp = dfn4_macro_fused_probempi[(dfn4_macro_fused_probempi.omp_thrd == 9) & (dfn4_macro_fused_probempi.GLB_T/dfn4_macro_fused_probempi.LOC_T == 7) & (dfn4_macro_fused_probempi.GLB_X/dfn4_macro_fused_probempi.LOC_X == 2) & (dfn4_macro_fused_probempi.GLB_Y/dfn4_macro_fused_probempi.LOC_Y == 2)& (dfn4_macro_fused_probempi.GLB_Z/dfn4_macro_fused_probempi.LOC_Z == 2)]
dfn4MacFusProb_248mpi_2omp = dfn4_macro_fused_probempi[(dfn4_macro_fused_probempi.omp_thrd == 2) & (dfn4_macro_fused_probempi.GLB_T/dfn4_macro_fused_probempi.LOC_T == 31) & (dfn4_macro_fused_probempi.GLB_X/dfn4_macro_fused_probempi.LOC_X == 2) & (dfn4_macro_fused_probempi.GLB_Y/dfn4_macro_fused_probempi.LOC_Y == 2)& (dfn4_macro_fused_probempi.GLB_Z/dfn4_macro_fused_probempi.LOC_Z == 2)]


## 2) Fused_Macro_nonprobempi

### Adding Columns

In [6]:
dfn4_macro_fused_nonprobempi["N_Nodes"] = 4
dfn4_macro_fused_nonprobempi['NPX'] = (dfn4_macro_fused_nonprobempi.GLB_X//dfn4_macro_fused_nonprobempi.LOC_X)
dfn4_macro_fused_nonprobempi['NPY'] = (dfn4_macro_fused_nonprobempi.GLB_Y//dfn4_macro_fused_nonprobempi.LOC_Y)
dfn4_macro_fused_nonprobempi['NPZ'] = (dfn4_macro_fused_nonprobempi.GLB_Z//dfn4_macro_fused_nonprobempi.LOC_Z)
dfn4_macro_fused_nonprobempi['NPT'] = (dfn4_macro_fused_nonprobempi.GLB_T//dfn4_macro_fused_nonprobempi.LOC_T)
dfn4_macro_fused_nonprobempi["Total_proc"] = dfn4_macro_fused_nonprobempi["omp_thrd"]* dfn4_macro_fused_nonprobempi.MPI_size
# dfn4_macro_fused_nonprobempi.columns

### Data Summary

In [7]:
dfn4_macro_fused_nonprobempi_summary =dfn4_macro_fused_nonprobempi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn4_macro_fused_nonprobempi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,32,16,4,2,2,2,512,138
1,56,9,7,2,2,2,504,39
2,64,8,8,2,2,2,512,70
3,72,7,9,2,2,2,504,30
4,128,4,16,2,2,2,512,34
5,168,3,21,2,2,2,504,14
6,248,2,31,2,2,2,496,8
7,256,2,32,2,2,2,512,17
8,488,1,61,2,2,2,488,4
9,496,1,62,2,2,2,496,7


### Cases Setup

In [8]:
dfn4MacFusNonProb_32mpi_16omp = dfn4_macro_fused_nonprobempi[(dfn4_macro_fused_nonprobempi.omp_thrd == 16) & (dfn4_macro_fused_nonprobempi.GLB_T/dfn4_macro_fused_nonprobempi.LOC_T == 4) & (dfn4_macro_fused_nonprobempi.GLB_X/dfn4_macro_fused_nonprobempi.LOC_X == 2) & (dfn4_macro_fused_nonprobempi.GLB_Y/dfn4_macro_fused_nonprobempi.LOC_Y == 2)& (dfn4_macro_fused_nonprobempi.GLB_Z/dfn4_macro_fused_nonprobempi.LOC_Z == 2)]
dfn4MacFusNonProb_64mpi_8omp = dfn4_macro_fused_nonprobempi[(dfn4_macro_fused_nonprobempi.omp_thrd == 8) & (dfn4_macro_fused_nonprobempi.GLB_T/dfn4_macro_fused_nonprobempi.LOC_T == 8) & (dfn4_macro_fused_nonprobempi.GLB_X/dfn4_macro_fused_nonprobempi.LOC_X == 2) & (dfn4_macro_fused_nonprobempi.GLB_Y/dfn4_macro_fused_nonprobempi.LOC_Y == 2)& (dfn4_macro_fused_nonprobempi.GLB_Z/dfn4_macro_fused_nonprobempi.LOC_Z == 2)]
dfn4MacFusnonProb_128mpi_4omp = dfn4_macro_fused_nonprobempi[(dfn4_macro_fused_nonprobempi.omp_thrd == 4) & (dfn4_macro_fused_nonprobempi.GLB_T/dfn4_macro_fused_nonprobempi.LOC_T == 16) & (dfn4_macro_fused_nonprobempi.GLB_X/dfn4_macro_fused_nonprobempi.LOC_X == 2) & (dfn4_macro_fused_nonprobempi.GLB_Y/dfn4_macro_fused_nonprobempi.LOC_Y == 2)& (dfn4_macro_fused_nonprobempi.GLB_Z/dfn4_macro_fused_nonprobempi.LOC_Z == 2)]
dfn4MacFusnonProb_256mpi_2omp = dfn4_macro_fused_nonprobempi[(dfn4_macro_fused_nonprobempi.omp_thrd == 2) & (dfn4_macro_fused_nonprobempi.GLB_T/dfn4_macro_fused_nonprobempi.LOC_T == 32) & (dfn4_macro_fused_nonprobempi.GLB_X/dfn4_macro_fused_nonprobempi.LOC_X == 2) & (dfn4_macro_fused_nonprobempi.GLB_Y/dfn4_macro_fused_nonprobempi.LOC_Y == 2)& (dfn4_macro_fused_nonprobempi.GLB_Z/dfn4_macro_fused_nonprobempi.LOC_Z == 2)]
dfn4MacFusnonProb_512mpi_1omp = dfn4_macro_fused_nonprobempi[(dfn4_macro_fused_nonprobempi.omp_thrd == 1) & (dfn4_macro_fused_nonprobempi.GLB_T/dfn4_macro_fused_nonprobempi.LOC_T == 64) & (dfn4_macro_fused_nonprobempi.GLB_X/dfn4_macro_fused_nonprobempi.LOC_X == 2) & (dfn4_macro_fused_nonprobempi.GLB_Y/dfn4_macro_fused_nonprobempi.LOC_Y == 2)& (dfn4_macro_fused_nonprobempi.GLB_Z/dfn4_macro_fused_nonprobempi.LOC_Z == 2)]
dfn4MacFusnonProb_56mpi_9omp = dfn4_macro_fused_nonprobempi[(dfn4_macro_fused_nonprobempi.omp_thrd == 9) & (dfn4_macro_fused_nonprobempi.GLB_T/dfn4_macro_fused_nonprobempi.LOC_T == 7) & (dfn4_macro_fused_nonprobempi.GLB_X/dfn4_macro_fused_nonprobempi.LOC_X == 2) & (dfn4_macro_fused_nonprobempi.GLB_Y/dfn4_macro_fused_nonprobempi.LOC_Y == 2)& (dfn4_macro_fused_nonprobempi.GLB_Z/dfn4_macro_fused_nonprobempi.LOC_Z == 2)]
dfn4MacFusnonProb_248mpi_2omp = dfn4_macro_fused_nonprobempi[(dfn4_macro_fused_nonprobempi.omp_thrd == 2) & (dfn4_macro_fused_nonprobempi.GLB_T/dfn4_macro_fused_nonprobempi.LOC_T == 31) & (dfn4_macro_fused_nonprobempi.GLB_X/dfn4_macro_fused_nonprobempi.LOC_X == 2) & (dfn4_macro_fused_nonprobempi.GLB_Y/dfn4_macro_fused_nonprobempi.LOC_Y == 2)& (dfn4_macro_fused_nonprobempi.GLB_Z/dfn4_macro_fused_nonprobempi.LOC_Z == 2)]


## GFLOPS - Fused Macro: Probempi vs NonProbempi

In [9]:
fig = go.Figure() 

# Case 1
fig.add_trace(go.Scatter(x=(dfn4MacFusProb_512mpi_1omp.Dirc_Data_Mov*(dfn4MacFusProb_512mpi_1omp.MPI_size/4)), 
                         y=(dfn4MacFusProb_512mpi_1omp.GFLOPS*(dfn4MacFusProb_512mpi_1omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacFusProb_512mpi_1omp',
                         line=dict(color='darkcyan',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))

fig.add_trace(go.Scatter(x=(dfn4MacFusnonProb_512mpi_1omp.Dirc_Data_Mov*(dfn4MacFusnonProb_512mpi_1omp.MPI_size/4)), 
                         y=(dfn4MacFusnonProb_512mpi_1omp.GFLOPS*(dfn4MacFusnonProb_512mpi_1omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacFusnonProb_512mpi_1omp',
                         line=dict(color='magenta',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))


# Case 2
fig.add_trace(go.Scatter(x=(dfn4MacFusProb_256mpi_2omp.Dirc_Data_Mov*(dfn4MacFusProb_256mpi_2omp.MPI_size/4)), 
                         y=(dfn4MacFusProb_256mpi_2omp.GFLOPS*(dfn4MacFusProb_256mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacFusProb_256mpi_2omp',
                         line=dict(color='royalblue',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))

fig.add_trace(go.Scatter(x=(dfn4MacFusnonProb_256mpi_2omp.Dirc_Data_Mov*(dfn4MacFusnonProb_256mpi_2omp.MPI_size/4)), 
                         y=(dfn4MacFusnonProb_256mpi_2omp.GFLOPS*(dfn4MacFusnonProb_256mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacFusnonProb_256mpi_2omp',
                         line=dict(color='sandybrown',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))


# Case 3
fig.add_trace(go.Scatter(x=(dfn4MacFusProb_128mpi_4omp.Dirc_Data_Mov*(dfn4MacFusProb_128mpi_4omp.MPI_size/4)), 
                         y=(dfn4MacFusProb_128mpi_4omp.GFLOPS*(dfn4MacFusProb_128mpi_4omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacFusProb_128mpi_4omp',
                         line=dict(color='darkblue',width=3),
                         marker=dict(symbol='circle',size=7, line=dict(color='tomato',width=2))))

fig.add_trace(go.Scatter(x=(dfn4MacFusnonProb_128mpi_4omp.Dirc_Data_Mov*(dfn4MacFusnonProb_128mpi_4omp.MPI_size/4)), 
                         y=(dfn4MacFusnonProb_128mpi_4omp.GFLOPS*(dfn4MacFusnonProb_128mpi_4omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacFusnonProb_128mpi_4omp',
                         line=dict(color='cyan',width=3),
                         marker=dict(symbol='circle',size=7,line=dict(color='Purple',width=2))))



# Case 4
fig.add_trace(go.Scatter(x=(dfn4MacFusProb_64mpi_8omp.Dirc_Data_Mov*(dfn4MacFusProb_64mpi_8omp.MPI_size/4)), 
                         y=(dfn4MacFusProb_64mpi_8omp.GFLOPS*(dfn4MacFusProb_64mpi_8omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacFusProb_64mpi_8omp',
                         line=dict(color='blue',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))

fig.add_trace(go.Scatter(x=(dfn4MacFusNonProb_64mpi_8omp.Dirc_Data_Mov*(dfn4MacFusNonProb_64mpi_8omp.MPI_size/4)), 
                         y=(dfn4MacFusNonProb_64mpi_8omp.GFLOPS*(dfn4MacFusNonProb_64mpi_8omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacFusNonProb_64mpi_8omp',
                         line=dict(color='red',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))


# Case 5
fig.add_trace(go.Scatter(x=(dfn4MacFusProb_32mpi_16omp.Dirc_Data_Mov*(dfn4MacFusProb_32mpi_16omp.MPI_size/4)), 
                         y=(dfn4MacFusProb_32mpi_16omp.GFLOPS*(dfn4MacFusProb_32mpi_16omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacFusProb_32mpi_16omp',
                         line=dict(color='chartreuse',width=3),
                         marker=dict(symbol='square-dot',size=6, line=dict(color='mediumvioletred',width=1))))

fig.add_trace(go.Scatter(x=(dfn4MacFusNonProb_32mpi_16omp.Dirc_Data_Mov*(dfn4MacFusNonProb_32mpi_16omp.MPI_size/4)), 
                         y=(dfn4MacFusNonProb_32mpi_16omp.GFLOPS*(dfn4MacFusNonProb_32mpi_16omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacFusNonProb_32mpi_16omp',
                         line=dict(color='black',width=3),
                         marker=dict(symbol='square-dot',size=6, line=dict(color='mediumvioletred',width=1))))


# Case 6
fig.add_trace(go.Scatter(x=(dfn4MacFusProb_56mpi_9omp.Dirc_Data_Mov*(dfn4MacFusProb_56mpi_9omp.MPI_size/4)), 
                         y=(dfn4MacFusProb_56mpi_9omp.GFLOPS*(dfn4MacFusProb_56mpi_9omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacFusProb_56mpi_9omp',
                         line=dict(color='gold',width=3),
                         marker=dict(symbol='triangle-up',size=6, line=dict(color='black',width=1))))

fig.add_trace(go.Scatter(x=(dfn4MacFusnonProb_56mpi_9omp.Dirc_Data_Mov*(dfn4MacFusnonProb_56mpi_9omp.MPI_size/4)), 
                         y=(dfn4MacFusnonProb_56mpi_9omp.GFLOPS*(dfn4MacFusnonProb_56mpi_9omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacFusnonProb_56mpi_9omp',
                         line=dict(color='dimgray',width=3),
                         marker=dict(symbol='triangle-up',size=6, line=dict(color='black',width=1))))



# Case 7
fig.add_trace(go.Scatter(x=(dfn4MacFusProb_248mpi_2omp.Dirc_Data_Mov*(dfn4MacFusProb_248mpi_2omp.MPI_size/4)), 
                         y=(dfn4MacFusProb_248mpi_2omp.GFLOPS*(dfn4MacFusProb_248mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacFusProb_248mpi_2omp',
                         line=dict(color='olive',width=3),
                         marker=dict(symbol='circle',size=6, line=dict(color='olive',width=2))))

fig.add_trace(go.Scatter(x=(dfn4MacFusnonProb_248mpi_2omp.Dirc_Data_Mov*(dfn4MacFusnonProb_248mpi_2omp.MPI_size/4)), 
                         y=(dfn4MacFusnonProb_248mpi_2omp.GFLOPS*(dfn4MacFusnonProb_248mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacFusnonProb_248mpi_2omp',
                         line=dict(color='Brown',width=3),
                         marker=dict(symbol='circle',size=6, line=dict(color='darkred',width=2))))




# Adding a vertical line
fig.add_vline(x=512e6, line_width=3, line_dash="dash", line_color="green")
# fig.add_hline(y=1024, line_width=3, line_dash="dash", line_color="red")

# Add annotation
fig.add_annotation(
    x='512e6'
    , y=130
    , text=f'LLC Size Per Node'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="purple", family="Sans Serif")
    , align="left"
    ,)

fig.update_layout(title='"Fused-Macro (4-Nodes): probempi vs nonprobempi" GFlops Measurement',
                  xaxis_title='Dirc_Data_Mov', yaxis_title='GFLOPs/Node', 
                  xaxis=dict(ticks="outside"),  
                  yaxis=dict(ticks="outside"))

## BANDWIDTH - Fused Macro: Probempi vs NonProbempi

In [10]:
fig = go.Figure() 

# Case 1
fig.add_trace(go.Scatter(x=(dfn4MacFusProb_512mpi_1omp.Dirc_Data_Mov*(dfn4MacFusProb_512mpi_1omp.MPI_size/4)), 
                         y=(dfn4MacFusProb_512mpi_1omp.BAND*(dfn4MacFusProb_512mpi_1omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacFusProb_512mpi_1omp',
                         line=dict(color='darkcyan',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))

fig.add_trace(go.Scatter(x=(dfn4MacFusnonProb_512mpi_1omp.Dirc_Data_Mov*(dfn4MacFusnonProb_512mpi_1omp.MPI_size/4)), 
                         y=(dfn4MacFusnonProb_512mpi_1omp.BAND*(dfn4MacFusnonProb_512mpi_1omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacFusnonProb_512mpi_1omp',
                         line=dict(color='magenta',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))


# Case 2
fig.add_trace(go.Scatter(x=(dfn4MacFusProb_256mpi_2omp.Dirc_Data_Mov*(dfn4MacFusProb_256mpi_2omp.MPI_size/4)), 
                         y=(dfn4MacFusProb_256mpi_2omp.BAND*(dfn4MacFusProb_256mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacFusProb_256mpi_2omp',
                         line=dict(color='royalblue',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))

fig.add_trace(go.Scatter(x=(dfn4MacFusnonProb_256mpi_2omp.Dirc_Data_Mov*(dfn4MacFusnonProb_256mpi_2omp.MPI_size/4)), 
                         y=(dfn4MacFusnonProb_256mpi_2omp.BAND*(dfn4MacFusnonProb_256mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacFusnonProb_256mpi_2omp',
                         line=dict(color='sandybrown',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))


# Case 3
fig.add_trace(go.Scatter(x=(dfn4MacFusProb_128mpi_4omp.Dirc_Data_Mov*(dfn4MacFusProb_128mpi_4omp.MPI_size/4)), 
                         y=(dfn4MacFusProb_128mpi_4omp.BAND*(dfn4MacFusProb_128mpi_4omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacFusProb_128mpi_4omp',
                         line=dict(color='darkblue',width=3),
                         marker=dict(symbol='circle',size=7, line=dict(color='tomato',width=2))))

fig.add_trace(go.Scatter(x=(dfn4MacFusnonProb_128mpi_4omp.Dirc_Data_Mov*(dfn4MacFusnonProb_128mpi_4omp.MPI_size/4)), 
                         y=(dfn4MacFusnonProb_128mpi_4omp.BAND*(dfn4MacFusnonProb_128mpi_4omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacFusnonProb_128mpi_4omp',
                         line=dict(color='cyan',width=3),
                         marker=dict(symbol='circle',size=7,line=dict(color='Purple',width=2))))



# Case 4
fig.add_trace(go.Scatter(x=(dfn4MacFusProb_64mpi_8omp.Dirc_Data_Mov*(dfn4MacFusProb_64mpi_8omp.MPI_size/4)), 
                         y=(dfn4MacFusProb_64mpi_8omp.BAND*(dfn4MacFusProb_64mpi_8omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacFusProb_64mpi_8omp',
                         line=dict(color='blue',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))

fig.add_trace(go.Scatter(x=(dfn4MacFusNonProb_64mpi_8omp.Dirc_Data_Mov*(dfn4MacFusNonProb_64mpi_8omp.MPI_size/4)), 
                         y=(dfn4MacFusNonProb_64mpi_8omp.BAND*(dfn4MacFusNonProb_64mpi_8omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacFusNonProb_64mpi_8omp',
                         line=dict(color='red',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))


# Case 5
fig.add_trace(go.Scatter(x=(dfn4MacFusProb_32mpi_16omp.Dirc_Data_Mov*(dfn4MacFusProb_32mpi_16omp.MPI_size/4)), 
                         y=(dfn4MacFusProb_32mpi_16omp.BAND*(dfn4MacFusProb_32mpi_16omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacFusProb_32mpi_16omp',
                         line=dict(color='chartreuse',width=3),
                         marker=dict(symbol='square-dot',size=6, line=dict(color='mediumvioletred',width=1))))

fig.add_trace(go.Scatter(x=(dfn4MacFusNonProb_32mpi_16omp.Dirc_Data_Mov*(dfn4MacFusNonProb_32mpi_16omp.MPI_size/4)), 
                         y=(dfn4MacFusNonProb_32mpi_16omp.BAND*(dfn4MacFusNonProb_32mpi_16omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacFusNonProb_32mpi_16omp',
                         line=dict(color='black',width=3),
                         marker=dict(symbol='square-dot',size=6, line=dict(color='mediumvioletred',width=1))))


# Case 6
fig.add_trace(go.Scatter(x=(dfn4MacFusProb_56mpi_9omp.Dirc_Data_Mov*(dfn4MacFusProb_56mpi_9omp.MPI_size/4)), 
                         y=(dfn4MacFusProb_56mpi_9omp.BAND*(dfn4MacFusProb_56mpi_9omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacFusProb_56mpi_9omp',
                         line=dict(color='gold',width=3),
                         marker=dict(symbol='triangle-up',size=6, line=dict(color='black',width=1))))

fig.add_trace(go.Scatter(x=(dfn4MacFusnonProb_56mpi_9omp.Dirc_Data_Mov*(dfn4MacFusnonProb_56mpi_9omp.MPI_size/4)), 
                         y=(dfn4MacFusnonProb_56mpi_9omp.BAND*(dfn4MacFusnonProb_56mpi_9omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacFusnonProb_56mpi_9omp',
                         line=dict(color='dimgray',width=3),
                         marker=dict(symbol='triangle-up',size=6, line=dict(color='black',width=1))))



# Case 7
fig.add_trace(go.Scatter(x=(dfn4MacFusProb_248mpi_2omp.Dirc_Data_Mov*(dfn4MacFusProb_248mpi_2omp.MPI_size/4)), 
                         y=(dfn4MacFusProb_248mpi_2omp.BAND*(dfn4MacFusProb_248mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacFusProb_248mpi_2omp',
                         line=dict(color='olive',width=3),
                         marker=dict(symbol='circle',size=6, line=dict(color='olive',width=2))))

fig.add_trace(go.Scatter(x=(dfn4MacFusnonProb_248mpi_2omp.Dirc_Data_Mov*(dfn4MacFusnonProb_248mpi_2omp.MPI_size/4)), 
                         y=(dfn4MacFusnonProb_248mpi_2omp.BAND*(dfn4MacFusnonProb_248mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacFusnonProb_248mpi_2omp',
                         line=dict(color='Brown',width=3),
                         marker=dict(symbol='circle',size=6, line=dict(color='darkred',width=2))))



# Adding vertical and horizontal lines 
fig.add_vline(x=512e6, line_width=3, line_dash="dot", line_color="green")
fig.add_hline(y=409, line_width=3, line_dash="dash", line_color="red")

# Add annotation with arrow
fig.add_annotation(
    x='512e6'
    , y=280
    , text=f'LLC Size Per Node: 512M'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="purple", family="Sans Serif")
    , align="left"
    ,)

fig.add_annotation(
    x='130e6'
    , y=370
    , text=f'Max Bandwidth Per Node'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="orange", family="Sans Serif")
    , align="left"
    ,)


fig.update_layout(title='"Fused-Macro: Probmpi vs Non-Probempi" Bandwidth Measurement',
                  xaxis_title='Dirc_Data_Mov', yaxis_title='Bandwidth/Node', 
                  xaxis=dict(ticks="outside"),  
                  yaxis=dict(ticks="outside"),
                  legend=go.layout.Legend(bordercolor="black",borderwidth=1)) 



## 3) Nonfused_Macro_Probempi

### Adding Columns

In [11]:
dfn4_macro_nonfused_probempi["N_Nodes"] = 4
dfn4_macro_nonfused_probempi['NPX'] = (dfn4_macro_nonfused_probempi.GLB_X//dfn4_macro_nonfused_probempi.LOC_X)
dfn4_macro_nonfused_probempi['NPY'] = (dfn4_macro_nonfused_probempi.GLB_Y//dfn4_macro_nonfused_probempi.LOC_Y)
dfn4_macro_nonfused_probempi['NPZ'] = (dfn4_macro_nonfused_probempi.GLB_Z//dfn4_macro_nonfused_probempi.LOC_Z)
dfn4_macro_nonfused_probempi['NPT'] = (dfn4_macro_nonfused_probempi.GLB_T//dfn4_macro_nonfused_probempi.LOC_T)
dfn4_macro_nonfused_probempi["Total_proc"] = dfn4_macro_nonfused_probempi["omp_thrd"]* dfn4_macro_nonfused_probempi.MPI_size
# dfn4_macro_nonfused_probempi.columns

### Data Summary

In [12]:
dfn4_macro_nonfused_probempi_summary = dfn4_macro_nonfused_probempi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn4_macro_nonfused_probempi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,32,16,4,2,2,2,512,99
1,56,9,7,2,2,2,504,39
2,64,8,8,2,2,2,512,68
3,72,7,9,2,2,2,504,31
4,128,4,16,2,2,2,512,33
5,168,3,21,2,2,2,504,13
6,248,2,31,2,2,2,496,9
7,256,2,32,2,2,2,512,16
8,488,1,61,2,2,2,488,4
9,496,1,62,2,2,2,496,7


### Cases Setup

In [13]:
dfn4MacNonFusProb_32mpi_16omp = dfn4_macro_nonfused_probempi[(dfn4_macro_nonfused_probempi.omp_thrd == 16) & (dfn4_macro_nonfused_probempi.GLB_T/dfn4_macro_nonfused_probempi.LOC_T == 4) & (dfn4_macro_nonfused_probempi.GLB_X/dfn4_macro_nonfused_probempi.LOC_X == 2) & (dfn4_macro_nonfused_probempi.GLB_Y/dfn4_macro_nonfused_probempi.LOC_Y == 2)& (dfn4_macro_nonfused_probempi.GLB_Z/dfn4_macro_nonfused_probempi.LOC_Z == 2)]
dfn4MacNonFusProb_64mpi_8omp = dfn4_macro_nonfused_probempi[(dfn4_macro_nonfused_probempi.omp_thrd == 8) & (dfn4_macro_nonfused_probempi.GLB_T/dfn4_macro_nonfused_probempi.LOC_T == 8) & (dfn4_macro_nonfused_probempi.GLB_X/dfn4_macro_nonfused_probempi.LOC_X == 2) & (dfn4_macro_nonfused_probempi.GLB_Y/dfn4_macro_nonfused_probempi.LOC_Y == 2)& (dfn4_macro_nonfused_probempi.GLB_Z/dfn4_macro_nonfused_probempi.LOC_Z == 2)]
dfn4MacNonFusProb_128mpi_4omp = dfn4_macro_nonfused_probempi[(dfn4_macro_nonfused_probempi.omp_thrd == 4) & (dfn4_macro_nonfused_probempi.GLB_T/dfn4_macro_nonfused_probempi.LOC_T == 16) & (dfn4_macro_nonfused_probempi.GLB_X/dfn4_macro_nonfused_probempi.LOC_X == 2) & (dfn4_macro_nonfused_probempi.GLB_Y/dfn4_macro_nonfused_probempi.LOC_Y == 2)& (dfn4_macro_nonfused_probempi.GLB_Z/dfn4_macro_nonfused_probempi.LOC_Z == 2)]
dfn4MacNonFusProb_256mpi_2omp = dfn4_macro_nonfused_probempi[(dfn4_macro_nonfused_probempi.omp_thrd == 2) & (dfn4_macro_nonfused_probempi.GLB_T/dfn4_macro_nonfused_probempi.LOC_T == 32) & (dfn4_macro_nonfused_probempi.GLB_X/dfn4_macro_nonfused_probempi.LOC_X == 2) & (dfn4_macro_nonfused_probempi.GLB_Y/dfn4_macro_nonfused_probempi.LOC_Y == 2)& (dfn4_macro_nonfused_probempi.GLB_Z/dfn4_macro_nonfused_probempi.LOC_Z == 2)]
dfn4MacNonFusProb_512mpi_1omp = dfn4_macro_nonfused_probempi[(dfn4_macro_nonfused_probempi.omp_thrd == 1) & (dfn4_macro_nonfused_probempi.GLB_T/dfn4_macro_nonfused_probempi.LOC_T == 64) & (dfn4_macro_nonfused_probempi.GLB_X/dfn4_macro_nonfused_probempi.LOC_X == 2) & (dfn4_macro_nonfused_probempi.GLB_Y/dfn4_macro_nonfused_probempi.LOC_Y == 2)& (dfn4_macro_nonfused_probempi.GLB_Z/dfn4_macro_nonfused_probempi.LOC_Z == 2)]
dfn4MacNonFusProb_56mpi_9omp = dfn4_macro_nonfused_probempi[(dfn4_macro_nonfused_probempi.omp_thrd == 9) & (dfn4_macro_nonfused_probempi.GLB_T/dfn4_macro_nonfused_probempi.LOC_T == 7) & (dfn4_macro_nonfused_probempi.GLB_X/dfn4_macro_nonfused_probempi.LOC_X == 2) & (dfn4_macro_nonfused_probempi.GLB_Y/dfn4_macro_nonfused_probempi.LOC_Y == 2)& (dfn4_macro_nonfused_probempi.GLB_Z/dfn4_macro_nonfused_probempi.LOC_Z == 2)]
dfn4MacNonFusProb_248mpi_2omp = dfn4_macro_nonfused_probempi[(dfn4_macro_nonfused_probempi.omp_thrd == 2) & (dfn4_macro_nonfused_probempi.GLB_T/dfn4_macro_nonfused_probempi.LOC_T == 31) & (dfn4_macro_nonfused_probempi.GLB_X/dfn4_macro_nonfused_probempi.LOC_X == 2) & (dfn4_macro_nonfused_probempi.GLB_Y/dfn4_macro_nonfused_probempi.LOC_Y == 2)& (dfn4_macro_nonfused_probempi.GLB_Z/dfn4_macro_nonfused_probempi.LOC_Z == 2)]


## 4) Nonfused_Macro_NonProbempi

### Adding Columns

In [14]:
dfn4_macro_nonfused_nonprobempi["N_Nodes"] = 4
dfn4_macro_nonfused_nonprobempi['NPX'] = (dfn4_macro_nonfused_nonprobempi.GLB_X//dfn4_macro_nonfused_nonprobempi.LOC_X)
dfn4_macro_nonfused_nonprobempi['NPY'] = (dfn4_macro_nonfused_nonprobempi.GLB_Y//dfn4_macro_nonfused_nonprobempi.LOC_Y)
dfn4_macro_nonfused_nonprobempi['NPZ'] = (dfn4_macro_nonfused_nonprobempi.GLB_Z//dfn4_macro_nonfused_nonprobempi.LOC_Z)
dfn4_macro_nonfused_nonprobempi['NPT'] = (dfn4_macro_nonfused_nonprobempi.GLB_T//dfn4_macro_nonfused_nonprobempi.LOC_T)
dfn4_macro_nonfused_nonprobempi["Total_proc"] = dfn4_macro_nonfused_nonprobempi["omp_thrd"]* dfn4_macro_nonfused_nonprobempi.MPI_size
# dfn4_macro_nonfused_nonprobempi.columns

### Data Summary

In [15]:
dfn4_macro_nonfused_nonprobempi_summary = dfn4_macro_nonfused_nonprobempi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn4_macro_nonfused_nonprobempi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,32,16,4,2,2,2,512,98
1,56,9,7,2,2,2,504,39
2,64,8,8,2,2,2,512,70
3,72,7,9,2,2,2,504,31
4,128,4,16,2,2,2,512,35
5,168,3,21,2,2,2,504,14
6,248,2,31,2,2,2,496,8
7,256,2,32,2,2,2,512,17
8,488,1,61,2,2,2,488,4
9,496,1,62,2,2,2,496,7


### Cases Setup

In [16]:
dfn4MacNonFusNonProb_32mpi_16omp = dfn4_macro_nonfused_nonprobempi[(dfn4_macro_nonfused_nonprobempi.omp_thrd == 16) & (dfn4_macro_nonfused_nonprobempi.GLB_T/dfn4_macro_nonfused_nonprobempi.LOC_T == 4) & (dfn4_macro_nonfused_nonprobempi.GLB_X/dfn4_macro_nonfused_nonprobempi.LOC_X == 2) & (dfn4_macro_nonfused_nonprobempi.GLB_Y/dfn4_macro_nonfused_nonprobempi.LOC_Y == 2)& (dfn4_macro_nonfused_nonprobempi.GLB_Z/dfn4_macro_nonfused_nonprobempi.LOC_Z == 2)]
dfn4MacNonFusNonProb_64mpi_8omp = dfn4_macro_nonfused_nonprobempi[(dfn4_macro_nonfused_nonprobempi.omp_thrd == 8) & (dfn4_macro_nonfused_nonprobempi.GLB_T/dfn4_macro_nonfused_nonprobempi.LOC_T == 8) & (dfn4_macro_nonfused_nonprobempi.GLB_X/dfn4_macro_nonfused_nonprobempi.LOC_X == 2) & (dfn4_macro_nonfused_nonprobempi.GLB_Y/dfn4_macro_nonfused_nonprobempi.LOC_Y == 2)& (dfn4_macro_nonfused_nonprobempi.GLB_Z/dfn4_macro_nonfused_nonprobempi.LOC_Z == 2)]
dfn4MacNonFusNonProb_128mpi_4omp = dfn4_macro_nonfused_nonprobempi[(dfn4_macro_nonfused_nonprobempi.omp_thrd == 4) & (dfn4_macro_nonfused_nonprobempi.GLB_T/dfn4_macro_nonfused_nonprobempi.LOC_T == 16) & (dfn4_macro_nonfused_nonprobempi.GLB_X/dfn4_macro_nonfused_nonprobempi.LOC_X == 2) & (dfn4_macro_nonfused_nonprobempi.GLB_Y/dfn4_macro_nonfused_nonprobempi.LOC_Y == 2)& (dfn4_macro_nonfused_nonprobempi.GLB_Z/dfn4_macro_nonfused_nonprobempi.LOC_Z == 2)]
dfn4MacNonFusNonProb_256mpi_2omp = dfn4_macro_nonfused_nonprobempi[(dfn4_macro_nonfused_nonprobempi.omp_thrd == 2) & (dfn4_macro_nonfused_nonprobempi.GLB_T/dfn4_macro_nonfused_nonprobempi.LOC_T == 32) & (dfn4_macro_nonfused_nonprobempi.GLB_X/dfn4_macro_nonfused_nonprobempi.LOC_X == 2) & (dfn4_macro_nonfused_nonprobempi.GLB_Y/dfn4_macro_nonfused_nonprobempi.LOC_Y == 2)& (dfn4_macro_nonfused_nonprobempi.GLB_Z/dfn4_macro_nonfused_nonprobempi.LOC_Z == 2)]
dfn4MacNonFusNonProb_512mpi_1omp = dfn4_macro_nonfused_nonprobempi[(dfn4_macro_nonfused_nonprobempi.omp_thrd == 1) & (dfn4_macro_nonfused_nonprobempi.GLB_T/dfn4_macro_nonfused_nonprobempi.LOC_T == 64) & (dfn4_macro_nonfused_nonprobempi.GLB_X/dfn4_macro_nonfused_nonprobempi.LOC_X == 2) & (dfn4_macro_nonfused_nonprobempi.GLB_Y/dfn4_macro_nonfused_nonprobempi.LOC_Y == 2)& (dfn4_macro_nonfused_nonprobempi.GLB_Z/dfn4_macro_nonfused_nonprobempi.LOC_Z == 2)]
dfn4MacNonFusNonProb_56mpi_9omp = dfn4_macro_nonfused_nonprobempi[(dfn4_macro_nonfused_nonprobempi.omp_thrd == 9) & (dfn4_macro_nonfused_nonprobempi.GLB_T/dfn4_macro_nonfused_nonprobempi.LOC_T == 7) & (dfn4_macro_nonfused_nonprobempi.GLB_X/dfn4_macro_nonfused_nonprobempi.LOC_X == 2) & (dfn4_macro_nonfused_nonprobempi.GLB_Y/dfn4_macro_nonfused_nonprobempi.LOC_Y == 2)& (dfn4_macro_nonfused_nonprobempi.GLB_Z/dfn4_macro_nonfused_nonprobempi.LOC_Z == 2)]
dfn4MacNonFusNonProb_248mpi_2omp = dfn4_macro_nonfused_nonprobempi[(dfn4_macro_nonfused_nonprobempi.omp_thrd == 2) & (dfn4_macro_nonfused_nonprobempi.GLB_T/dfn4_macro_nonfused_nonprobempi.LOC_T == 31) & (dfn4_macro_nonfused_nonprobempi.GLB_X/dfn4_macro_nonfused_nonprobempi.LOC_X == 2) & (dfn4_macro_nonfused_nonprobempi.GLB_Y/dfn4_macro_nonfused_nonprobempi.LOC_Y == 2)& (dfn4_macro_nonfused_nonprobempi.GLB_Z/dfn4_macro_nonfused_nonprobempi.LOC_Z == 2)]


## GFLOPS - Nonfused Macro: Probempi vs Nonprobempi

In [17]:
fig = go.Figure() 

# Case 1
fig.add_trace(go.Scatter(x=(dfn4MacNonFusProb_512mpi_1omp.Dirc_Data_Mov*(dfn4MacNonFusProb_512mpi_1omp.MPI_size/4)), 
                         y=(dfn4MacNonFusProb_512mpi_1omp.GFLOPS*(dfn4MacNonFusProb_512mpi_1omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacNonFusProb_512mpi_1omp',
                         line=dict(color='darkcyan',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))

fig.add_trace(go.Scatter(x=(dfn4MacNonFusNonProb_512mpi_1omp.Dirc_Data_Mov*(dfn4MacNonFusNonProb_512mpi_1omp.MPI_size/4)), 
                         y=(dfn4MacNonFusNonProb_512mpi_1omp.GFLOPS*(dfn4MacNonFusNonProb_512mpi_1omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacNonFusNonProb_512mpi_1omp',
                         line=dict(color='magenta',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))


# Case 2
fig.add_trace(go.Scatter(x=(dfn4MacNonFusProb_256mpi_2omp.Dirc_Data_Mov*(dfn4MacNonFusProb_256mpi_2omp.MPI_size/4)), 
                         y=(dfn4MacNonFusProb_256mpi_2omp.GFLOPS*(dfn4MacNonFusProb_256mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacNonFusProb_256mpi_2omp',
                         line=dict(color='royalblue',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))

fig.add_trace(go.Scatter(x=(dfn4MacNonFusNonProb_256mpi_2omp.Dirc_Data_Mov*(dfn4MacNonFusNonProb_256mpi_2omp.MPI_size/4)), 
                         y=(dfn4MacNonFusNonProb_256mpi_2omp.GFLOPS*(dfn4MacNonFusNonProb_256mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacNonFusNonProb_256mpi_2omp',
                         line=dict(color='sandybrown',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))


# Case 3
fig.add_trace(go.Scatter(x=(dfn4MacNonFusProb_128mpi_4omp.Dirc_Data_Mov*(dfn4MacNonFusProb_128mpi_4omp.MPI_size/4)), 
                         y=(dfn4MacNonFusProb_128mpi_4omp.GFLOPS*(dfn4MacNonFusProb_128mpi_4omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacNonFusProb_128mpi_4omp',
                         line=dict(color='darkblue',width=3),
                         marker=dict(symbol='circle',size=7, line=dict(color='tomato',width=2))))

fig.add_trace(go.Scatter(x=(dfn4MacNonFusNonProb_128mpi_4omp.Dirc_Data_Mov*(dfn4MacNonFusNonProb_128mpi_4omp.MPI_size/4)), 
                         y=(dfn4MacNonFusNonProb_128mpi_4omp.GFLOPS*(dfn4MacNonFusNonProb_128mpi_4omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacNonFusNonProb_128mpi_4omp',
                         line=dict(color='cyan',width=3),
                         marker=dict(symbol='circle',size=7,line=dict(color='Purple',width=2))))



# Case 4
fig.add_trace(go.Scatter(x=(dfn4MacNonFusProb_64mpi_8omp.Dirc_Data_Mov*(dfn4MacNonFusProb_64mpi_8omp.MPI_size/4)), 
                         y=(dfn4MacNonFusProb_64mpi_8omp.GFLOPS*(dfn4MacNonFusProb_64mpi_8omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacNonFusProb_64mpi_8omp',
                         line=dict(color='blue',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))

fig.add_trace(go.Scatter(x=(dfn4MacNonFusNonProb_64mpi_8omp.Dirc_Data_Mov*(dfn4MacNonFusNonProb_64mpi_8omp.MPI_size/4)), 
                         y=(dfn4MacNonFusNonProb_64mpi_8omp.GFLOPS*(dfn4MacNonFusNonProb_64mpi_8omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacNonFusNonProb_64mpi_8omp',
                         line=dict(color='red',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))


# Case 5
fig.add_trace(go.Scatter(x=(dfn4MacNonFusProb_32mpi_16omp.Dirc_Data_Mov*(dfn4MacNonFusProb_32mpi_16omp.MPI_size/4)), 
                         y=(dfn4MacNonFusProb_32mpi_16omp.GFLOPS*(dfn4MacNonFusProb_32mpi_16omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacNonFusProb_32mpi_16omp',
                         line=dict(color='chartreuse',width=3),
                         marker=dict(symbol='square-dot',size=6, line=dict(color='mediumvioletred',width=1))))

fig.add_trace(go.Scatter(x=(dfn4MacNonFusNonProb_32mpi_16omp.Dirc_Data_Mov*(dfn4MacNonFusNonProb_32mpi_16omp.MPI_size/4)), 
                         y=(dfn4MacNonFusNonProb_32mpi_16omp.GFLOPS*(dfn4MacNonFusNonProb_32mpi_16omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacNonFusNonProb_32mpi_16omp',
                         line=dict(color='black',width=3),
                         marker=dict(symbol='square-dot',size=6, line=dict(color='mediumvioletred',width=1))))


# Case 6
fig.add_trace(go.Scatter(x=(dfn4MacNonFusProb_56mpi_9omp.Dirc_Data_Mov*(dfn4MacNonFusProb_56mpi_9omp.MPI_size/4)), 
                         y=(dfn4MacNonFusProb_56mpi_9omp.GFLOPS*(dfn4MacNonFusProb_56mpi_9omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacNonFusProb_56mpi_9omp',
                         line=dict(color='gold',width=3),
                         marker=dict(symbol='triangle-up',size=6, line=dict(color='black',width=1))))

fig.add_trace(go.Scatter(x=(dfn4MacNonFusNonProb_56mpi_9omp.Dirc_Data_Mov*(dfn4MacNonFusNonProb_56mpi_9omp.MPI_size/4)), 
                         y=(dfn4MacNonFusNonProb_56mpi_9omp.GFLOPS*(dfn4MacNonFusNonProb_56mpi_9omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacNonFusNonProb_56mpi_9omp',
                         line=dict(color='dimgray',width=3),
                         marker=dict(symbol='triangle-up',size=6, line=dict(color='black',width=1))))



# Case 7
fig.add_trace(go.Scatter(x=(dfn4MacNonFusProb_248mpi_2omp.Dirc_Data_Mov*(dfn4MacNonFusProb_248mpi_2omp.MPI_size/4)), 
                         y=(dfn4MacNonFusProb_248mpi_2omp.GFLOPS*(dfn4MacNonFusProb_248mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacNonFusProb_248mpi_2omp',
                         line=dict(color='olive',width=3),
                         marker=dict(symbol='circle',size=6, line=dict(color='olive',width=2))))

fig.add_trace(go.Scatter(x=(dfn4MacNonFusNonProb_248mpi_2omp.Dirc_Data_Mov*(dfn4MacNonFusNonProb_248mpi_2omp.MPI_size/4)), 
                         y=(dfn4MacNonFusNonProb_248mpi_2omp.GFLOPS*(dfn4MacNonFusNonProb_248mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacNonFusNonProb_248mpi_2omp',
                         line=dict(color='Brown',width=3),
                         marker=dict(symbol='circle',size=6, line=dict(color='darkred',width=2))))




# Adding a vertical line
fig.add_vline(x=512e6, line_width=3, line_dash="dash", line_color="green")
# fig.add_hline(y=1024, line_width=3, line_dash="dash", line_color="red")

# Add annotation
fig.add_annotation(
    x='512e6'
    , y=130
    , text=f'LLC Size Per Node'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="purple", family="Sans Serif")
    , align="left"
    ,)

fig.update_layout(title='"NonFused-Macro (4-Nodes): probempi vs nonprobempi" GFlops Measurement',
                  xaxis_title='Dirc_Data_Mov', yaxis_title='GFLOPs/Node', 
                  xaxis=dict(ticks="outside"),  
                  yaxis=dict(ticks="outside"))

## BANDWIDTH - Nonfused Macro: Probempi vs Nonprobempi

In [18]:
fig = go.Figure() 

# Case 1
fig.add_trace(go.Scatter(x=(dfn4MacNonFusProb_512mpi_1omp.Dirc_Data_Mov*(dfn4MacNonFusProb_512mpi_1omp.MPI_size/4)), 
                         y=(dfn4MacNonFusProb_512mpi_1omp.BAND*(dfn4MacNonFusProb_512mpi_1omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacNonFusProb_512mpi_1omp',
                         line=dict(color='darkcyan',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))

fig.add_trace(go.Scatter(x=(dfn4MacNonFusNonProb_512mpi_1omp.Dirc_Data_Mov*(dfn4MacNonFusNonProb_512mpi_1omp.MPI_size/4)), 
                         y=(dfn4MacNonFusNonProb_512mpi_1omp.BAND*(dfn4MacNonFusNonProb_512mpi_1omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacNonFusNonProb_512mpi_1omp',
                         line=dict(color='magenta',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))


# Case 2
fig.add_trace(go.Scatter(x=(dfn4MacNonFusProb_256mpi_2omp.Dirc_Data_Mov*(dfn4MacNonFusProb_256mpi_2omp.MPI_size/4)), 
                         y=(dfn4MacNonFusProb_256mpi_2omp.BAND*(dfn4MacNonFusProb_256mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacNonFusProb_256mpi_2omp',
                         line=dict(color='royalblue',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))

fig.add_trace(go.Scatter(x=(dfn4MacNonFusNonProb_256mpi_2omp.Dirc_Data_Mov*(dfn4MacNonFusNonProb_256mpi_2omp.MPI_size/4)), 
                         y=(dfn4MacNonFusNonProb_256mpi_2omp.BAND*(dfn4MacNonFusNonProb_256mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacNonFusNonProb_256mpi_2omp',
                         line=dict(color='sandybrown',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))


# Case 3
fig.add_trace(go.Scatter(x=(dfn4MacNonFusProb_128mpi_4omp.Dirc_Data_Mov*(dfn4MacNonFusProb_128mpi_4omp.MPI_size/4)), 
                         y=(dfn4MacNonFusProb_128mpi_4omp.BAND*(dfn4MacNonFusProb_128mpi_4omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacNonFusProb_128mpi_4omp',
                         line=dict(color='darkblue',width=3),
                         marker=dict(symbol='circle',size=7, line=dict(color='tomato',width=2))))

fig.add_trace(go.Scatter(x=(dfn4MacNonFusNonProb_128mpi_4omp.Dirc_Data_Mov*(dfn4MacNonFusNonProb_128mpi_4omp.MPI_size/4)), 
                         y=(dfn4MacNonFusNonProb_128mpi_4omp.BAND*(dfn4MacNonFusNonProb_128mpi_4omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacNonFusNonProb_128mpi_4omp',
                         line=dict(color='cyan',width=3),
                         marker=dict(symbol='circle',size=7,line=dict(color='Purple',width=2))))



# Case 4
fig.add_trace(go.Scatter(x=(dfn4MacNonFusProb_64mpi_8omp.Dirc_Data_Mov*(dfn4MacNonFusProb_64mpi_8omp.MPI_size/4)), 
                         y=(dfn4MacNonFusProb_64mpi_8omp.BAND*(dfn4MacNonFusProb_64mpi_8omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacNonFusProb_64mpi_8omp',
                         line=dict(color='blue',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))

fig.add_trace(go.Scatter(x=(dfn4MacNonFusNonProb_64mpi_8omp.Dirc_Data_Mov*(dfn4MacNonFusNonProb_64mpi_8omp.MPI_size/4)), 
                         y=(dfn4MacNonFusNonProb_64mpi_8omp.BAND*(dfn4MacNonFusNonProb_64mpi_8omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacNonFusNonProb_64mpi_8omp',
                         line=dict(color='red',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))


# Case 5
fig.add_trace(go.Scatter(x=(dfn4MacNonFusProb_32mpi_16omp.Dirc_Data_Mov*(dfn4MacNonFusProb_32mpi_16omp.MPI_size/4)), 
                         y=(dfn4MacNonFusProb_32mpi_16omp.BAND*(dfn4MacNonFusProb_32mpi_16omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacNonFusProb_32mpi_16omp',
                         line=dict(color='chartreuse',width=3),
                         marker=dict(symbol='square-dot',size=6, line=dict(color='mediumvioletred',width=1))))

fig.add_trace(go.Scatter(x=(dfn4MacNonFusNonProb_32mpi_16omp.Dirc_Data_Mov*(dfn4MacNonFusNonProb_32mpi_16omp.MPI_size/4)), 
                         y=(dfn4MacNonFusNonProb_32mpi_16omp.BAND*(dfn4MacNonFusNonProb_32mpi_16omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacNonFusNonProb_32mpi_16omp',
                         line=dict(color='black',width=3),
                         marker=dict(symbol='square-dot',size=6, line=dict(color='mediumvioletred',width=1))))


# Case 6
fig.add_trace(go.Scatter(x=(dfn4MacNonFusProb_56mpi_9omp.Dirc_Data_Mov*(dfn4MacNonFusProb_56mpi_9omp.MPI_size/4)), 
                         y=(dfn4MacNonFusProb_56mpi_9omp.BAND*(dfn4MacNonFusProb_56mpi_9omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacNonFusProb_56mpi_9omp',
                         line=dict(color='gold',width=3),
                         marker=dict(symbol='triangle-up',size=6, line=dict(color='black',width=1))))

fig.add_trace(go.Scatter(x=(dfn4MacNonFusNonProb_56mpi_9omp.Dirc_Data_Mov*(dfn4MacNonFusNonProb_56mpi_9omp.MPI_size/4)), 
                         y=(dfn4MacNonFusNonProb_56mpi_9omp.BAND*(dfn4MacNonFusNonProb_56mpi_9omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacNonFusNonProb_56mpi_9omp',
                         line=dict(color='dimgray',width=3),
                         marker=dict(symbol='triangle-up',size=6, line=dict(color='black',width=1))))



# Case 7
fig.add_trace(go.Scatter(x=(dfn4MacNonFusProb_248mpi_2omp.Dirc_Data_Mov*(dfn4MacNonFusProb_248mpi_2omp.MPI_size/4)), 
                         y=(dfn4MacNonFusProb_248mpi_2omp.BAND*(dfn4MacNonFusProb_248mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacNonFusProb_248mpi_2omp',
                         line=dict(color='olive',width=3),
                         marker=dict(symbol='circle',size=6, line=dict(color='olive',width=2))))

fig.add_trace(go.Scatter(x=(dfn4MacNonFusNonProb_248mpi_2omp.Dirc_Data_Mov*(dfn4MacNonFusNonProb_248mpi_2omp.MPI_size/4)), 
                         y=(dfn4MacNonFusNonProb_248mpi_2omp.BAND*(dfn4MacNonFusNonProb_248mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacNonFusNonProb_248mpi_2omp',
                         line=dict(color='Brown',width=3),
                         marker=dict(symbol='circle',size=6, line=dict(color='darkred',width=2))))



# Adding vertical and horizontal lines 
fig.add_vline(x=512e6, line_width=3, line_dash="dot", line_color="green")
fig.add_hline(y=409, line_width=3, line_dash="dash", line_color="red")

# Add annotation with arrow
fig.add_annotation(
    x='512e6'
    , y=280
    , text=f'LLC Size Per Node: 512M'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="purple", family="Sans Serif")
    , align="left"
    ,)

fig.add_annotation(
    x='130e6'
    , y=370
    , text=f'Max Bandwidth Per Node'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="orange", family="Sans Serif")
    , align="left"
    ,)


fig.update_layout(title='"NonFused-Macro: Probmpi vs Non-Probempi" Bandwidth Measurement',
                  xaxis_title='Dirc_Data_Mov', yaxis_title='Bandwidth/Node', 
                  xaxis=dict(ticks="outside"),  
                  yaxis=dict(ticks="outside"),
                  legend=go.layout.Legend(bordercolor="black",borderwidth=1)) 





## 5) Newgeo_Macro_Probempi

### Adding Columns

In [19]:
dfn4_macro_newgeo_probempi["N_Nodes"] = 4
dfn4_macro_newgeo_probempi['NPX'] = (dfn4_macro_newgeo_probempi.GLB_X//dfn4_macro_newgeo_probempi.LOC_X)
dfn4_macro_newgeo_probempi['NPY'] = (dfn4_macro_newgeo_probempi.GLB_Y//dfn4_macro_newgeo_probempi.LOC_Y)
dfn4_macro_newgeo_probempi['NPZ'] = (dfn4_macro_newgeo_probempi.GLB_Z//dfn4_macro_newgeo_probempi.LOC_Z)
dfn4_macro_newgeo_probempi['NPT'] = (dfn4_macro_newgeo_probempi.GLB_T//dfn4_macro_newgeo_probempi.LOC_T)
dfn4_macro_newgeo_probempi["Total_proc"] = dfn4_macro_newgeo_probempi["omp_thrd"]* dfn4_macro_newgeo_probempi.MPI_size
# dfn4_macro_newgeo_probempi.columns

### Data Summary

In [20]:
dfn4_macro_newgeo_probempi_summary =dfn4_macro_newgeo_probempi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn4_macro_newgeo_probempi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,32,16,4,2,2,2,512,140
1,56,9,7,2,2,2,504,40
2,64,8,8,2,2,2,512,68
3,72,7,9,2,2,2,504,31
4,128,4,16,2,2,2,512,33
5,168,3,21,2,2,2,504,14
6,248,2,31,2,2,2,496,9
7,256,2,32,2,2,2,512,17
8,488,1,61,2,2,2,488,4
9,496,1,62,2,2,2,496,7


### Cases Setup

In [21]:
dfn4MacNewgeoProb_32mpi_16omp = dfn4_macro_newgeo_probempi[(dfn4_macro_newgeo_probempi.omp_thrd == 16) & (dfn4_macro_newgeo_probempi.GLB_T/dfn4_macro_newgeo_probempi.LOC_T == 4) & (dfn4_macro_newgeo_probempi.GLB_X/dfn4_macro_newgeo_probempi.LOC_X == 2) & (dfn4_macro_newgeo_probempi.GLB_Y/dfn4_macro_newgeo_probempi.LOC_Y == 2)& (dfn4_macro_newgeo_probempi.GLB_Z/dfn4_macro_newgeo_probempi.LOC_Z == 2)]
dfn4MacNewgeoProb_64mpi_8omp = dfn4_macro_newgeo_probempi[(dfn4_macro_newgeo_probempi.omp_thrd == 8) & (dfn4_macro_newgeo_probempi.GLB_T/dfn4_macro_newgeo_probempi.LOC_T == 8) & (dfn4_macro_newgeo_probempi.GLB_X/dfn4_macro_newgeo_probempi.LOC_X == 2) & (dfn4_macro_newgeo_probempi.GLB_Y/dfn4_macro_newgeo_probempi.LOC_Y == 2)& (dfn4_macro_newgeo_probempi.GLB_Z/dfn4_macro_newgeo_probempi.LOC_Z == 2)]
dfn4MacNewgeoProb_128mpi_4omp = dfn4_macro_newgeo_probempi[(dfn4_macro_newgeo_probempi.omp_thrd == 4) & (dfn4_macro_newgeo_probempi.GLB_T/dfn4_macro_newgeo_probempi.LOC_T == 16) & (dfn4_macro_newgeo_probempi.GLB_X/dfn4_macro_newgeo_probempi.LOC_X == 2) & (dfn4_macro_newgeo_probempi.GLB_Y/dfn4_macro_newgeo_probempi.LOC_Y == 2)& (dfn4_macro_newgeo_probempi.GLB_Z/dfn4_macro_newgeo_probempi.LOC_Z == 2)]
dfn4MacNewgeoProb_256mpi_2omp = dfn4_macro_newgeo_probempi[(dfn4_macro_newgeo_probempi.omp_thrd == 2) & (dfn4_macro_newgeo_probempi.GLB_T/dfn4_macro_newgeo_probempi.LOC_T == 32) & (dfn4_macro_newgeo_probempi.GLB_X/dfn4_macro_newgeo_probempi.LOC_X == 2) & (dfn4_macro_newgeo_probempi.GLB_Y/dfn4_macro_newgeo_probempi.LOC_Y == 2)& (dfn4_macro_newgeo_probempi.GLB_Z/dfn4_macro_newgeo_probempi.LOC_Z == 2)]
dfn4MacNewgeoProb_512mpi_1omp = dfn4_macro_newgeo_probempi[(dfn4_macro_newgeo_probempi.omp_thrd == 1) & (dfn4_macro_newgeo_probempi.GLB_T/dfn4_macro_newgeo_probempi.LOC_T == 64) & (dfn4_macro_newgeo_probempi.GLB_X/dfn4_macro_newgeo_probempi.LOC_X == 2) & (dfn4_macro_newgeo_probempi.GLB_Y/dfn4_macro_newgeo_probempi.LOC_Y == 2)& (dfn4_macro_newgeo_probempi.GLB_Z/dfn4_macro_newgeo_probempi.LOC_Z == 2)]
dfn4MacNewgeoProb_56mpi_9omp = dfn4_macro_newgeo_probempi[(dfn4_macro_newgeo_probempi.omp_thrd == 9) & (dfn4_macro_newgeo_probempi.GLB_T/dfn4_macro_newgeo_probempi.LOC_T == 7) & (dfn4_macro_newgeo_probempi.GLB_X/dfn4_macro_newgeo_probempi.LOC_X == 2) & (dfn4_macro_newgeo_probempi.GLB_Y/dfn4_macro_newgeo_probempi.LOC_Y == 2)& (dfn4_macro_newgeo_probempi.GLB_Z/dfn4_macro_newgeo_probempi.LOC_Z == 2)]
dfn4MacNewgeoProb_248mpi_2omp = dfn4_macro_newgeo_probempi[(dfn4_macro_newgeo_probempi.omp_thrd == 2) & (dfn4_macro_newgeo_probempi.GLB_T/dfn4_macro_newgeo_probempi.LOC_T == 31) & (dfn4_macro_newgeo_probempi.GLB_X/dfn4_macro_newgeo_probempi.LOC_X == 2) & (dfn4_macro_newgeo_probempi.GLB_Y/dfn4_macro_newgeo_probempi.LOC_Y == 2)& (dfn4_macro_newgeo_probempi.GLB_Z/dfn4_macro_newgeo_probempi.LOC_Z == 2)]


## 6) Newgeo_Macro_NonProbempi

### Adding Columns

In [22]:
dfn4_macro_newgeo_nonprobempi["N_Nodes"] = 4
dfn4_macro_newgeo_nonprobempi['NPX'] = (dfn4_macro_newgeo_nonprobempi.GLB_X//dfn4_macro_newgeo_nonprobempi.LOC_X)
dfn4_macro_newgeo_nonprobempi['NPY'] = (dfn4_macro_newgeo_nonprobempi.GLB_Y//dfn4_macro_newgeo_nonprobempi.LOC_Y)
dfn4_macro_newgeo_nonprobempi['NPZ'] = (dfn4_macro_newgeo_nonprobempi.GLB_Z//dfn4_macro_newgeo_nonprobempi.LOC_Z)
dfn4_macro_newgeo_nonprobempi['NPT'] = (dfn4_macro_newgeo_nonprobempi.GLB_T//dfn4_macro_newgeo_nonprobempi.LOC_T)
dfn4_macro_newgeo_nonprobempi["Total_proc"] = dfn4_macro_newgeo_nonprobempi["omp_thrd"]* dfn4_macro_newgeo_nonprobempi.MPI_size
# dfn4_macro_newgeo_nonprobempi.columns

### Data Summary

In [23]:
dfn4_macro_newgeo_nonprobempi_summary = dfn4_macro_newgeo_nonprobempi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn4_macro_newgeo_nonprobempi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,32,16,4,2,2,2,512,140
1,56,9,7,2,2,2,504,40
2,64,8,8,2,2,2,512,67
3,72,7,9,2,2,2,504,31
4,128,4,16,2,2,2,512,35
5,168,3,21,2,2,2,504,14
6,248,2,31,2,2,2,496,9
7,256,2,32,2,2,2,512,17
8,488,1,61,2,2,2,488,4
9,496,1,62,2,2,2,496,7


### Cases Setup

In [24]:
dfn4MacNewgeoNonProb_32mpi_16omp = dfn4_macro_newgeo_nonprobempi[(dfn4_macro_newgeo_nonprobempi.omp_thrd == 16) & (dfn4_macro_newgeo_nonprobempi.GLB_T/dfn4_macro_newgeo_nonprobempi.LOC_T == 4) & (dfn4_macro_newgeo_nonprobempi.GLB_X/dfn4_macro_newgeo_nonprobempi.LOC_X == 2) & (dfn4_macro_newgeo_nonprobempi.GLB_Y/dfn4_macro_newgeo_nonprobempi.LOC_Y == 2)& (dfn4_macro_newgeo_nonprobempi.GLB_Z/dfn4_macro_newgeo_nonprobempi.LOC_Z == 2)]
dfn4MacNewgeoNonProb_64mpi_8omp = dfn4_macro_newgeo_nonprobempi[(dfn4_macro_newgeo_nonprobempi.omp_thrd == 8) & (dfn4_macro_newgeo_nonprobempi.GLB_T/dfn4_macro_newgeo_nonprobempi.LOC_T == 8) & (dfn4_macro_newgeo_nonprobempi.GLB_X/dfn4_macro_newgeo_nonprobempi.LOC_X == 2) & (dfn4_macro_newgeo_nonprobempi.GLB_Y/dfn4_macro_newgeo_nonprobempi.LOC_Y == 2)& (dfn4_macro_newgeo_nonprobempi.GLB_Z/dfn4_macro_newgeo_nonprobempi.LOC_Z == 2)]
dfn4MacNewgeoNonProb_128mpi_4omp = dfn4_macro_newgeo_nonprobempi[(dfn4_macro_newgeo_nonprobempi.omp_thrd == 4) & (dfn4_macro_newgeo_nonprobempi.GLB_T/dfn4_macro_newgeo_nonprobempi.LOC_T == 16) & (dfn4_macro_newgeo_nonprobempi.GLB_X/dfn4_macro_newgeo_nonprobempi.LOC_X == 2) & (dfn4_macro_newgeo_nonprobempi.GLB_Y/dfn4_macro_newgeo_nonprobempi.LOC_Y == 2)& (dfn4_macro_newgeo_nonprobempi.GLB_Z/dfn4_macro_newgeo_nonprobempi.LOC_Z == 2)]
dfn4MacNewgeoNonProb_256mpi_2omp = dfn4_macro_newgeo_nonprobempi[(dfn4_macro_newgeo_nonprobempi.omp_thrd == 2) & (dfn4_macro_newgeo_nonprobempi.GLB_T/dfn4_macro_newgeo_nonprobempi.LOC_T == 32) & (dfn4_macro_newgeo_nonprobempi.GLB_X/dfn4_macro_newgeo_nonprobempi.LOC_X == 2) & (dfn4_macro_newgeo_nonprobempi.GLB_Y/dfn4_macro_newgeo_nonprobempi.LOC_Y == 2)& (dfn4_macro_newgeo_nonprobempi.GLB_Z/dfn4_macro_newgeo_nonprobempi.LOC_Z == 2)]
dfn4MacNewgeoNonProb_512mpi_1omp = dfn4_macro_newgeo_nonprobempi[(dfn4_macro_newgeo_nonprobempi.omp_thrd == 1) & (dfn4_macro_newgeo_nonprobempi.GLB_T/dfn4_macro_newgeo_nonprobempi.LOC_T == 64) & (dfn4_macro_newgeo_nonprobempi.GLB_X/dfn4_macro_newgeo_nonprobempi.LOC_X == 2) & (dfn4_macro_newgeo_nonprobempi.GLB_Y/dfn4_macro_newgeo_nonprobempi.LOC_Y == 2)& (dfn4_macro_newgeo_nonprobempi.GLB_Z/dfn4_macro_newgeo_nonprobempi.LOC_Z == 2)]
dfn4MacNewgeoNonProb_56mpi_9omp = dfn4_macro_newgeo_nonprobempi[(dfn4_macro_newgeo_nonprobempi.omp_thrd == 9) & (dfn4_macro_newgeo_nonprobempi.GLB_T/dfn4_macro_newgeo_nonprobempi.LOC_T == 7) & (dfn4_macro_newgeo_nonprobempi.GLB_X/dfn4_macro_newgeo_nonprobempi.LOC_X == 2) & (dfn4_macro_newgeo_nonprobempi.GLB_Y/dfn4_macro_newgeo_nonprobempi.LOC_Y == 2)& (dfn4_macro_newgeo_nonprobempi.GLB_Z/dfn4_macro_newgeo_nonprobempi.LOC_Z == 2)]
dfn4MacNewgeoNonProb_248mpi_2omp = dfn4_macro_newgeo_nonprobempi[(dfn4_macro_newgeo_nonprobempi.omp_thrd == 2) & (dfn4_macro_newgeo_nonprobempi.GLB_T/dfn4_macro_newgeo_nonprobempi.LOC_T == 31) & (dfn4_macro_newgeo_nonprobempi.GLB_X/dfn4_macro_newgeo_nonprobempi.LOC_X == 2) & (dfn4_macro_newgeo_nonprobempi.GLB_Y/dfn4_macro_newgeo_nonprobempi.LOC_Y == 2)& (dfn4_macro_newgeo_nonprobempi.GLB_Z/dfn4_macro_newgeo_nonprobempi.LOC_Z == 2)]


## GFLOPS - Newgeo Macro: Probempi vs Nonprobempi

In [25]:
fig = go.Figure() 

# Case 1
fig.add_trace(go.Scatter(x=(dfn4MacNewgeoProb_512mpi_1omp.Dirc_Data_Mov*(dfn4MacNewgeoProb_512mpi_1omp.MPI_size/4)), 
                         y=(dfn4MacNewgeoProb_512mpi_1omp.GFLOPS*(dfn4MacNewgeoProb_512mpi_1omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacNewgeoProb_512mpi_1omp',
                         line=dict(color='darkcyan',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))

fig.add_trace(go.Scatter(x=(dfn4MacNewgeoNonProb_512mpi_1omp.Dirc_Data_Mov*(dfn4MacNewgeoNonProb_512mpi_1omp.MPI_size/4)), 
                         y=(dfn4MacNewgeoNonProb_512mpi_1omp.GFLOPS*(dfn4MacNewgeoNonProb_512mpi_1omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacNewgeoNonProb_512mpi_1omp',
                         line=dict(color='magenta',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))


# Case 2
fig.add_trace(go.Scatter(x=(dfn4MacNewgeoProb_256mpi_2omp.Dirc_Data_Mov*(dfn4MacNewgeoProb_256mpi_2omp.MPI_size/4)), 
                         y=(dfn4MacNewgeoProb_256mpi_2omp.GFLOPS*(dfn4MacNewgeoProb_256mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacNewgeoProb_256mpi_2omp',
                         line=dict(color='royalblue',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))

fig.add_trace(go.Scatter(x=(dfn4MacNewgeoNonProb_256mpi_2omp.Dirc_Data_Mov*(dfn4MacNewgeoNonProb_256mpi_2omp.MPI_size/4)), 
                         y=(dfn4MacNewgeoNonProb_256mpi_2omp.GFLOPS*(dfn4MacNewgeoNonProb_256mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacNewgeoNonProb_256mpi_2omp',
                         line=dict(color='sandybrown',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))


# Case 3
fig.add_trace(go.Scatter(x=(dfn4MacNewgeoProb_128mpi_4omp.Dirc_Data_Mov*(dfn4MacNewgeoProb_128mpi_4omp.MPI_size/4)), 
                         y=(dfn4MacNewgeoProb_128mpi_4omp.GFLOPS*(dfn4MacNewgeoProb_128mpi_4omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacNewgeoProb_128mpi_4omp',
                         line=dict(color='darkblue',width=3),
                         marker=dict(symbol='circle',size=7, line=dict(color='tomato',width=2))))

fig.add_trace(go.Scatter(x=(dfn4MacNewgeoNonProb_128mpi_4omp.Dirc_Data_Mov*(dfn4MacNewgeoNonProb_128mpi_4omp.MPI_size/4)), 
                         y=(dfn4MacNewgeoNonProb_128mpi_4omp.GFLOPS*(dfn4MacNewgeoNonProb_128mpi_4omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacNewgeoNonProb_128mpi_4omp',
                         line=dict(color='cyan',width=3),
                         marker=dict(symbol='circle',size=7,line=dict(color='Purple',width=2))))



# Case 4
fig.add_trace(go.Scatter(x=(dfn4MacNewgeoProb_64mpi_8omp.Dirc_Data_Mov*(dfn4MacNewgeoProb_64mpi_8omp.MPI_size/4)), 
                         y=(dfn4MacNewgeoProb_64mpi_8omp.GFLOPS*(dfn4MacNewgeoProb_64mpi_8omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='4MacNewgeoProb_64mpi_8omp',
                         line=dict(color='blue',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))

fig.add_trace(go.Scatter(x=(dfn4MacNewgeoNonProb_64mpi_8omp.Dirc_Data_Mov*(dfn4MacNewgeoNonProb_64mpi_8omp.MPI_size/4)), 
                         y=(dfn4MacNewgeoNonProb_64mpi_8omp.GFLOPS*(dfn4MacNewgeoNonProb_64mpi_8omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacNewgeoNonProb_64mpi_8omp',
                         line=dict(color='red',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))


# Case 5
fig.add_trace(go.Scatter(x=(dfn4MacNewgeoProb_32mpi_16omp.Dirc_Data_Mov*(dfn4MacNewgeoProb_32mpi_16omp.MPI_size/4)), 
                         y=(dfn4MacNewgeoProb_32mpi_16omp.GFLOPS*(dfn4MacNewgeoProb_32mpi_16omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacNewgeoProb_32mpi_16omp',
                         line=dict(color='chartreuse',width=3),
                         marker=dict(symbol='square-dot',size=6, line=dict(color='mediumvioletred',width=1))))

fig.add_trace(go.Scatter(x=(dfn4MacNewgeoNonProb_32mpi_16omp.Dirc_Data_Mov*(dfn4MacNewgeoNonProb_32mpi_16omp.MPI_size/4)), 
                         y=(dfn4MacNewgeoNonProb_32mpi_16omp.GFLOPS*(dfn4MacNewgeoNonProb_32mpi_16omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacNewgeoNonProb_32mpi_16omp',
                         line=dict(color='black',width=3),
                         marker=dict(symbol='square-dot',size=6, line=dict(color='mediumvioletred',width=1))))


# Case 6
fig.add_trace(go.Scatter(x=(dfn4MacNewgeoProb_56mpi_9omp.Dirc_Data_Mov*(dfn4MacNewgeoProb_56mpi_9omp.MPI_size/4)), 
                         y=(dfn4MacNewgeoProb_56mpi_9omp.GFLOPS*(dfn4MacNewgeoProb_56mpi_9omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacNewgeoProb_56mpi_9omp',
                         line=dict(color='gold',width=3),
                         marker=dict(symbol='triangle-up',size=6, line=dict(color='black',width=1))))

fig.add_trace(go.Scatter(x=(dfn4MacNewgeoNonProb_56mpi_9omp.Dirc_Data_Mov*(dfn4MacNewgeoNonProb_56mpi_9omp.MPI_size/4)), 
                         y=(dfn4MacNewgeoNonProb_56mpi_9omp.GFLOPS*(dfn4MacNewgeoNonProb_56mpi_9omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacNewgeoNonProb_56mpi_9omp',
                         line=dict(color='dimgray',width=3),
                         marker=dict(symbol='triangle-up',size=6, line=dict(color='black',width=1))))



# Case 7
fig.add_trace(go.Scatter(x=(dfn4MacNewgeoProb_248mpi_2omp.Dirc_Data_Mov*(dfn4MacNewgeoProb_248mpi_2omp.MPI_size/4)), 
                         y=(dfn4MacNewgeoProb_248mpi_2omp.GFLOPS*(dfn4MacNewgeoProb_248mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacNewgeoProb_248mpi_2omp',
                         line=dict(color='olive',width=3),
                         marker=dict(symbol='circle',size=6, line=dict(color='olive',width=2))))

fig.add_trace(go.Scatter(x=(dfn4MacNewgeoNonProb_248mpi_2omp.Dirc_Data_Mov*(dfn4MacNewgeoNonProb_248mpi_2omp.MPI_size/4)), 
                         y=(dfn4MacNewgeoNonProb_248mpi_2omp.GFLOPS*(dfn4MacNewgeoNonProb_248mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacNewgeoNonProb_248mpi_2omp',
                         line=dict(color='Brown',width=3),
                         marker=dict(symbol='circle',size=6, line=dict(color='darkred',width=2))))




# Adding a vertical line
fig.add_vline(x=512e6, line_width=3, line_dash="dash", line_color="green")
# fig.add_hline(y=1024, line_width=3, line_dash="dash", line_color="red")

# Add annotation
fig.add_annotation(
    x='512e6'
    , y=130
    , text=f'LLC Size Per Node'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="purple", family="Sans Serif")
    , align="left"
    ,)

fig.update_layout(title='"Newgeo-Macro (4-Nodes): probempi vs nonprobempi" GFlops Measurement',
                  xaxis_title='Dirc_Data_Mov', yaxis_title='GFLOPs/Node', 
                  xaxis=dict(ticks="outside"),  
                  yaxis=dict(ticks="outside"))

## BANDWIDTH - Newgeo Macro: Probempi vs Nonprobempi

In [26]:
fig = go.Figure() 

# Case 1
fig.add_trace(go.Scatter(x=(dfn4MacNewgeoProb_512mpi_1omp.Dirc_Data_Mov*(dfn4MacNewgeoProb_512mpi_1omp.MPI_size/4)), 
                         y=(dfn4MacNewgeoProb_512mpi_1omp.BAND*(dfn4MacNewgeoProb_512mpi_1omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacNewgeoProb_512mpi_1omp',
                         line=dict(color='darkcyan',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))

fig.add_trace(go.Scatter(x=(dfn4MacNewgeoNonProb_512mpi_1omp.Dirc_Data_Mov*(dfn4MacNewgeoNonProb_512mpi_1omp.MPI_size/4)), 
                         y=(dfn4MacNewgeoNonProb_512mpi_1omp.BAND*(dfn4MacNewgeoNonProb_512mpi_1omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacNewgeoNonProb_512mpi_1omp',
                         line=dict(color='magenta',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))


# Case 2
fig.add_trace(go.Scatter(x=(dfn4MacNewgeoProb_256mpi_2omp.Dirc_Data_Mov*(dfn4MacNewgeoProb_256mpi_2omp.MPI_size/4)), 
                         y=(dfn4MacNewgeoProb_256mpi_2omp.BAND*(dfn4MacNewgeoProb_256mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacNewgeoProb_256mpi_2omp',
                         line=dict(color='royalblue',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))

fig.add_trace(go.Scatter(x=(dfn4MacNewgeoNonProb_256mpi_2omp.Dirc_Data_Mov*(dfn4MacNewgeoNonProb_256mpi_2omp.MPI_size/4)), 
                         y=(dfn4MacNewgeoNonProb_256mpi_2omp.BAND*(dfn4MacNewgeoNonProb_256mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacNewgeoNonProb_256mpi_2omp',
                         line=dict(color='sandybrown',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))


# Case 3
fig.add_trace(go.Scatter(x=(dfn4MacNewgeoProb_128mpi_4omp.Dirc_Data_Mov*(dfn4MacNewgeoProb_128mpi_4omp.MPI_size/4)), 
                         y=(dfn4MacNewgeoProb_128mpi_4omp.BAND*(dfn4MacNewgeoProb_128mpi_4omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacNewgeoProb_128mpi_4omp',
                         line=dict(color='darkblue',width=3),
                         marker=dict(symbol='circle',size=7, line=dict(color='tomato',width=2))))

fig.add_trace(go.Scatter(x=(dfn4MacNewgeoNonProb_128mpi_4omp.Dirc_Data_Mov*(dfn4MacNewgeoNonProb_128mpi_4omp.MPI_size/4)), 
                         y=(dfn4MacNewgeoNonProb_128mpi_4omp.BAND*(dfn4MacNewgeoNonProb_128mpi_4omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacNewgeoNonProb_128mpi_4omp',
                         line=dict(color='cyan',width=3),
                         marker=dict(symbol='circle',size=7,line=dict(color='Purple',width=2))))



# Case 4
fig.add_trace(go.Scatter(x=(dfn4MacNewgeoProb_64mpi_8omp.Dirc_Data_Mov*(dfn4MacNewgeoProb_64mpi_8omp.MPI_size/4)), 
                         y=(dfn4MacNewgeoProb_64mpi_8omp.BAND*(dfn4MacNewgeoProb_64mpi_8omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='4MacNewgeoProb_64mpi_8omp',
                         line=dict(color='blue',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))

fig.add_trace(go.Scatter(x=(dfn4MacNewgeoNonProb_64mpi_8omp.Dirc_Data_Mov*(dfn4MacNewgeoNonProb_64mpi_8omp.MPI_size/4)), 
                         y=(dfn4MacNewgeoNonProb_64mpi_8omp.BAND*(dfn4MacNewgeoNonProb_64mpi_8omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacNewgeoNonProb_64mpi_8omp',
                         line=dict(color='red',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))


# Case 5
fig.add_trace(go.Scatter(x=(dfn4MacNewgeoProb_32mpi_16omp.Dirc_Data_Mov*(dfn4MacNewgeoProb_32mpi_16omp.MPI_size/4)), 
                         y=(dfn4MacNewgeoProb_32mpi_16omp.BAND*(dfn4MacNewgeoProb_32mpi_16omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacNewgeoProb_32mpi_16omp',
                         line=dict(color='chartreuse',width=3),
                         marker=dict(symbol='square-dot',size=6, line=dict(color='mediumvioletred',width=1))))

fig.add_trace(go.Scatter(x=(dfn4MacNewgeoNonProb_32mpi_16omp.Dirc_Data_Mov*(dfn4MacNewgeoNonProb_32mpi_16omp.MPI_size/4)), 
                         y=(dfn4MacNewgeoNonProb_32mpi_16omp.BAND*(dfn4MacNewgeoNonProb_32mpi_16omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacNewgeoNonProb_32mpi_16omp',
                         line=dict(color='black',width=3),
                         marker=dict(symbol='square-dot',size=6, line=dict(color='mediumvioletred',width=1))))


# Case 6
fig.add_trace(go.Scatter(x=(dfn4MacNewgeoProb_56mpi_9omp.Dirc_Data_Mov*(dfn4MacNewgeoProb_56mpi_9omp.MPI_size/4)), 
                         y=(dfn4MacNewgeoProb_56mpi_9omp.BAND*(dfn4MacNewgeoProb_56mpi_9omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacNewgeoProb_56mpi_9omp',
                         line=dict(color='gold',width=3),
                         marker=dict(symbol='triangle-up',size=6, line=dict(color='black',width=1))))

fig.add_trace(go.Scatter(x=(dfn4MacNewgeoNonProb_56mpi_9omp.Dirc_Data_Mov*(dfn4MacNewgeoNonProb_56mpi_9omp.MPI_size/4)), 
                         y=(dfn4MacNewgeoNonProb_56mpi_9omp.BAND*(dfn4MacNewgeoNonProb_56mpi_9omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacNewgeoNonProb_56mpi_9omp',
                         line=dict(color='dimgray',width=3),
                         marker=dict(symbol='triangle-up',size=6, line=dict(color='black',width=1))))



# Case 7
fig.add_trace(go.Scatter(x=(dfn4MacNewgeoProb_248mpi_2omp.Dirc_Data_Mov*(dfn4MacNewgeoProb_248mpi_2omp.MPI_size/4)), 
                         y=(dfn4MacNewgeoProb_248mpi_2omp.BAND*(dfn4MacNewgeoProb_248mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacNewgeoProb_248mpi_2omp',
                         line=dict(color='olive',width=3),
                         marker=dict(symbol='circle',size=6, line=dict(color='olive',width=2))))

fig.add_trace(go.Scatter(x=(dfn4MacNewgeoNonProb_248mpi_2omp.Dirc_Data_Mov*(dfn4MacNewgeoNonProb_248mpi_2omp.MPI_size/4)), 
                         y=(dfn4MacNewgeoNonProb_248mpi_2omp.BAND*(dfn4MacNewgeoNonProb_248mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='MacNewgeoNonProb_248mpi_2omp',
                         line=dict(color='Brown',width=3),
                         marker=dict(symbol='circle',size=6, line=dict(color='darkred',width=2))))



# Adding vertical and horizontal lines 
fig.add_vline(x=512e6, line_width=3, line_dash="dot", line_color="green")
fig.add_hline(y=409, line_width=3, line_dash="dash", line_color="red")

# Add annotation with arrow
fig.add_annotation(
    x='512e6'
    , y=280
    , text=f'LLC Size Per Node: 512M'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="purple", family="Sans Serif")
    , align="left"
    ,)

fig.add_annotation(
    x='130e6'
    , y=370
    , text=f'Max Bandwidth Per Node'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="orange", family="Sans Serif")
    , align="left"
    ,)


fig.update_layout(title='"Newgeo-Macro: Probmpi vs Non-Probempi" Bandwidth Measurement',
                  xaxis_title='Dirc_Data_Mov', yaxis_title='Bandwidth/Node', 
                  xaxis=dict(ticks="outside"),  
                  yaxis=dict(ticks="outside"),
                  legend=go.layout.Legend(bordercolor="black",borderwidth=1)) 





# HIPPO AVX CASES

## 1) Fused_AVX_Probmpi

### Adding Columns

In [27]:
dfn4_avx_fused_probmpi["N_Nodes"] = 4
dfn4_avx_fused_probmpi['NPX'] = (dfn4_avx_fused_probmpi.GLB_X//dfn4_avx_fused_probmpi.LOC_X)
dfn4_avx_fused_probmpi['NPY'] = (dfn4_avx_fused_probmpi.GLB_Y//dfn4_avx_fused_probmpi.LOC_Y)
dfn4_avx_fused_probmpi['NPZ'] = (dfn4_avx_fused_probmpi.GLB_Z//dfn4_avx_fused_probmpi.LOC_Z)
dfn4_avx_fused_probmpi['NPT'] = (dfn4_avx_fused_probmpi.GLB_T//dfn4_avx_fused_probmpi.LOC_T)
dfn4_avx_fused_probmpi["Total_proc"] = dfn4_avx_fused_probmpi["omp_thrd"]* dfn4_avx_fused_probmpi.MPI_size
# dfn4_avx_fused_probmpi.columns

### Data Summary

In [28]:
dfn4_avx_fused_probmpi_summary = dfn4_avx_fused_probmpi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn4_avx_fused_probmpi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,32,16,4,2,2,2,512,138
1,56,9,7,2,2,2,504,40
2,64,8,8,2,2,2,512,68
3,72,7,9,2,2,2,504,30
4,128,4,16,2,2,2,512,35
5,168,3,21,2,2,2,504,14
6,248,2,31,2,2,2,496,8
7,256,2,32,2,2,2,512,17
8,488,1,61,2,2,2,488,4
9,496,1,62,2,2,2,496,7


### Cases Setup

In [29]:
dfn4AvxFusProb_8mpi_64omp = dfn4_avx_fused_probmpi[(dfn4_avx_fused_probmpi.omp_thrd == 64) & (dfn4_avx_fused_probmpi.GLB_T/dfn4_avx_fused_probmpi.LOC_T == 1) & (dfn4_avx_fused_probmpi.GLB_X/dfn4_avx_fused_probmpi.LOC_X == 2) & (dfn4_avx_fused_probmpi.GLB_Y/dfn4_avx_fused_probmpi.LOC_Y == 2)& (dfn4_avx_fused_probmpi.GLB_Z/dfn4_avx_fused_probmpi.LOC_Z == 2)]
dfn4AvxFusProb_16mpi_32omp = dfn4_avx_fused_probmpi[(dfn4_avx_fused_probmpi.omp_thrd == 32) & (dfn4_avx_fused_probmpi.GLB_T/dfn4_avx_fused_probmpi.LOC_T == 2) & (dfn4_avx_fused_probmpi.GLB_X/dfn4_avx_fused_probmpi.LOC_X == 2) & (dfn4_avx_fused_probmpi.GLB_Y/dfn4_avx_fused_probmpi.LOC_Y == 2)& (dfn4_avx_fused_probmpi.GLB_Z/dfn4_avx_fused_probmpi.LOC_Z == 2)]
dfn4AvxFusProb_32mpi_16omp = dfn4_avx_fused_probmpi[(dfn4_avx_fused_probmpi.omp_thrd == 16) & (dfn4_avx_fused_probmpi.GLB_T/dfn4_avx_fused_probmpi.LOC_T == 4) & (dfn4_avx_fused_probmpi.GLB_X/dfn4_avx_fused_probmpi.LOC_X == 2) & (dfn4_avx_fused_probmpi.GLB_Y/dfn4_avx_fused_probmpi.LOC_Y == 2)& (dfn4_avx_fused_probmpi.GLB_Z/dfn4_avx_fused_probmpi.LOC_Z == 2)]
dfn4AvxFusProb_64mpi_8omp = dfn4_avx_fused_probmpi[(dfn4_avx_fused_probmpi.omp_thrd == 8) & (dfn4_avx_fused_probmpi.GLB_T/dfn4_avx_fused_probmpi.LOC_T == 8) & (dfn4_avx_fused_probmpi.GLB_X/dfn4_avx_fused_probmpi.LOC_X == 2) & (dfn4_avx_fused_probmpi.GLB_Y/dfn4_avx_fused_probmpi.LOC_Y == 2)& (dfn4_avx_fused_probmpi.GLB_Z/dfn4_avx_fused_probmpi.LOC_Z == 2)]
dfn4AvxFusProb_128mpi_4omp = dfn4_avx_fused_probmpi[(dfn4_avx_fused_probmpi.omp_thrd == 4) & (dfn4_avx_fused_probmpi.GLB_T/dfn4_avx_fused_probmpi.LOC_T == 16) & (dfn4_avx_fused_probmpi.GLB_X/dfn4_avx_fused_probmpi.LOC_X == 2) & (dfn4_avx_fused_probmpi.GLB_Y/dfn4_avx_fused_probmpi.LOC_Y == 2)& (dfn4_avx_fused_probmpi.GLB_Z/dfn4_avx_fused_probmpi.LOC_Z == 2)]
dfn4AvxFusProb_256mpi_2omp = dfn4_avx_fused_probmpi[(dfn4_avx_fused_probmpi.omp_thrd == 2) & (dfn4_avx_fused_probmpi.GLB_T/dfn4_avx_fused_probmpi.LOC_T == 32) & (dfn4_avx_fused_probmpi.GLB_X/dfn4_avx_fused_probmpi.LOC_X == 2) & (dfn4_avx_fused_probmpi.GLB_Y/dfn4_avx_fused_probmpi.LOC_Y == 2)& (dfn4_avx_fused_probmpi.GLB_Z/dfn4_avx_fused_probmpi.LOC_Z == 2)]
dfn4AvxFusProb_512mpi_1omp = dfn4_avx_fused_probmpi[(dfn4_avx_fused_probmpi.omp_thrd == 1) & (dfn4_avx_fused_probmpi.GLB_T/dfn4_avx_fused_probmpi.LOC_T == 64) & (dfn4_avx_fused_probmpi.GLB_X/dfn4_avx_fused_probmpi.LOC_X == 2) & (dfn4_avx_fused_probmpi.GLB_Y/dfn4_avx_fused_probmpi.LOC_Y == 2)& (dfn4_avx_fused_probmpi.GLB_Z/dfn4_avx_fused_probmpi.LOC_Z == 2)]

dfn4AvxFusProb_248mpi_2omp = dfn4_avx_fused_probmpi[(dfn4_avx_fused_probmpi.omp_thrd == 2) & (dfn4_avx_fused_probmpi.GLB_T/dfn4_avx_fused_probmpi.LOC_T == 31) & (dfn4_avx_fused_probmpi.GLB_X/dfn4_avx_fused_probmpi.LOC_X == 2) & (dfn4_avx_fused_probmpi.GLB_Y/dfn4_avx_fused_probmpi.LOC_Y == 2)& (dfn4_avx_fused_probmpi.GLB_Z/dfn4_avx_fused_probmpi.LOC_Z == 2)]


## 2) Fused_AVX_NonProbmpi

### Adding Columns

In [30]:
dfn4_avx_fused_nonprobmpi["N_Nodes"] = 4
dfn4_avx_fused_nonprobmpi['NPX'] = (dfn4_avx_fused_nonprobmpi.GLB_X//dfn4_avx_fused_nonprobmpi.LOC_X)
dfn4_avx_fused_nonprobmpi['NPY'] = (dfn4_avx_fused_nonprobmpi.GLB_Y//dfn4_avx_fused_nonprobmpi.LOC_Y)
dfn4_avx_fused_nonprobmpi['NPZ'] = (dfn4_avx_fused_nonprobmpi.GLB_Z//dfn4_avx_fused_nonprobmpi.LOC_Z)
dfn4_avx_fused_nonprobmpi['NPT'] = (dfn4_avx_fused_nonprobmpi.GLB_T//dfn4_avx_fused_nonprobmpi.LOC_T)
dfn4_avx_fused_nonprobmpi["Total_proc"] = dfn4_avx_fused_nonprobmpi["omp_thrd"]* dfn4_avx_fused_nonprobmpi.MPI_size
# dfn4_avx_fused_nonprobmpi.columns

### Data Summary

In [31]:
dfn4_avx_fused_nonprobmpi_summary = dfn4_avx_fused_nonprobmpi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn4_avx_fused_nonprobmpi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,32,16,4,2,2,2,512,138
1,56,9,7,2,2,2,504,40
2,64,8,8,2,2,2,512,69
3,72,7,9,2,2,2,504,31
4,128,4,16,2,2,2,512,35
5,168,3,21,2,2,2,504,14
6,248,2,31,2,2,2,496,9
7,256,2,32,2,2,2,512,16
8,488,1,61,2,2,2,488,4
9,496,1,62,2,2,2,496,7


### Cases Setup

In [32]:
dfn4AvxFusNonProb_32mpi_16omp = dfn4_avx_fused_nonprobmpi[(dfn4_avx_fused_nonprobmpi.omp_thrd == 16) & (dfn4_avx_fused_nonprobmpi.GLB_T/dfn4_avx_fused_nonprobmpi.LOC_T == 4) & (dfn4_avx_fused_nonprobmpi.GLB_X/dfn4_avx_fused_nonprobmpi.LOC_X == 2) & (dfn4_avx_fused_nonprobmpi.GLB_Y/dfn4_avx_fused_nonprobmpi.LOC_Y == 2)& (dfn4_avx_fused_nonprobmpi.GLB_Z/dfn4_avx_fused_nonprobmpi.LOC_Z == 2)]
dfn4AvxFusNonProb_64mpi_8omp = dfn4_avx_fused_nonprobmpi[(dfn4_avx_fused_nonprobmpi.omp_thrd == 8) & (dfn4_avx_fused_nonprobmpi.GLB_T/dfn4_avx_fused_nonprobmpi.LOC_T == 8) & (dfn4_avx_fused_nonprobmpi.GLB_X/dfn4_avx_fused_nonprobmpi.LOC_X == 2) & (dfn4_avx_fused_nonprobmpi.GLB_Y/dfn4_avx_fused_nonprobmpi.LOC_Y == 2)& (dfn4_avx_fused_nonprobmpi.GLB_Z/dfn4_avx_fused_nonprobmpi.LOC_Z == 2)]
dfn4AvxFusNonProb_128mpi_4omp = dfn4_avx_fused_nonprobmpi[(dfn4_avx_fused_nonprobmpi.omp_thrd == 4) & (dfn4_avx_fused_nonprobmpi.GLB_T/dfn4_avx_fused_nonprobmpi.LOC_T == 16) & (dfn4_avx_fused_nonprobmpi.GLB_X/dfn4_avx_fused_nonprobmpi.LOC_X == 2) & (dfn4_avx_fused_nonprobmpi.GLB_Y/dfn4_avx_fused_nonprobmpi.LOC_Y == 2)& (dfn4_avx_fused_nonprobmpi.GLB_Z/dfn4_avx_fused_nonprobmpi.LOC_Z == 2)]
dfn4AvxFusNonProb_256mpi_2omp = dfn4_avx_fused_nonprobmpi[(dfn4_avx_fused_nonprobmpi.omp_thrd == 2) & (dfn4_avx_fused_nonprobmpi.GLB_T/dfn4_avx_fused_nonprobmpi.LOC_T == 32) & (dfn4_avx_fused_nonprobmpi.GLB_X/dfn4_avx_fused_nonprobmpi.LOC_X == 2) & (dfn4_avx_fused_nonprobmpi.GLB_Y/dfn4_avx_fused_nonprobmpi.LOC_Y == 2)& (dfn4_avx_fused_nonprobmpi.GLB_Z/dfn4_avx_fused_nonprobmpi.LOC_Z == 2)]
dfn4AvxFusNonProb_512mpi_1omp = dfn4_avx_fused_nonprobmpi[(dfn4_avx_fused_nonprobmpi.omp_thrd == 1) & (dfn4_avx_fused_nonprobmpi.GLB_T/dfn4_avx_fused_nonprobmpi.LOC_T == 64) & (dfn4_avx_fused_nonprobmpi.GLB_X/dfn4_avx_fused_nonprobmpi.LOC_X == 2) & (dfn4_avx_fused_nonprobmpi.GLB_Y/dfn4_avx_fused_nonprobmpi.LOC_Y == 2)& (dfn4_avx_fused_nonprobmpi.GLB_Z/dfn4_avx_fused_nonprobmpi.LOC_Z == 2)]
dfn4AvxFusNonProb_56mpi_9omp = dfn4_avx_fused_nonprobmpi[(dfn4_avx_fused_nonprobmpi.omp_thrd == 9) & (dfn4_avx_fused_nonprobmpi.GLB_T/dfn4_avx_fused_nonprobmpi.LOC_T == 7) & (dfn4_avx_fused_nonprobmpi.GLB_X/dfn4_avx_fused_nonprobmpi.LOC_X == 2) & (dfn4_avx_fused_nonprobmpi.GLB_Y/dfn4_avx_fused_nonprobmpi.LOC_Y == 2)& (dfn4_avx_fused_nonprobmpi.GLB_Z/dfn4_avx_fused_nonprobmpi.LOC_Z == 2)]
dfn4AvxFusNonProb_248mpi_2omp = dfn4_avx_fused_nonprobmpi[(dfn4_avx_fused_nonprobmpi.omp_thrd == 2) & (dfn4_avx_fused_nonprobmpi.GLB_T/dfn4_avx_fused_nonprobmpi.LOC_T == 31) & (dfn4_avx_fused_nonprobmpi.GLB_X/dfn4_avx_fused_nonprobmpi.LOC_X == 2) & (dfn4_avx_fused_nonprobmpi.GLB_Y/dfn4_avx_fused_nonprobmpi.LOC_Y == 2)& (dfn4_avx_fused_nonprobmpi.GLB_Z/dfn4_avx_fused_nonprobmpi.LOC_Z == 2)]


## GFLOPS - AVX-Fused: Probmpi vs Non-Probempim

In [33]:
fig = go.Figure() 

# Case 1
fig.add_trace(go.Scatter(x=(dfn4AvxFusProb_512mpi_1omp.Dirc_Data_Mov*(dfn4AvxFusProb_512mpi_1omp.MPI_size/4)), 
                         y=(dfn4AvxFusProb_512mpi_1omp.GFLOPS*(dfn4AvxFusProb_512mpi_1omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxFusProb_512mpi_1omp',
                         line=dict(color='darkcyan',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))

fig.add_trace(go.Scatter(x=(dfn4AvxFusNonProb_512mpi_1omp.Dirc_Data_Mov*(dfn4AvxFusNonProb_512mpi_1omp.MPI_size/4)), 
                         y=(dfn4AvxFusNonProb_512mpi_1omp.GFLOPS*(dfn4AvxFusNonProb_512mpi_1omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxFusNonProb_512mpi_1omp',
                         line=dict(color='magenta',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))


# Case 2
fig.add_trace(go.Scatter(x=(dfn4AvxFusProb_256mpi_2omp.Dirc_Data_Mov*(dfn4AvxFusProb_256mpi_2omp.MPI_size/4)), 
                         y=(dfn4AvxFusProb_256mpi_2omp.GFLOPS*(dfn4AvxFusProb_256mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxFusProb_256mpi_2omp',
                         line=dict(color='royalblue',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))

fig.add_trace(go.Scatter(x=(dfn4AvxFusNonProb_256mpi_2omp.Dirc_Data_Mov*(dfn4AvxFusNonProb_256mpi_2omp.MPI_size/4)), 
                         y=(dfn4AvxFusNonProb_256mpi_2omp.GFLOPS*(dfn4AvxFusNonProb_256mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxFusNonProb_256mpi_2omp',
                         line=dict(color='sandybrown',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))


# Case 3
fig.add_trace(go.Scatter(x=(dfn4AvxFusProb_128mpi_4omp.Dirc_Data_Mov*(dfn4AvxFusProb_128mpi_4omp.MPI_size/4)), 
                         y=(dfn4AvxFusProb_128mpi_4omp.GFLOPS*(dfn4AvxFusProb_128mpi_4omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxFusProb_128mpi_4omp',
                         line=dict(color='darkblue',width=3),
                         marker=dict(symbol='circle',size=7, line=dict(color='tomato',width=2))))

fig.add_trace(go.Scatter(x=(dfn4AvxFusNonProb_128mpi_4omp.Dirc_Data_Mov*(dfn4AvxFusNonProb_128mpi_4omp.MPI_size/4)), 
                         y=(dfn4AvxFusNonProb_128mpi_4omp.GFLOPS*(dfn4AvxFusNonProb_128mpi_4omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxFusNonProb_128mpi_4omp',
                         line=dict(color='cyan',width=3),
                         marker=dict(symbol='circle',size=7,line=dict(color='Purple',width=2))))



# Case 4
fig.add_trace(go.Scatter(x=(dfn4AvxFusProb_64mpi_8omp.Dirc_Data_Mov*(dfn4AvxFusProb_64mpi_8omp.MPI_size/4)), 
                         y=(dfn4AvxFusProb_64mpi_8omp.GFLOPS*(dfn4AvxFusProb_64mpi_8omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxFusProb_64mpi_8omp',
                         line=dict(color='blue',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))

fig.add_trace(go.Scatter(x=(dfn4AvxFusNonProb_64mpi_8omp.Dirc_Data_Mov*(dfn4AvxFusNonProb_64mpi_8omp.MPI_size/4)), 
                         y=(dfn4AvxFusNonProb_64mpi_8omp.GFLOPS*(dfn4AvxFusNonProb_64mpi_8omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxFusNonProb_640mpi_8omp',
                         line=dict(color='red',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))


# Case 5
fig.add_trace(go.Scatter(x=(dfn4AvxFusProb_32mpi_16omp.Dirc_Data_Mov*(dfn4AvxFusProb_32mpi_16omp.MPI_size/4)), 
                         y=(dfn4AvxFusProb_32mpi_16omp.GFLOPS*(dfn4AvxFusProb_32mpi_16omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxFusProb_32mpi_16omp',
                         line=dict(color='chartreuse',width=3),
                         marker=dict(symbol='square-dot',size=6, line=dict(color='mediumvioletred',width=1))))

fig.add_trace(go.Scatter(x=(dfn4AvxFusNonProb_32mpi_16omp.Dirc_Data_Mov*(dfn4AvxFusNonProb_32mpi_16omp.MPI_size/4)), 
                         y=(dfn4AvxFusNonProb_32mpi_16omp.GFLOPS*(dfn4AvxFusNonProb_32mpi_16omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxFusNonProb_32mpi_16omp',
                         line=dict(color='black',width=3),
                         marker=dict(symbol='square-dot',size=6, line=dict(color='mediumvioletred',width=1))))


# Case 6
fig.add_trace(go.Scatter(x=(dfn4AvxFusProb_16mpi_32omp.Dirc_Data_Mov*(dfn4AvxFusProb_16mpi_32omp.MPI_size/4)), 
                         y=(dfn4AvxFusProb_16mpi_32omp.GFLOPS*(dfn4AvxFusProb_16mpi_32omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxFusProb_16mpi_32omp',
                         line=dict(color='gold',width=3),
                         marker=dict(symbol='triangle-up',size=6, line=dict(color='black',width=1))))

fig.add_trace(go.Scatter(x=(dfn4AvxFusNonProb_56mpi_9omp.Dirc_Data_Mov*(dfn4AvxFusNonProb_56mpi_9omp.MPI_size/4)), 
                         y=(dfn4AvxFusNonProb_56mpi_9omp.GFLOPS*(dfn4AvxFusNonProb_56mpi_9omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxFusNonProb_56mpi_9omp',
                         line=dict(color='dimgray',width=3),
                         marker=dict(symbol='triangle-up',size=6, line=dict(color='black',width=1))))



# Case 7
# fig.add_trace(go.Scatter(x=(dfn4AvxFusProb_8mpi_64omp.Dirc_Data_Mov*(dfn4AvxFusProb_8mpi_64omp.MPI_size/4)), 
#                          y=(dfn4AvxFusProb_8mpi_64omp.GFLOPS*(dfn4AvxFusProb_8mpi_64omp.MPI_size/4)), 
#                          mode='lines+markers', 
#                          name='AvxFusProb_8mpi_64omp',
#                          line=dict(color='magenta',width=3),
#                          marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))

fig.add_trace(go.Scatter(x=(dfn4AvxFusNonProb_248mpi_2omp.Dirc_Data_Mov*(dfn4AvxFusNonProb_248mpi_2omp.MPI_size/4)), 
                         y=(dfn4AvxFusNonProb_248mpi_2omp.GFLOPS*(dfn4AvxFusNonProb_248mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxFusNonProb_248mpi_2omp',
                         line=dict(color='Brown',width=3),
                         marker=dict(symbol='circle',size=6, line=dict(color='darkred',width=2))))



# Case 8
fig.add_trace(go.Scatter(x=(dfn4AvxFusProb_248mpi_2omp.Dirc_Data_Mov*(dfn4AvxFusProb_248mpi_2omp.MPI_size/4)), 
                         y=(dfn4AvxFusProb_248mpi_2omp.GFLOPS*(dfn4AvxFusProb_248mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxFusProb_248mpi_2omp',
                        line=dict(color='Brown',width=3),
                        marker=dict(symbol='circle',size=6, line=dict(color='darkred',width=2))))



# Adding a vertical line
fig.add_vline(x=512e6, line_width=3, line_dash="dash", line_color="green")
# fig.add_hline(y=1024, line_width=3, line_dash="dash", line_color="red")

# Add annotation
fig.add_annotation(
    x='512e6'
    , y=130
    , text=f'LLC Size Per Node'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="purple", family="Sans Serif")
    , align="left"
    ,)

fig.update_layout(title='"AVX-Fused (4-Nodes): probempi vs nonprobempi" GFlops Measurement',
                  xaxis_title='Dirc_Data_Mov', yaxis_title='GFLOPs/Node', 
                  xaxis=dict(ticks="outside"),  
                  yaxis=dict(ticks="outside"))

## BANDWIDTHM - AVX-Fused: Probmpi vs Non-Probempim

In [34]:
fig = go.Figure() 

# Case 1
fig.add_trace(go.Scatter(x=(dfn4AvxFusProb_512mpi_1omp.Dirc_Data_Mov*(dfn4AvxFusProb_512mpi_1omp.MPI_size/4)), 
                         y=(dfn4AvxFusProb_512mpi_1omp.BAND*(dfn4AvxFusProb_512mpi_1omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxFusProb_512mpi_1omp',
                         line=dict(color='darkcyan',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))

fig.add_trace(go.Scatter(x=(dfn4AvxFusNonProb_512mpi_1omp.Dirc_Data_Mov*(dfn4AvxFusNonProb_512mpi_1omp.MPI_size/4)), 
                         y=(dfn4AvxFusNonProb_512mpi_1omp.BAND*(dfn4AvxFusNonProb_512mpi_1omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxFusNonProb_512mpi_1omp',
                         line=dict(color='magenta',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))


# Case 2
fig.add_trace(go.Scatter(x=(dfn4AvxFusProb_256mpi_2omp.Dirc_Data_Mov*(dfn4AvxFusProb_256mpi_2omp.MPI_size/4)), 
                         y=(dfn4AvxFusProb_256mpi_2omp.BAND*(dfn4AvxFusProb_256mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxFusProb_256mpi_2omp',
                         line=dict(color='royalblue',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))

fig.add_trace(go.Scatter(x=(dfn4AvxFusNonProb_256mpi_2omp.Dirc_Data_Mov*(dfn4AvxFusNonProb_256mpi_2omp.MPI_size/4)), 
                         y=(dfn4AvxFusNonProb_256mpi_2omp.BAND*(dfn4AvxFusNonProb_256mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxFusNonProb_256mpi_2omp',
                         line=dict(color='sandybrown',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))


# Case 3
fig.add_trace(go.Scatter(x=(dfn4AvxFusProb_128mpi_4omp.Dirc_Data_Mov*(dfn4AvxFusProb_128mpi_4omp.MPI_size/4)), 
                         y=(dfn4AvxFusProb_128mpi_4omp.BAND*(dfn4AvxFusProb_128mpi_4omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxFusProb_128mpi_4omp',
                         line=dict(color='darkblue',width=3),
                         marker=dict(symbol='circle',size=7, line=dict(color='tomato',width=2))))

fig.add_trace(go.Scatter(x=(dfn4AvxFusNonProb_128mpi_4omp.Dirc_Data_Mov*(dfn4AvxFusNonProb_128mpi_4omp.MPI_size/4)), 
                         y=(dfn4AvxFusNonProb_128mpi_4omp.BAND*(dfn4AvxFusNonProb_128mpi_4omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxFusNonProb_128mpi_4omp',
                         line=dict(color='cyan',width=3),
                         marker=dict(symbol='circle',size=7,line=dict(color='Purple',width=2))))



# Case 4
fig.add_trace(go.Scatter(x=(dfn4AvxFusProb_64mpi_8omp.Dirc_Data_Mov*(dfn4AvxFusProb_64mpi_8omp.MPI_size/4)), 
                         y=(dfn4AvxFusProb_64mpi_8omp.BAND*(dfn4AvxFusProb_64mpi_8omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxFusProb_64mpi_8omp',
                         line=dict(color='blue',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))

fig.add_trace(go.Scatter(x=(dfn4AvxFusNonProb_64mpi_8omp.Dirc_Data_Mov*(dfn4AvxFusNonProb_64mpi_8omp.MPI_size/4)), 
                         y=(dfn4AvxFusNonProb_64mpi_8omp.BAND*(dfn4AvxFusNonProb_64mpi_8omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxFusNonProb_640mpi_8omp',
                         line=dict(color='red',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))


# Case 5
fig.add_trace(go.Scatter(x=(dfn4AvxFusProb_32mpi_16omp.Dirc_Data_Mov*(dfn4AvxFusProb_32mpi_16omp.MPI_size/4)), 
                         y=(dfn4AvxFusProb_32mpi_16omp.BAND*(dfn4AvxFusProb_32mpi_16omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxFusProb_32mpi_16omp',
                         line=dict(color='chartreuse',width=3),
                         marker=dict(symbol='square-dot',size=6, line=dict(color='mediumvioletred',width=1))))

fig.add_trace(go.Scatter(x=(dfn4AvxFusNonProb_32mpi_16omp.Dirc_Data_Mov*(dfn4AvxFusNonProb_32mpi_16omp.MPI_size/4)), 
                         y=(dfn4AvxFusNonProb_32mpi_16omp.BAND*(dfn4AvxFusNonProb_32mpi_16omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxFusNonProb_32mpi_16omp',
                         line=dict(color='black',width=3),
                         marker=dict(symbol='square-dot',size=6, line=dict(color='mediumvioletred',width=1))))


# Case 6
# fig.add_trace(go.Scatter(x=(dfn4AvxFusProb_56mpi_9omp.Dirc_Data_Mov*(dfn4AvxFusProb_56mpi_9omp.MPI_size/4)), 
#                          y=(dfn4AvxFusProb_56mpi_9omp.BAND*(dfn4AvxFusProb_56mpi_9omp.MPI_size/4)), 
#                          mode='lines+markers', 
#                          name='AvxFusProb_56mpi_9omp',
#                          line=dict(color='gold',width=3),
#                          marker=dict(symbol='triangle-up',size=6, line=dict(color='black',width=1))))

fig.add_trace(go.Scatter(x=(dfn4AvxFusNonProb_56mpi_9omp.Dirc_Data_Mov*(dfn4AvxFusNonProb_56mpi_9omp.MPI_size/4)), 
                         y=(dfn4AvxFusNonProb_56mpi_9omp.BAND*(dfn4AvxFusNonProb_56mpi_9omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxFusNonProb_56mpi_9omp',
                         line=dict(color='dimgray',width=3),
                         marker=dict(symbol='triangle-up',size=6, line=dict(color='black',width=1))))



# Case 7
fig.add_trace(go.Scatter(x=(dfn4AvxFusProb_248mpi_2omp.Dirc_Data_Mov*(dfn4AvxFusProb_248mpi_2omp.MPI_size/4)), 
                         y=(dfn4AvxFusProb_248mpi_2omp.BAND*(dfn4AvxFusProb_248mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxFusProb_248mpi_2omp',
                         line=dict(color='olive',width=3),
                         marker=dict(symbol='circle',size=6, line=dict(color='olive',width=2))))

fig.add_trace(go.Scatter(x=(dfn4AvxFusNonProb_248mpi_2omp.Dirc_Data_Mov*(dfn4AvxFusNonProb_248mpi_2omp.MPI_size/4)), 
                         y=(dfn4AvxFusNonProb_248mpi_2omp.BAND*(dfn4AvxFusNonProb_248mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxFusNonProb_248mpi_2omp',
                         line=dict(color='Brown',width=3),
                         marker=dict(symbol='circle',size=6, line=dict(color='darkred',width=2))))




# Adding vertical and horizontal lines 
fig.add_vline(x=512e6, line_width=3, line_dash="dot", line_color="green")
fig.add_hline(y=409, line_width=3, line_dash="dash", line_color="red")

# Add annotation with arrow
fig.add_annotation(
    x='512e6'
    , y=280
    , text=f'LLC Size Per Node: 512M'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="purple", family="Sans Serif")
    , align="left"
    ,)

fig.add_annotation(
    x='130e6'
    , y=370
    , text=f'Max Bandwidth Per Node'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="orange", family="Sans Serif")
    , align="left"
    ,)


fig.update_layout(title='"AVX-Fused: Probmpi vs Non-Probempi" Bandwidth Measurement',
                  xaxis_title='Dirc_Data_Mov', yaxis_title='Bandwidth/Node', 
                  xaxis=dict(ticks="outside"),  
                  yaxis=dict(ticks="outside"),
                  legend=go.layout.Legend(bordercolor="black",borderwidth=1)) 




## 3) NonFused_AVX_Probempi

### Adding Columns

In [35]:
dfn4_avx_nonfused_probempi["N_Nodes"] = 4
dfn4_avx_nonfused_probempi['NPX'] = (dfn4_avx_nonfused_probempi.GLB_X//dfn4_avx_nonfused_probempi.LOC_X)
dfn4_avx_nonfused_probempi['NPY'] = (dfn4_avx_nonfused_probempi.GLB_Y//dfn4_avx_nonfused_probempi.LOC_Y)
dfn4_avx_nonfused_probempi['NPZ'] = (dfn4_avx_nonfused_probempi.GLB_Z//dfn4_avx_nonfused_probempi.LOC_Z)
dfn4_avx_nonfused_probempi['NPT'] = (dfn4_avx_nonfused_probempi.GLB_T//dfn4_avx_nonfused_probempi.LOC_T)
dfn4_avx_nonfused_probempi["Total_proc"] = dfn4_avx_nonfused_probempi["omp_thrd"]* dfn4_avx_nonfused_probempi.MPI_size
# dfn4_avx_nonfused_probempi.columns

### Data Summary

In [36]:
dfn4_avx_nonfused_probempi_summary = dfn4_avx_nonfused_probempi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn4_avx_nonfused_probempi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,32,16,4,2,2,2,512,119
1,56,9,7,2,2,2,504,39
2,64,8,8,2,2,2,512,68
3,72,7,9,2,2,2,504,31
4,128,4,16,2,2,2,512,35
5,168,3,21,2,2,2,504,14
6,248,2,31,2,2,2,496,9
7,256,2,32,2,2,2,512,17
8,488,1,61,2,2,2,488,3
9,496,1,62,2,2,2,496,7


### Cases Setup

In [37]:
dfn4AvxNonFusProb_32mpi_16omp = dfn4_avx_nonfused_probempi[(dfn4_avx_nonfused_probempi.omp_thrd == 16) & (dfn4_avx_nonfused_probempi.GLB_T/dfn4_avx_nonfused_probempi.LOC_T == 4) & (dfn4_avx_nonfused_probempi.GLB_X/dfn4_avx_nonfused_probempi.LOC_X == 2) & (dfn4_avx_nonfused_probempi.GLB_Y/dfn4_avx_nonfused_probempi.LOC_Y == 2)& (dfn4_avx_nonfused_probempi.GLB_Z/dfn4_avx_nonfused_probempi.LOC_Z == 2)]
dfn4AvxNonFusProb_64mpi_8omp = dfn4_avx_nonfused_probempi[(dfn4_avx_nonfused_probempi.omp_thrd == 8) & (dfn4_avx_nonfused_probempi.GLB_T/dfn4_avx_nonfused_probempi.LOC_T == 8) & (dfn4_avx_nonfused_probempi.GLB_X/dfn4_avx_nonfused_probempi.LOC_X == 2) & (dfn4_avx_nonfused_probempi.GLB_Y/dfn4_avx_nonfused_probempi.LOC_Y == 2)& (dfn4_avx_nonfused_probempi.GLB_Z/dfn4_avx_nonfused_probempi.LOC_Z == 2)]
dfn4AvxNonFusProb_128mpi_4omp = dfn4_avx_nonfused_probempi[(dfn4_avx_nonfused_probempi.omp_thrd == 4) & (dfn4_avx_nonfused_probempi.GLB_T/dfn4_avx_nonfused_probempi.LOC_T == 16) & (dfn4_avx_nonfused_probempi.GLB_X/dfn4_avx_nonfused_probempi.LOC_X == 2) & (dfn4_avx_nonfused_probempi.GLB_Y/dfn4_avx_nonfused_probempi.LOC_Y == 2)& (dfn4_avx_nonfused_probempi.GLB_Z/dfn4_avx_nonfused_probempi.LOC_Z == 2)]
dfn4AvxNonFusProb_256mpi_2omp = dfn4_avx_nonfused_probempi[(dfn4_avx_nonfused_probempi.omp_thrd == 2) & (dfn4_avx_nonfused_probempi.GLB_T/dfn4_avx_nonfused_probempi.LOC_T == 32) & (dfn4_avx_nonfused_probempi.GLB_X/dfn4_avx_nonfused_probempi.LOC_X == 2) & (dfn4_avx_nonfused_probempi.GLB_Y/dfn4_avx_nonfused_probempi.LOC_Y == 2)& (dfn4_avx_nonfused_probempi.GLB_Z/dfn4_avx_nonfused_probempi.LOC_Z == 2)]
dfn4AvxNonFusProb_512mpi_1omp = dfn4_avx_nonfused_probempi[(dfn4_avx_nonfused_probempi.omp_thrd == 1) & (dfn4_avx_nonfused_probempi.GLB_T/dfn4_avx_nonfused_probempi.LOC_T == 64) & (dfn4_avx_nonfused_probempi.GLB_X/dfn4_avx_nonfused_probempi.LOC_X == 2) & (dfn4_avx_nonfused_probempi.GLB_Y/dfn4_avx_nonfused_probempi.LOC_Y == 2)& (dfn4_avx_nonfused_probempi.GLB_Z/dfn4_avx_nonfused_probempi.LOC_Z == 2)]

dfn4AvxNonFusProb_56mpi_9omp = dfn4_avx_nonfused_probempi[(dfn4_avx_nonfused_probempi.omp_thrd == 9) & (dfn4_avx_nonfused_probempi.GLB_T/dfn4_avx_nonfused_probempi.LOC_T == 7) & (dfn4_avx_nonfused_probempi.GLB_X/dfn4_avx_nonfused_probempi.LOC_X == 2) & (dfn4_avx_nonfused_probempi.GLB_Y/dfn4_avx_nonfused_probempi.LOC_Y == 2)& (dfn4_avx_nonfused_probempi.GLB_Z/dfn4_avx_nonfused_probempi.LOC_Z == 2)]
dfn4AvxNonFusProb_248mpi_2omp = dfn4_avx_nonfused_probempi[(dfn4_avx_nonfused_probempi.omp_thrd == 2) & (dfn4_avx_nonfused_probempi.GLB_T/dfn4_avx_nonfused_probempi.LOC_T == 31) & (dfn4_avx_nonfused_probempi.GLB_X/dfn4_avx_nonfused_probempi.LOC_X == 2) & (dfn4_avx_nonfused_probempi.GLB_Y/dfn4_avx_nonfused_probempi.LOC_Y == 2)& (dfn4_avx_nonfused_probempi.GLB_Z/dfn4_avx_nonfused_probempi.LOC_Z == 2)]


## 4) NonFused_AVX_NonProbempi

### Adding Columns

In [38]:
dfn4_avx_nonfused_nonprobempi["N_Nodes"] = 4
dfn4_avx_nonfused_nonprobempi['NPX'] = (dfn4_avx_nonfused_nonprobempi.GLB_X//dfn4_avx_nonfused_nonprobempi.LOC_X)
dfn4_avx_nonfused_nonprobempi['NPY'] = (dfn4_avx_nonfused_nonprobempi.GLB_Y//dfn4_avx_nonfused_nonprobempi.LOC_Y)
dfn4_avx_nonfused_nonprobempi['NPZ'] = (dfn4_avx_nonfused_nonprobempi.GLB_Z//dfn4_avx_nonfused_nonprobempi.LOC_Z)
dfn4_avx_nonfused_nonprobempi['NPT'] = (dfn4_avx_nonfused_nonprobempi.GLB_T//dfn4_avx_nonfused_nonprobempi.LOC_T)
dfn4_avx_nonfused_nonprobempi["Total_proc"] = dfn4_avx_nonfused_nonprobempi["omp_thrd"]* dfn4_avx_nonfused_nonprobempi.MPI_size
# dfn4_avx_nonfused_nonprobempi.columns

### Data Summary

In [39]:
dfn4_avx_nonfused_nonprobempi_summary = dfn4_avx_nonfused_nonprobempi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn4_avx_nonfused_nonprobempi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,32,16,4,2,2,2,512,111
1,56,9,7,2,2,2,504,40
2,64,8,8,2,2,2,512,68
3,72,7,9,2,2,2,504,30
4,128,4,16,2,2,2,512,35
5,168,3,21,2,2,2,504,14
6,248,2,31,2,2,2,496,9
7,256,2,32,2,2,2,512,17
8,488,1,61,2,2,2,488,3
9,496,1,62,2,2,2,496,7


### Cases Setup

In [40]:
dfn4AvxNonFusNonProb_32mpi_16omp = dfn4_avx_nonfused_nonprobempi[(dfn4_avx_nonfused_nonprobempi.omp_thrd == 16) & (dfn4_avx_nonfused_nonprobempi.GLB_T/dfn4_avx_nonfused_nonprobempi.LOC_T == 4) & (dfn4_avx_nonfused_nonprobempi.GLB_X/dfn4_avx_nonfused_nonprobempi.LOC_X == 2) & (dfn4_avx_nonfused_nonprobempi.GLB_Y/dfn4_avx_nonfused_nonprobempi.LOC_Y == 2)& (dfn4_avx_nonfused_nonprobempi.GLB_Z/dfn4_avx_nonfused_nonprobempi.LOC_Z == 2)]
dfn4AvxNonFusNonProb_64mpi_8omp = dfn4_avx_nonfused_nonprobempi[(dfn4_avx_nonfused_nonprobempi.omp_thrd == 8) & (dfn4_avx_nonfused_nonprobempi.GLB_T/dfn4_avx_nonfused_nonprobempi.LOC_T == 8) & (dfn4_avx_nonfused_nonprobempi.GLB_X/dfn4_avx_nonfused_nonprobempi.LOC_X == 2) & (dfn4_avx_nonfused_nonprobempi.GLB_Y/dfn4_avx_nonfused_nonprobempi.LOC_Y == 2)& (dfn4_avx_nonfused_nonprobempi.GLB_Z/dfn4_avx_nonfused_nonprobempi.LOC_Z == 2)]
dfn4AvxNonFusNonProb_128mpi_4omp = dfn4_avx_nonfused_nonprobempi[(dfn4_avx_nonfused_nonprobempi.omp_thrd == 4) & (dfn4_avx_nonfused_nonprobempi.GLB_T/dfn4_avx_nonfused_nonprobempi.LOC_T == 16) & (dfn4_avx_nonfused_nonprobempi.GLB_X/dfn4_avx_nonfused_nonprobempi.LOC_X == 2) & (dfn4_avx_nonfused_nonprobempi.GLB_Y/dfn4_avx_nonfused_nonprobempi.LOC_Y == 2)& (dfn4_avx_nonfused_nonprobempi.GLB_Z/dfn4_avx_nonfused_nonprobempi.LOC_Z == 2)]
dfn4AvxNonFusNonProb_256mpi_2omp = dfn4_avx_nonfused_nonprobempi[(dfn4_avx_nonfused_nonprobempi.omp_thrd == 2) & (dfn4_avx_nonfused_nonprobempi.GLB_T/dfn4_avx_nonfused_nonprobempi.LOC_T == 32) & (dfn4_avx_nonfused_nonprobempi.GLB_X/dfn4_avx_nonfused_nonprobempi.LOC_X == 2) & (dfn4_avx_nonfused_nonprobempi.GLB_Y/dfn4_avx_nonfused_nonprobempi.LOC_Y == 2)& (dfn4_avx_nonfused_nonprobempi.GLB_Z/dfn4_avx_nonfused_nonprobempi.LOC_Z == 2)]
dfn4AvxNonFusNonProb_512mpi_1omp = dfn4_avx_nonfused_nonprobempi[(dfn4_avx_nonfused_nonprobempi.omp_thrd == 1) & (dfn4_avx_nonfused_nonprobempi.GLB_T/dfn4_avx_nonfused_nonprobempi.LOC_T == 64) & (dfn4_avx_nonfused_nonprobempi.GLB_X/dfn4_avx_nonfused_nonprobempi.LOC_X == 2) & (dfn4_avx_nonfused_nonprobempi.GLB_Y/dfn4_avx_nonfused_nonprobempi.LOC_Y == 2)& (dfn4_avx_nonfused_nonprobempi.GLB_Z/dfn4_avx_nonfused_nonprobempi.LOC_Z == 2)]
dfn4AvxNonFusNonProb_56mpi_9omp = dfn4_avx_nonfused_nonprobempi[(dfn4_avx_nonfused_nonprobempi.omp_thrd == 9) & (dfn4_avx_nonfused_nonprobempi.GLB_T/dfn4_avx_nonfused_nonprobempi.LOC_T == 7) & (dfn4_avx_nonfused_nonprobempi.GLB_X/dfn4_avx_nonfused_nonprobempi.LOC_X == 2) & (dfn4_avx_nonfused_nonprobempi.GLB_Y/dfn4_avx_nonfused_nonprobempi.LOC_Y == 2)& (dfn4_avx_nonfused_nonprobempi.GLB_Z/dfn4_avx_nonfused_nonprobempi.LOC_Z == 2)]
dfn4AvxNonFusNonProb_248mpi_2omp = dfn4_avx_nonfused_nonprobempi[(dfn4_avx_nonfused_nonprobempi.omp_thrd == 2) & (dfn4_avx_nonfused_nonprobempi.GLB_T/dfn4_avx_nonfused_nonprobempi.LOC_T == 31) & (dfn4_avx_nonfused_nonprobempi.GLB_X/dfn4_avx_nonfused_nonprobempi.LOC_X == 2) & (dfn4_avx_nonfused_nonprobempi.GLB_Y/dfn4_avx_nonfused_nonprobempi.LOC_Y == 2)& (dfn4_avx_nonfused_nonprobempi.GLB_Z/dfn4_avx_nonfused_nonprobempi.LOC_Z == 2)]


## GFLOPS - NonFused-AVX: Probmpi vs Non-Probempi

In [41]:
fig = go.Figure() 

# Case 1
fig.add_trace(go.Scatter(x=(dfn4AvxNonFusProb_512mpi_1omp.Dirc_Data_Mov*(dfn4AvxNonFusProb_512mpi_1omp.MPI_size/4)), 
                         y=(dfn4AvxNonFusProb_512mpi_1omp.GFLOPS*(dfn4AvxNonFusProb_512mpi_1omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxNonFusProb_512mpi_1omp',
                         line=dict(color='darkcyan',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))

fig.add_trace(go.Scatter(x=(dfn4AvxNonFusNonProb_512mpi_1omp.Dirc_Data_Mov*(dfn4AvxNonFusNonProb_512mpi_1omp.MPI_size/4)), 
                         y=(dfn4AvxNonFusNonProb_512mpi_1omp.GFLOPS*(dfn4AvxNonFusNonProb_512mpi_1omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxNonFusNonProb_512mpi_1omp',
                         line=dict(color='magenta',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))


# Case 2
fig.add_trace(go.Scatter(x=(dfn4AvxNonFusProb_256mpi_2omp.Dirc_Data_Mov*(dfn4AvxNonFusProb_256mpi_2omp.MPI_size/4)), 
                         y=(dfn4AvxNonFusProb_256mpi_2omp.GFLOPS*(dfn4AvxNonFusProb_256mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxNonFusProb_256mpi_2omp',
                         line=dict(color='royalblue',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))

fig.add_trace(go.Scatter(x=(dfn4AvxNonFusNonProb_256mpi_2omp.Dirc_Data_Mov*(dfn4AvxNonFusNonProb_256mpi_2omp.MPI_size/4)), 
                         y=(dfn4AvxNonFusNonProb_256mpi_2omp.GFLOPS*(dfn4AvxNonFusNonProb_256mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxNonFusNonProb_256mpi_2omp',
                         line=dict(color='sandybrown',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))


# Case 3
fig.add_trace(go.Scatter(x=(dfn4AvxNonFusProb_128mpi_4omp.Dirc_Data_Mov*(dfn4AvxNonFusProb_128mpi_4omp.MPI_size/4)), 
                         y=(dfn4AvxNonFusProb_128mpi_4omp.GFLOPS*(dfn4AvxNonFusProb_128mpi_4omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxNonFusProb_128mpi_4omp',
                         line=dict(color='darkblue',width=3),
                         marker=dict(symbol='circle',size=7, line=dict(color='tomato',width=2))))

fig.add_trace(go.Scatter(x=(dfn4AvxNonFusNonProb_128mpi_4omp.Dirc_Data_Mov*(dfn4AvxNonFusNonProb_128mpi_4omp.MPI_size/4)), 
                         y=(dfn4AvxNonFusNonProb_128mpi_4omp.GFLOPS*(dfn4AvxNonFusNonProb_128mpi_4omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxNonFusNonProb_128mpi_4omp',
                         line=dict(color='cyan',width=3),
                         marker=dict(symbol='circle',size=7,line=dict(color='Purple',width=2))))



# Case 4
fig.add_trace(go.Scatter(x=(dfn4AvxNonFusProb_64mpi_8omp.Dirc_Data_Mov*(dfn4AvxNonFusProb_64mpi_8omp.MPI_size/4)), 
                         y=(dfn4AvxNonFusProb_64mpi_8omp.GFLOPS*(dfn4AvxNonFusProb_64mpi_8omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxNonFusProb_64mpi_8omp',
                         line=dict(color='blue',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))

fig.add_trace(go.Scatter(x=(dfn4AvxNonFusNonProb_64mpi_8omp.Dirc_Data_Mov*(dfn4AvxNonFusNonProb_64mpi_8omp.MPI_size/4)), 
                         y=(dfn4AvxNonFusNonProb_64mpi_8omp.GFLOPS*(dfn4AvxNonFusNonProb_64mpi_8omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxNonFusNonProb_64mpi_8omp',
                         line=dict(color='red',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))


# Case 5
fig.add_trace(go.Scatter(x=(dfn4AvxNonFusProb_32mpi_16omp.Dirc_Data_Mov*(dfn4AvxNonFusProb_32mpi_16omp.MPI_size/4)), 
                         y=(dfn4AvxNonFusProb_32mpi_16omp.GFLOPS*(dfn4AvxNonFusProb_32mpi_16omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxNonFusProb_32mpi_16omp',
                         line=dict(color='chartreuse',width=3),
                         marker=dict(symbol='square-dot',size=6, line=dict(color='mediumvioletred',width=1))))

fig.add_trace(go.Scatter(x=(dfn4AvxNonFusNonProb_32mpi_16omp.Dirc_Data_Mov*(dfn4AvxNonFusNonProb_32mpi_16omp.MPI_size/4)), 
                         y=(dfn4AvxNonFusNonProb_32mpi_16omp.GFLOPS*(dfn4AvxNonFusNonProb_32mpi_16omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxNonFusNonProb_32mpi_16omp',
                         line=dict(color='black',width=3),
                         marker=dict(symbol='square-dot',size=6, line=dict(color='mediumvioletred',width=1))))


# Case 6
fig.add_trace(go.Scatter(x=(dfn4AvxNonFusProb_56mpi_9omp.Dirc_Data_Mov*(dfn4AvxNonFusProb_56mpi_9omp.MPI_size/4)), 
                         y=(dfn4AvxNonFusProb_56mpi_9omp.GFLOPS*(dfn4AvxNonFusProb_56mpi_9omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxNonFusProb_56mpi_9omp',
                         line=dict(color='gold',width=3),
                         marker=dict(symbol='triangle-up',size=6, line=dict(color='black',width=1))))

fig.add_trace(go.Scatter(x=(dfn4AvxNonFusNonProb_56mpi_9omp.Dirc_Data_Mov*(dfn4AvxNonFusNonProb_56mpi_9omp.MPI_size/4)), 
                         y=(dfn4AvxNonFusNonProb_56mpi_9omp.GFLOPS*(dfn4AvxNonFusNonProb_56mpi_9omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxNonFusNonProb_56mpi_9omp',
                         line=dict(color='dimgray',width=3),
                         marker=dict(symbol='triangle-up',size=6, line=dict(color='black',width=1))))



# Case 7
fig.add_trace(go.Scatter(x=(dfn4AvxNonFusProb_248mpi_2omp.Dirc_Data_Mov*(dfn4AvxNonFusProb_248mpi_2omp.MPI_size/4)), 
                         y=(dfn4AvxNonFusProb_248mpi_2omp.GFLOPS*(dfn4AvxNonFusProb_248mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxNonFusProb_248mpi_2omp',
                         line=dict(color='olive',width=3),
                         marker=dict(symbol='circle',size=6, line=dict(color='olive',width=2))))

fig.add_trace(go.Scatter(x=(dfn4AvxNonFusNonProb_248mpi_2omp.Dirc_Data_Mov*(dfn4AvxNonFusNonProb_248mpi_2omp.MPI_size/4)), 
                         y=(dfn4AvxNonFusNonProb_248mpi_2omp.GFLOPS*(dfn4AvxNonFusNonProb_248mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxNonFusNonProb_248mpi_2omp',
                         line=dict(color='Brown',width=3),
                         marker=dict(symbol='circle',size=6, line=dict(color='darkred',width=2))))




# Adding a vertical line
fig.add_vline(x=512e6, line_width=3, line_dash="dash", line_color="green")
# fig.add_hline(y=1024, line_width=3, line_dash="dash", line_color="red")

# Add annotation
fig.add_annotation(
    x='512e6'
    , y=130
    , text=f'LLC Size Per Node'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="purple", family="Sans Serif")
    , align="left"
    ,)

fig.update_layout(title='"NonFused-AVX (4-Nodes): probempi vs nonprobempi" GFlops Measurement',
                  xaxis_title='Dirc_Data_Mov', yaxis_title='GFLOPs/Node', 
                  xaxis=dict(ticks="outside"),  
                  yaxis=dict(ticks="outside"))

## BANDWIDTH - NonFused-AVX: Probmpi vs Non-Probempi

In [42]:
fig = go.Figure() 

# Case 1
fig.add_trace(go.Scatter(x=(dfn4AvxNonFusProb_512mpi_1omp.Dirc_Data_Mov*(dfn4AvxNonFusProb_512mpi_1omp.MPI_size/4)), 
                         y=(dfn4AvxNonFusProb_512mpi_1omp.BAND*(dfn4AvxNonFusProb_512mpi_1omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxNonFusProb_512mpi_1omp',
                         line=dict(color='darkcyan',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))

fig.add_trace(go.Scatter(x=(dfn4AvxNonFusNonProb_512mpi_1omp.Dirc_Data_Mov*(dfn4AvxNonFusNonProb_512mpi_1omp.MPI_size/4)), 
                         y=(dfn4AvxNonFusNonProb_512mpi_1omp.BAND*(dfn4AvxNonFusNonProb_512mpi_1omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxNonFusNonProb_512mpi_1omp',
                         line=dict(color='magenta',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))


# Case 2
fig.add_trace(go.Scatter(x=(dfn4AvxNonFusProb_256mpi_2omp.Dirc_Data_Mov*(dfn4AvxNonFusProb_256mpi_2omp.MPI_size/4)), 
                         y=(dfn4AvxNonFusProb_256mpi_2omp.BAND*(dfn4AvxNonFusProb_256mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxNonFusProb_256mpi_2omp',
                         line=dict(color='royalblue',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))

fig.add_trace(go.Scatter(x=(dfn4AvxNonFusNonProb_256mpi_2omp.Dirc_Data_Mov*(dfn4AvxNonFusNonProb_256mpi_2omp.MPI_size/4)), 
                         y=(dfn4AvxNonFusNonProb_256mpi_2omp.BAND*(dfn4AvxNonFusNonProb_256mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxNonFusNonProb_256mpi_2omp',
                         line=dict(color='sandybrown',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))


# Case 3
fig.add_trace(go.Scatter(x=(dfn4AvxNonFusProb_128mpi_4omp.Dirc_Data_Mov*(dfn4AvxNonFusProb_128mpi_4omp.MPI_size/4)), 
                         y=(dfn4AvxNonFusProb_128mpi_4omp.BAND*(dfn4AvxNonFusProb_128mpi_4omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxNonFusProb_128mpi_4omp',
                         line=dict(color='darkblue',width=3),
                         marker=dict(symbol='circle',size=7, line=dict(color='tomato',width=2))))

fig.add_trace(go.Scatter(x=(dfn4AvxNonFusNonProb_128mpi_4omp.Dirc_Data_Mov*(dfn4AvxNonFusNonProb_128mpi_4omp.MPI_size/4)), 
                         y=(dfn4AvxNonFusNonProb_128mpi_4omp.BAND*(dfn4AvxNonFusNonProb_128mpi_4omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxNonFusNonProb_128mpi_4omp',
                         line=dict(color='cyan',width=3),
                         marker=dict(symbol='circle',size=7,line=dict(color='Purple',width=2))))



# Case 4
fig.add_trace(go.Scatter(x=(dfn4AvxNonFusProb_64mpi_8omp.Dirc_Data_Mov*(dfn4AvxNonFusProb_64mpi_8omp.MPI_size/4)), 
                         y=(dfn4AvxNonFusProb_64mpi_8omp.BAND*(dfn4AvxNonFusProb_64mpi_8omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxNonFusProb_64mpi_8omp',
                         line=dict(color='blue',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))

fig.add_trace(go.Scatter(x=(dfn4AvxNonFusNonProb_64mpi_8omp.Dirc_Data_Mov*(dfn4AvxNonFusNonProb_64mpi_8omp.MPI_size/4)), 
                         y=(dfn4AvxNonFusNonProb_64mpi_8omp.BAND*(dfn4AvxNonFusNonProb_64mpi_8omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxNonFusNonProb_64mpi_8omp',
                         line=dict(color='red',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))


# Case 5
fig.add_trace(go.Scatter(x=(dfn4AvxNonFusProb_32mpi_16omp.Dirc_Data_Mov*(dfn4AvxNonFusProb_32mpi_16omp.MPI_size/4)), 
                         y=(dfn4AvxNonFusProb_32mpi_16omp.BAND*(dfn4AvxNonFusProb_32mpi_16omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxNonFusProb_32mpi_16omp',
                         line=dict(color='chartreuse',width=3),
                         marker=dict(symbol='square-dot',size=6, line=dict(color='mediumvioletred',width=1))))

fig.add_trace(go.Scatter(x=(dfn4AvxNonFusNonProb_32mpi_16omp.Dirc_Data_Mov*(dfn4AvxNonFusNonProb_32mpi_16omp.MPI_size/4)), 
                         y=(dfn4AvxNonFusNonProb_32mpi_16omp.BAND*(dfn4AvxNonFusNonProb_32mpi_16omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxNonFusNonProb_32mpi_16omp',
                         line=dict(color='black',width=3),
                         marker=dict(symbol='square-dot',size=6, line=dict(color='mediumvioletred',width=1))))


# Case 6
fig.add_trace(go.Scatter(x=(dfn4AvxNonFusProb_56mpi_9omp.Dirc_Data_Mov*(dfn4AvxNonFusProb_56mpi_9omp.MPI_size/4)), 
                         y=(dfn4AvxNonFusProb_56mpi_9omp.BAND*(dfn4AvxNonFusProb_56mpi_9omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxNonFusProb_56mpi_9omp',
                         line=dict(color='gold',width=3),
                         marker=dict(symbol='triangle-up',size=6, line=dict(color='black',width=1))))

fig.add_trace(go.Scatter(x=(dfn4AvxNonFusNonProb_56mpi_9omp.Dirc_Data_Mov*(dfn4AvxNonFusNonProb_56mpi_9omp.MPI_size/4)), 
                         y=(dfn4AvxNonFusNonProb_56mpi_9omp.BAND*(dfn4AvxNonFusNonProb_56mpi_9omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxNonFusNonProb_56mpi_9omp',
                         line=dict(color='dimgray',width=3),
                         marker=dict(symbol='triangle-up',size=6, line=dict(color='black',width=1))))



# Case 7
fig.add_trace(go.Scatter(x=(dfn4AvxNonFusProb_248mpi_2omp.Dirc_Data_Mov*(dfn4AvxNonFusProb_248mpi_2omp.MPI_size/4)), 
                         y=(dfn4AvxNonFusProb_248mpi_2omp.BAND*(dfn4AvxNonFusProb_248mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxNonFusProb_248mpi_2omp',
                         line=dict(color='olive',width=3),
                         marker=dict(symbol='circle',size=6, line=dict(color='olive',width=2))))

fig.add_trace(go.Scatter(x=(dfn4AvxNonFusNonProb_248mpi_2omp.Dirc_Data_Mov*(dfn4AvxNonFusNonProb_248mpi_2omp.MPI_size/4)), 
                         y=(dfn4AvxNonFusNonProb_248mpi_2omp.BAND*(dfn4AvxNonFusNonProb_248mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxNonFusNonProb_248mpi_2omp',
                         line=dict(color='Brown',width=3),
                         marker=dict(symbol='circle',size=6, line=dict(color='darkred',width=2))))


# Adding vertical and horizontal lines 
fig.add_vline(x=512e6, line_width=3, line_dash="dot", line_color="green")
fig.add_hline(y=409, line_width=3, line_dash="dash", line_color="red")

# Add annotation with arrow
fig.add_annotation(
    x='512e6'
    , y=280
    , text=f'LLC Size Per Node: 512M'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="purple", family="Sans Serif")
    , align="left"
    ,)

fig.add_annotation(
    x='130e6'
    , y=370
    , text=f'Max Bandwidth Per Node'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="orange", family="Sans Serif")
    , align="left"
    ,)


fig.update_layout(title='"NonFused-AVX: Probmpi vs Non-Probempi" Bandwidth Measurement',
                  xaxis_title='Dirc_Data_Mov', yaxis_title='Bandwidth/Node', 
                  xaxis=dict(ticks="outside"),  
                  yaxis=dict(ticks="outside"),
                  legend=go.layout.Legend(bordercolor="black",borderwidth=1)) 







## 5) Newgeo_AVX_Probempi

### Adding Columns

In [43]:
dfn4_avx_newgeo_probempi["N_Nodes"] = 4
dfn4_avx_newgeo_probempi['NPX'] = (dfn4_avx_newgeo_probempi.GLB_X//dfn4_avx_newgeo_probempi.LOC_X)
dfn4_avx_newgeo_probempi['NPY'] = (dfn4_avx_newgeo_probempi.GLB_Y//dfn4_avx_newgeo_probempi.LOC_Y)
dfn4_avx_newgeo_probempi['NPZ'] = (dfn4_avx_newgeo_probempi.GLB_Z//dfn4_avx_newgeo_probempi.LOC_Z)
dfn4_avx_newgeo_probempi['NPT'] = (dfn4_avx_newgeo_probempi.GLB_T//dfn4_avx_newgeo_probempi.LOC_T)
dfn4_avx_newgeo_probempi["Total_proc"] = dfn4_avx_newgeo_probempi["omp_thrd"]* dfn4_avx_newgeo_probempi.MPI_size
# dfn4_avx_newgeo_probempi.columns

### Data Summary

In [44]:
dfn4_avx_newgeo_probempi_summary = dfn4_avx_newgeo_probempi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn4_avx_newgeo_probempi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,32,16,4,2,2,2,512,139
1,56,9,7,2,2,2,504,39
2,64,8,8,2,2,2,512,69
3,72,7,9,2,2,2,504,31
4,128,4,16,2,2,2,512,34
5,168,3,21,2,2,2,504,14
6,248,2,31,2,2,2,496,9
7,256,2,32,2,2,2,512,17
8,488,1,61,2,2,2,488,4
9,496,1,62,2,2,2,496,7


### Cases Setup

In [45]:
dfn4AvxNewgeoProb_32mpi_16omp = dfn4_avx_newgeo_probempi[(dfn4_avx_newgeo_probempi.omp_thrd == 16) & (dfn4_avx_newgeo_probempi.GLB_T/dfn4_avx_newgeo_probempi.LOC_T == 4) & (dfn4_avx_newgeo_probempi.GLB_X/dfn4_avx_newgeo_probempi.LOC_X == 2) & (dfn4_avx_newgeo_probempi.GLB_Y/dfn4_avx_newgeo_probempi.LOC_Y == 2)& (dfn4_avx_newgeo_probempi.GLB_Z/dfn4_avx_newgeo_probempi.LOC_Z == 2)]
dfn4AvxNewgeoProb_64mpi_8omp = dfn4_avx_newgeo_probempi[(dfn4_avx_newgeo_probempi.omp_thrd == 8) & (dfn4_avx_newgeo_probempi.GLB_T/dfn4_avx_newgeo_probempi.LOC_T == 8) & (dfn4_avx_newgeo_probempi.GLB_X/dfn4_avx_newgeo_probempi.LOC_X == 2) & (dfn4_avx_newgeo_probempi.GLB_Y/dfn4_avx_newgeo_probempi.LOC_Y == 2)& (dfn4_avx_newgeo_probempi.GLB_Z/dfn4_avx_newgeo_probempi.LOC_Z == 2)]
dfn4AvxNewgeoProb_128mpi_4omp = dfn4_avx_newgeo_probempi[(dfn4_avx_newgeo_probempi.omp_thrd == 4) & (dfn4_avx_newgeo_probempi.GLB_T/dfn4_avx_newgeo_probempi.LOC_T == 16) & (dfn4_avx_newgeo_probempi.GLB_X/dfn4_avx_newgeo_probempi.LOC_X == 2) & (dfn4_avx_newgeo_probempi.GLB_Y/dfn4_avx_newgeo_probempi.LOC_Y == 2)& (dfn4_avx_newgeo_probempi.GLB_Z/dfn4_avx_newgeo_probempi.LOC_Z == 2)]
dfn4AvxNewgeoProb_256mpi_2omp = dfn4_avx_newgeo_probempi[(dfn4_avx_newgeo_probempi.omp_thrd == 2) & (dfn4_avx_newgeo_probempi.GLB_T/dfn4_avx_newgeo_probempi.LOC_T == 32) & (dfn4_avx_newgeo_probempi.GLB_X/dfn4_avx_newgeo_probempi.LOC_X == 2) & (dfn4_avx_newgeo_probempi.GLB_Y/dfn4_avx_newgeo_probempi.LOC_Y == 2)& (dfn4_avx_newgeo_probempi.GLB_Z/dfn4_avx_newgeo_probempi.LOC_Z == 2)]
dfn4AvxNewgeoProb_512mpi_1omp = dfn4_avx_newgeo_probempi[(dfn4_avx_newgeo_probempi.omp_thrd == 1) & (dfn4_avx_newgeo_probempi.GLB_T/dfn4_avx_newgeo_probempi.LOC_T == 64) & (dfn4_avx_newgeo_probempi.GLB_X/dfn4_avx_newgeo_probempi.LOC_X == 2) & (dfn4_avx_newgeo_probempi.GLB_Y/dfn4_avx_newgeo_probempi.LOC_Y == 2)& (dfn4_avx_newgeo_probempi.GLB_Z/dfn4_avx_newgeo_probempi.LOC_Z == 2)]
dfn4AvxNewgeoProb_56mpi_9omp = dfn4_avx_newgeo_probempi[(dfn4_avx_newgeo_probempi.omp_thrd == 9) & (dfn4_avx_newgeo_probempi.GLB_T/dfn4_avx_newgeo_probempi.LOC_T == 7) & (dfn4_avx_newgeo_probempi.GLB_X/dfn4_avx_newgeo_probempi.LOC_X == 2) & (dfn4_avx_newgeo_probempi.GLB_Y/dfn4_avx_newgeo_probempi.LOC_Y == 2)& (dfn4_avx_newgeo_probempi.GLB_Z/dfn4_avx_newgeo_probempi.LOC_Z == 2)]
dfn4AvxNewgeoProb_248mpi_2omp = dfn4_avx_newgeo_probempi[(dfn4_avx_newgeo_probempi.omp_thrd == 2) & (dfn4_avx_newgeo_probempi.GLB_T/dfn4_avx_newgeo_probempi.LOC_T == 31) & (dfn4_avx_newgeo_probempi.GLB_X/dfn4_avx_newgeo_probempi.LOC_X == 2) & (dfn4_avx_newgeo_probempi.GLB_Y/dfn4_avx_newgeo_probempi.LOC_Y == 2)& (dfn4_avx_newgeo_probempi.GLB_Z/dfn4_avx_newgeo_probempi.LOC_Z == 2)]


## 6) Newgeo_AVX_NonProbempi

### Adding Columns

In [46]:
dfn4_avx_newgeo_nonprobempi["N_Nodes"] = 4
dfn4_avx_newgeo_nonprobempi['NPX'] = (dfn4_avx_newgeo_nonprobempi.GLB_X//dfn4_avx_newgeo_nonprobempi.LOC_X)
dfn4_avx_newgeo_nonprobempi['NPY'] = (dfn4_avx_newgeo_nonprobempi.GLB_Y//dfn4_avx_newgeo_nonprobempi.LOC_Y)
dfn4_avx_newgeo_nonprobempi['NPZ'] = (dfn4_avx_newgeo_nonprobempi.GLB_Z//dfn4_avx_newgeo_nonprobempi.LOC_Z)
dfn4_avx_newgeo_nonprobempi['NPT'] = (dfn4_avx_newgeo_nonprobempi.GLB_T//dfn4_avx_newgeo_nonprobempi.LOC_T)
dfn4_avx_newgeo_nonprobempi["Total_proc"] = dfn4_avx_newgeo_nonprobempi["omp_thrd"]* dfn4_avx_newgeo_nonprobempi.MPI_size
# dfn4_avx_newgeo_nonprobempi.columns

### Data Summary

In [47]:
dfn4_avx_newgeo_nonprobempi_summary = dfn4_avx_newgeo_nonprobempi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn4_avx_newgeo_nonprobempi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,32,16,4,2,2,2,512,138
1,56,9,7,2,2,2,504,40
2,64,8,8,2,2,2,512,68
3,72,7,9,2,2,2,504,31
4,128,4,16,2,2,2,512,34
5,168,3,21,2,2,2,504,14
6,248,2,31,2,2,2,496,9
7,256,2,32,2,2,2,512,16
8,488,1,61,2,2,2,488,4
9,496,1,62,2,2,2,496,6


### Cases Setup

In [48]:
dfn4AvxNewgeoNonProb_32mpi_16omp = dfn4_avx_newgeo_nonprobempi[(dfn4_avx_newgeo_nonprobempi.omp_thrd == 16) & (dfn4_avx_newgeo_nonprobempi.GLB_T/dfn4_avx_newgeo_nonprobempi.LOC_T == 4) & (dfn4_avx_newgeo_nonprobempi.GLB_X/dfn4_avx_newgeo_nonprobempi.LOC_X == 2) & (dfn4_avx_newgeo_nonprobempi.GLB_Y/dfn4_avx_newgeo_nonprobempi.LOC_Y == 2)& (dfn4_avx_newgeo_nonprobempi.GLB_Z/dfn4_avx_newgeo_nonprobempi.LOC_Z == 2)]
dfn4AvxNewgeoNonProb_64mpi_8omp = dfn4_avx_newgeo_nonprobempi[(dfn4_avx_newgeo_nonprobempi.omp_thrd == 8) & (dfn4_avx_newgeo_nonprobempi.GLB_T/dfn4_avx_newgeo_nonprobempi.LOC_T == 8) & (dfn4_avx_newgeo_nonprobempi.GLB_X/dfn4_avx_newgeo_nonprobempi.LOC_X == 2) & (dfn4_avx_newgeo_nonprobempi.GLB_Y/dfn4_avx_newgeo_nonprobempi.LOC_Y == 2)& (dfn4_avx_newgeo_nonprobempi.GLB_Z/dfn4_avx_newgeo_nonprobempi.LOC_Z == 2)]
dfn4AvxNewgeoNonProb_128mpi_4omp = dfn4_avx_newgeo_nonprobempi[(dfn4_avx_newgeo_nonprobempi.omp_thrd == 4) & (dfn4_avx_newgeo_nonprobempi.GLB_T/dfn4_avx_newgeo_nonprobempi.LOC_T == 16) & (dfn4_avx_newgeo_nonprobempi.GLB_X/dfn4_avx_newgeo_nonprobempi.LOC_X == 2) & (dfn4_avx_newgeo_nonprobempi.GLB_Y/dfn4_avx_newgeo_nonprobempi.LOC_Y == 2)& (dfn4_avx_newgeo_nonprobempi.GLB_Z/dfn4_avx_newgeo_nonprobempi.LOC_Z == 2)]
dfn4AvxNewgeoNonProb_256mpi_2omp = dfn4_avx_newgeo_nonprobempi[(dfn4_avx_newgeo_nonprobempi.omp_thrd == 2) & (dfn4_avx_newgeo_nonprobempi.GLB_T/dfn4_avx_newgeo_nonprobempi.LOC_T == 32) & (dfn4_avx_newgeo_nonprobempi.GLB_X/dfn4_avx_newgeo_nonprobempi.LOC_X == 2) & (dfn4_avx_newgeo_nonprobempi.GLB_Y/dfn4_avx_newgeo_nonprobempi.LOC_Y == 2)& (dfn4_avx_newgeo_nonprobempi.GLB_Z/dfn4_avx_newgeo_nonprobempi.LOC_Z == 2)]
dfn4AvxNewgeoNonProb_512mpi_1omp = dfn4_avx_newgeo_nonprobempi[(dfn4_avx_newgeo_nonprobempi.omp_thrd == 1) & (dfn4_avx_newgeo_nonprobempi.GLB_T/dfn4_avx_newgeo_nonprobempi.LOC_T == 64) & (dfn4_avx_newgeo_nonprobempi.GLB_X/dfn4_avx_newgeo_nonprobempi.LOC_X == 2) & (dfn4_avx_newgeo_nonprobempi.GLB_Y/dfn4_avx_newgeo_nonprobempi.LOC_Y == 2)& (dfn4_avx_newgeo_nonprobempi.GLB_Z/dfn4_avx_newgeo_nonprobempi.LOC_Z == 2)]
dfn4AvxNewgeoNonProb_56mpi_9omp = dfn4_avx_newgeo_nonprobempi[(dfn4_avx_newgeo_nonprobempi.omp_thrd == 9) & (dfn4_avx_newgeo_nonprobempi.GLB_T/dfn4_avx_newgeo_nonprobempi.LOC_T == 7) & (dfn4_avx_newgeo_nonprobempi.GLB_X/dfn4_avx_newgeo_nonprobempi.LOC_X == 2) & (dfn4_avx_newgeo_nonprobempi.GLB_Y/dfn4_avx_newgeo_nonprobempi.LOC_Y == 2)& (dfn4_avx_newgeo_nonprobempi.GLB_Z/dfn4_avx_newgeo_nonprobempi.LOC_Z == 2)]
dfn4AvxNewgeoNonProb_248mpi_2omp = dfn4_avx_newgeo_nonprobempi[(dfn4_avx_newgeo_nonprobempi.omp_thrd == 2) & (dfn4_avx_newgeo_nonprobempi.GLB_T/dfn4_avx_newgeo_nonprobempi.LOC_T == 31) & (dfn4_avx_newgeo_nonprobempi.GLB_X/dfn4_avx_newgeo_nonprobempi.LOC_X == 2) & (dfn4_avx_newgeo_nonprobempi.GLB_Y/dfn4_avx_newgeo_nonprobempi.LOC_Y == 2)& (dfn4_avx_newgeo_nonprobempi.GLB_Z/dfn4_avx_newgeo_nonprobempi.LOC_Z == 2)]


## GFLOPS - Newgeo-AVX: Probmpi vs Non-Probempi

In [49]:
fig = go.Figure() 

# Case 1
fig.add_trace(go.Scatter(x=(dfn4AvxNewgeoProb_512mpi_1omp.Dirc_Data_Mov*(dfn4AvxNewgeoProb_512mpi_1omp.MPI_size/4)), 
                         y=(dfn4AvxNewgeoProb_512mpi_1omp.GFLOPS*(dfn4AvxNewgeoProb_512mpi_1omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxNewgeoProb_512mpi_1omp',
                         line=dict(color='darkcyan',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))

fig.add_trace(go.Scatter(x=(dfn4AvxNewgeoNonProb_512mpi_1omp.Dirc_Data_Mov*(dfn4AvxNewgeoNonProb_512mpi_1omp.MPI_size/4)), 
                         y=(dfn4AvxNewgeoNonProb_512mpi_1omp.GFLOPS*(dfn4AvxNewgeoNonProb_512mpi_1omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxNewgeoNonProb_512mpi_1omp',
                         line=dict(color='magenta',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))


# Case 2
fig.add_trace(go.Scatter(x=(dfn4AvxNewgeoProb_256mpi_2omp.Dirc_Data_Mov*(dfn4AvxNewgeoProb_256mpi_2omp.MPI_size/4)), 
                         y=(dfn4AvxNewgeoProb_256mpi_2omp.GFLOPS*(dfn4AvxNewgeoProb_256mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxNewgeoProb_256mpi_2omp',
                         line=dict(color='royalblue',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))

fig.add_trace(go.Scatter(x=(dfn4AvxNewgeoNonProb_256mpi_2omp.Dirc_Data_Mov*(dfn4AvxNewgeoNonProb_256mpi_2omp.MPI_size/4)), 
                         y=(dfn4AvxNewgeoNonProb_256mpi_2omp.GFLOPS*(dfn4AvxNewgeoNonProb_256mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxNewgeoNonProb_256mpi_2omp',
                         line=dict(color='sandybrown',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))


# Case 3
fig.add_trace(go.Scatter(x=(dfn4AvxNewgeoProb_128mpi_4omp.Dirc_Data_Mov*(dfn4AvxNewgeoProb_128mpi_4omp.MPI_size/4)), 
                         y=(dfn4AvxNewgeoProb_128mpi_4omp.GFLOPS*(dfn4AvxNewgeoProb_128mpi_4omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxNewgeoProb_128mpi_4omp',
                         line=dict(color='darkblue',width=3),
                         marker=dict(symbol='circle',size=7, line=dict(color='tomato',width=2))))

fig.add_trace(go.Scatter(x=(dfn4AvxNewgeoNonProb_128mpi_4omp.Dirc_Data_Mov*(dfn4AvxNewgeoNonProb_128mpi_4omp.MPI_size/4)), 
                         y=(dfn4AvxNewgeoNonProb_128mpi_4omp.GFLOPS*(dfn4AvxNewgeoNonProb_128mpi_4omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxNewgeoNonProb_128mpi_4omp',
                         line=dict(color='cyan',width=3),
                         marker=dict(symbol='circle',size=7,line=dict(color='Purple',width=2))))



# Case 4
fig.add_trace(go.Scatter(x=(dfn4AvxNewgeoProb_64mpi_8omp.Dirc_Data_Mov*(dfn4AvxNewgeoProb_64mpi_8omp.MPI_size/4)), 
                         y=(dfn4AvxNewgeoProb_64mpi_8omp.GFLOPS*(dfn4AvxNewgeoProb_64mpi_8omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxNewgeoProb_64mpi_8omp',
                         line=dict(color='blue',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))

fig.add_trace(go.Scatter(x=(dfn4AvxNewgeoNonProb_64mpi_8omp.Dirc_Data_Mov*(dfn4AvxNewgeoNonProb_64mpi_8omp.MPI_size/4)), 
                         y=(dfn4AvxNewgeoNonProb_64mpi_8omp.GFLOPS*(dfn4AvxNewgeoNonProb_64mpi_8omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxNewgeoNonProb_64mpi_8omp',
                         line=dict(color='red',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))


# Case 5
fig.add_trace(go.Scatter(x=(dfn4AvxNewgeoProb_32mpi_16omp.Dirc_Data_Mov*(dfn4AvxNewgeoProb_32mpi_16omp.MPI_size/4)), 
                         y=(dfn4AvxNewgeoProb_32mpi_16omp.GFLOPS*(dfn4AvxNewgeoProb_32mpi_16omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxNonFusProb_32mpi_16omp',
                         line=dict(color='chartreuse',width=3),
                         marker=dict(symbol='square-dot',size=6, line=dict(color='mediumvioletred',width=1))))

fig.add_trace(go.Scatter(x=(dfn4AvxNewgeoNonProb_32mpi_16omp.Dirc_Data_Mov*(dfn4AvxNewgeoNonProb_32mpi_16omp.MPI_size/4)), 
                         y=(dfn4AvxNewgeoNonProb_32mpi_16omp.GFLOPS*(dfn4AvxNewgeoNonProb_32mpi_16omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxNewgeoNonProb_32mpi_16omp',
                         line=dict(color='black',width=3),
                         marker=dict(symbol='square-dot',size=6, line=dict(color='mediumvioletred',width=1))))


# Case 6
fig.add_trace(go.Scatter(x=(dfn4AvxNewgeoProb_56mpi_9omp.Dirc_Data_Mov*(dfn4AvxNewgeoProb_56mpi_9omp.MPI_size/4)), 
                         y=(dfn4AvxNewgeoProb_56mpi_9omp.GFLOPS*(dfn4AvxNewgeoProb_56mpi_9omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxNewgeoProb_56mpi_9omp',
                         line=dict(color='gold',width=3),
                         marker=dict(symbol='triangle-up',size=6, line=dict(color='black',width=1))))

fig.add_trace(go.Scatter(x=(dfn4AvxNewgeoNonProb_56mpi_9omp.Dirc_Data_Mov*(dfn4AvxNewgeoNonProb_56mpi_9omp.MPI_size/4)), 
                         y=(dfn4AvxNewgeoNonProb_56mpi_9omp.GFLOPS*(dfn4AvxNewgeoNonProb_56mpi_9omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxNewgeoNonProb_56mpi_9omp',
                         line=dict(color='dimgray',width=3),
                         marker=dict(symbol='triangle-up',size=6, line=dict(color='black',width=1))))



# Case 7
fig.add_trace(go.Scatter(x=(dfn4AvxNewgeoProb_248mpi_2omp.Dirc_Data_Mov*(dfn4AvxNewgeoProb_248mpi_2omp.MPI_size/4)), 
                         y=(dfn4AvxNewgeoProb_248mpi_2omp.GFLOPS*(dfn4AvxNewgeoProb_248mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxNewgeoProb_248mpi_2omp',
                         line=dict(color='olive',width=3),
                         marker=dict(symbol='circle',size=6, line=dict(color='olive',width=2))))

fig.add_trace(go.Scatter(x=(dfn4AvxNewgeoNonProb_248mpi_2omp.Dirc_Data_Mov*(dfn4AvxNewgeoNonProb_248mpi_2omp.MPI_size/4)), 
                         y=(dfn4AvxNewgeoNonProb_248mpi_2omp.GFLOPS*(dfn4AvxNewgeoNonProb_248mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxNewgeoNonProb_248mpi_2omp',
                         line=dict(color='Brown',width=3),
                         marker=dict(symbol='circle',size=6, line=dict(color='darkred',width=2))))




# Adding a vertical line
fig.add_vline(x=512e6, line_width=3, line_dash="dash", line_color="green")
# fig.add_hline(y=1024, line_width=3, line_dash="dash", line_color="red")

# Add annotation
fig.add_annotation(
    x='512e6'
    , y=130
    , text=f'LLC Size Per Node'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="purple", family="Sans Serif")
    , align="left"
    ,)

fig.update_layout(title='"Newgeo-AVX (4-Nodes): probempi vs nonprobempi" GFlops Measurement',
                  xaxis_title='Dirc_Data_Mov', yaxis_title='GFLOPs/Node', 
                  xaxis=dict(ticks="outside"),  
                  yaxis=dict(ticks="outside"))

## BANDWIDTH - Newgeo-AVX: Probmpi vs Non-Probempi

In [50]:
fig = go.Figure() 

# Case 1
fig.add_trace(go.Scatter(x=(dfn4AvxNewgeoProb_512mpi_1omp.Dirc_Data_Mov*(dfn4AvxNewgeoProb_512mpi_1omp.MPI_size/4)), 
                         y=(dfn4AvxNewgeoProb_512mpi_1omp.BAND*(dfn4AvxNewgeoProb_512mpi_1omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxNewgeoProb_512mpi_1omp',
                         line=dict(color='darkcyan',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))

fig.add_trace(go.Scatter(x=(dfn4AvxNewgeoNonProb_512mpi_1omp.Dirc_Data_Mov*(dfn4AvxNewgeoNonProb_512mpi_1omp.MPI_size/4)), 
                         y=(dfn4AvxNewgeoNonProb_512mpi_1omp.BAND*(dfn4AvxNewgeoNonProb_512mpi_1omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxNewgeoNonProb_512mpi_1omp',
                         line=dict(color='magenta',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))


# Case 2
fig.add_trace(go.Scatter(x=(dfn4AvxNewgeoProb_256mpi_2omp.Dirc_Data_Mov*(dfn4AvxNewgeoProb_256mpi_2omp.MPI_size/4)), 
                         y=(dfn4AvxNewgeoProb_256mpi_2omp.BAND*(dfn4AvxNewgeoProb_256mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxNewgeoProb_256mpi_2omp',
                         line=dict(color='royalblue',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))

fig.add_trace(go.Scatter(x=(dfn4AvxNewgeoNonProb_256mpi_2omp.Dirc_Data_Mov*(dfn4AvxNewgeoNonProb_256mpi_2omp.MPI_size/4)), 
                         y=(dfn4AvxNewgeoNonProb_256mpi_2omp.BAND*(dfn4AvxNewgeoNonProb_256mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxNewgeoNonProb_256mpi_2omp',
                         line=dict(color='sandybrown',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))


# Case 3
fig.add_trace(go.Scatter(x=(dfn4AvxNewgeoProb_128mpi_4omp.Dirc_Data_Mov*(dfn4AvxNewgeoProb_128mpi_4omp.MPI_size/4)), 
                         y=(dfn4AvxNewgeoProb_128mpi_4omp.BAND*(dfn4AvxNewgeoProb_128mpi_4omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxNewgeoProb_128mpi_4omp',
                         line=dict(color='darkblue',width=3),
                         marker=dict(symbol='circle',size=7, line=dict(color='tomato',width=2))))

fig.add_trace(go.Scatter(x=(dfn4AvxNewgeoNonProb_128mpi_4omp.Dirc_Data_Mov*(dfn4AvxNewgeoNonProb_128mpi_4omp.MPI_size/4)), 
                         y=(dfn4AvxNewgeoNonProb_128mpi_4omp.BAND*(dfn4AvxNewgeoNonProb_128mpi_4omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxNewgeoNonProb_128mpi_4omp',
                         line=dict(color='cyan',width=3),
                         marker=dict(symbol='circle',size=7,line=dict(color='Purple',width=2))))



# Case 4
fig.add_trace(go.Scatter(x=(dfn4AvxNewgeoProb_64mpi_8omp.Dirc_Data_Mov*(dfn4AvxNewgeoProb_64mpi_8omp.MPI_size/4)), 
                         y=(dfn4AvxNewgeoProb_64mpi_8omp.BAND*(dfn4AvxNewgeoProb_64mpi_8omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxNewgeoProb_64mpi_8omp',
                         line=dict(color='blue',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))

fig.add_trace(go.Scatter(x=(dfn4AvxNewgeoNonProb_64mpi_8omp.Dirc_Data_Mov*(dfn4AvxNewgeoNonProb_64mpi_8omp.MPI_size/4)), 
                         y=(dfn4AvxNewgeoNonProb_64mpi_8omp.BAND*(dfn4AvxNewgeoNonProb_64mpi_8omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxNewgeoNonProb_64mpi_8omp',
                         line=dict(color='red',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))


# Case 5
fig.add_trace(go.Scatter(x=(dfn4AvxNewgeoProb_32mpi_16omp.Dirc_Data_Mov*(dfn4AvxNewgeoProb_32mpi_16omp.MPI_size/4)), 
                         y=(dfn4AvxNewgeoProb_32mpi_16omp.BAND*(dfn4AvxNewgeoProb_32mpi_16omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxNonFusProb_32mpi_16omp',
                         line=dict(color='chartreuse',width=3),
                         marker=dict(symbol='square-dot',size=6, line=dict(color='mediumvioletred',width=1))))

fig.add_trace(go.Scatter(x=(dfn4AvxNewgeoNonProb_32mpi_16omp.Dirc_Data_Mov*(dfn4AvxNewgeoNonProb_32mpi_16omp.MPI_size/4)), 
                         y=(dfn4AvxNewgeoNonProb_32mpi_16omp.BAND*(dfn4AvxNewgeoNonProb_32mpi_16omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxNewgeoNonProb_32mpi_16omp',
                         line=dict(color='black',width=3),
                         marker=dict(symbol='square-dot',size=6, line=dict(color='mediumvioletred',width=1))))


# Case 6
fig.add_trace(go.Scatter(x=(dfn4AvxNewgeoProb_56mpi_9omp.Dirc_Data_Mov*(dfn4AvxNewgeoProb_56mpi_9omp.MPI_size/4)), 
                         y=(dfn4AvxNewgeoProb_56mpi_9omp.BAND*(dfn4AvxNewgeoProb_56mpi_9omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxNewgeoProb_56mpi_9omp',
                         line=dict(color='gold',width=3),
                         marker=dict(symbol='triangle-up',size=6, line=dict(color='black',width=1))))

fig.add_trace(go.Scatter(x=(dfn4AvxNewgeoNonProb_56mpi_9omp.Dirc_Data_Mov*(dfn4AvxNewgeoNonProb_56mpi_9omp.MPI_size/4)), 
                         y=(dfn4AvxNewgeoNonProb_56mpi_9omp.BAND*(dfn4AvxNewgeoNonProb_56mpi_9omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxNewgeoNonProb_56mpi_9omp',
                         line=dict(color='dimgray',width=3),
                         marker=dict(symbol='triangle-up',size=6, line=dict(color='black',width=1))))



# Case 7
fig.add_trace(go.Scatter(x=(dfn4AvxNewgeoProb_248mpi_2omp.Dirc_Data_Mov*(dfn4AvxNewgeoProb_248mpi_2omp.MPI_size/4)), 
                         y=(dfn4AvxNewgeoProb_248mpi_2omp.BAND*(dfn4AvxNewgeoProb_248mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxNewgeoProb_248mpi_2omp',
                         line=dict(color='olive',width=3),
                         marker=dict(symbol='circle',size=6, line=dict(color='olive',width=2))))

fig.add_trace(go.Scatter(x=(dfn4AvxNewgeoNonProb_248mpi_2omp.Dirc_Data_Mov*(dfn4AvxNewgeoNonProb_248mpi_2omp.MPI_size/4)), 
                         y=(dfn4AvxNewgeoNonProb_248mpi_2omp.BAND*(dfn4AvxNewgeoNonProb_248mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='AvxNewgeoNonProb_248mpi_2omp',
                         line=dict(color='Brown',width=3),
                         marker=dict(symbol='circle',size=6, line=dict(color='darkred',width=2))))




# Adding vertical and horizontal lines 
fig.add_vline(x=512e6, line_width=3, line_dash="dot", line_color="green")
fig.add_hline(y=409, line_width=3, line_dash="dash", line_color="red")

# Add annotation with arrow
fig.add_annotation(
    x='512e6'
    , y=280
    , text=f'LLC Size Per Node: 512M'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="purple", family="Sans Serif")
    , align="left"
    ,)

fig.add_annotation(
    x='130e6'
    , y=370
    , text=f'Max Bandwidth Per Node'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="orange", family="Sans Serif")
    , align="left"
    ,)


fig.update_layout(title='"Newgeo-AVX: Probmpi vs Non-Probempi" Bandwidth Measurement',
                  xaxis_title='Dirc_Data_Mov', yaxis_title='Bandwidth/Node', 
                  xaxis=dict(ticks="outside"),  
                  yaxis=dict(ticks="outside"),
                  legend=go.layout.Legend(bordercolor="black",borderwidth=1)) 



# HIPPO VECT CASES

## 1) Fused_VECT_Probmpi

### Adding Columns

In [51]:
dfn4_vect_fused_probmpi["N_Nodes"] = 4
dfn4_vect_fused_probmpi['NPX'] = (dfn4_vect_fused_probmpi.GLB_X//dfn4_vect_fused_probmpi.LOC_X)
dfn4_vect_fused_probmpi['NPY'] = (dfn4_vect_fused_probmpi.GLB_Y//dfn4_vect_fused_probmpi.LOC_Y)
dfn4_vect_fused_probmpi['NPZ'] = (dfn4_vect_fused_probmpi.GLB_Z//dfn4_vect_fused_probmpi.LOC_Z)
dfn4_vect_fused_probmpi['NPT'] = (dfn4_vect_fused_probmpi.GLB_T//dfn4_vect_fused_probmpi.LOC_T)
dfn4_vect_fused_probmpi["Total_proc"] = dfn4_vect_fused_probmpi["omp_thrd"]* dfn4_vect_fused_probmpi.MPI_size
# dfn4_vect_fused_probmpi.columns

### Data Summary

In [52]:
dfn4_vect_fused_probmpi_summary = dfn4_vect_fused_probmpi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn4_vect_fused_probmpi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,32,16,4,2,2,2,512,139
1,56,9,7,2,2,2,504,38
2,64,8,8,2,2,2,512,70
3,72,7,9,2,2,2,504,30
4,128,4,16,2,2,2,512,33
5,168,3,21,2,2,2,504,14
6,248,2,31,2,2,2,496,9
7,256,2,32,2,2,2,512,15
8,488,1,61,2,2,2,488,3
9,496,1,62,2,2,2,496,7


### Cases Setup

In [53]:
dfn4VectFusProb_32mpi_16omp = dfn4_vect_fused_probmpi[(dfn4_vect_fused_probmpi.omp_thrd == 16) & (dfn4_vect_fused_probmpi.GLB_T/dfn4_vect_fused_probmpi.LOC_T == 4) & (dfn4_vect_fused_probmpi.GLB_X/dfn4_vect_fused_probmpi.LOC_X == 2) & (dfn4_vect_fused_probmpi.GLB_Y/dfn4_vect_fused_probmpi.LOC_Y == 2)& (dfn4_vect_fused_probmpi.GLB_Z/dfn4_vect_fused_probmpi.LOC_Z == 2)]
dfn4VectFusProb_64mpi_8omp = dfn4_vect_fused_probmpi[(dfn4_vect_fused_probmpi.omp_thrd == 8) & (dfn4_vect_fused_probmpi.GLB_T/dfn4_vect_fused_probmpi.LOC_T == 8) & (dfn4_vect_fused_probmpi.GLB_X/dfn4_vect_fused_probmpi.LOC_X == 2) & (dfn4_vect_fused_probmpi.GLB_Y/dfn4_vect_fused_probmpi.LOC_Y == 2)& (dfn4_vect_fused_probmpi.GLB_Z/dfn4_vect_fused_probmpi.LOC_Z == 2)]
dfn4VectFusProb_128mpi_4omp = dfn4_vect_fused_probmpi[(dfn4_vect_fused_probmpi.omp_thrd == 4) & (dfn4_vect_fused_probmpi.GLB_T/dfn4_vect_fused_probmpi.LOC_T == 16) & (dfn4_vect_fused_probmpi.GLB_X/dfn4_vect_fused_probmpi.LOC_X == 2) & (dfn4_vect_fused_probmpi.GLB_Y/dfn4_vect_fused_probmpi.LOC_Y == 2)& (dfn4_vect_fused_probmpi.GLB_Z/dfn4_vect_fused_probmpi.LOC_Z == 2)]
dfn4VectFusProb_256mpi_2omp = dfn4_vect_fused_probmpi[(dfn4_vect_fused_probmpi.omp_thrd == 2) & (dfn4_vect_fused_probmpi.GLB_T/dfn4_vect_fused_probmpi.LOC_T == 32) & (dfn4_vect_fused_probmpi.GLB_X/dfn4_vect_fused_probmpi.LOC_X == 2) & (dfn4_vect_fused_probmpi.GLB_Y/dfn4_vect_fused_probmpi.LOC_Y == 2)& (dfn4_vect_fused_probmpi.GLB_Z/dfn4_vect_fused_probmpi.LOC_Z == 2)]
dfn4VectFusProb_512mpi_1omp = dfn4_vect_fused_probmpi[(dfn4_vect_fused_probmpi.omp_thrd == 1) & (dfn4_vect_fused_probmpi.GLB_T/dfn4_vect_fused_probmpi.LOC_T == 64) & (dfn4_vect_fused_probmpi.GLB_X/dfn4_vect_fused_probmpi.LOC_X == 2) & (dfn4_vect_fused_probmpi.GLB_Y/dfn4_vect_fused_probmpi.LOC_Y == 2)& (dfn4_vect_fused_probmpi.GLB_Z/dfn4_vect_fused_probmpi.LOC_Z == 2)]
dfn4VectFusProb_56mpi_9omp = dfn4_vect_fused_probmpi[(dfn4_vect_fused_probmpi.omp_thrd == 9) & (dfn4_vect_fused_probmpi.GLB_T/dfn4_vect_fused_probmpi.LOC_T == 7) & (dfn4_vect_fused_probmpi.GLB_X/dfn4_vect_fused_probmpi.LOC_X == 2) & (dfn4_vect_fused_probmpi.GLB_Y/dfn4_vect_fused_probmpi.LOC_Y == 2)& (dfn4_vect_fused_probmpi.GLB_Z/dfn4_vect_fused_probmpi.LOC_Z == 2)]
dfn4VectFusProb_248mpi_2omp = dfn4_vect_fused_probmpi[(dfn4_vect_fused_probmpi.omp_thrd == 2) & (dfn4_vect_fused_probmpi.GLB_T/dfn4_vect_fused_probmpi.LOC_T == 31) & (dfn4_vect_fused_probmpi.GLB_X/dfn4_vect_fused_probmpi.LOC_X == 2) & (dfn4_vect_fused_probmpi.GLB_Y/dfn4_vect_fused_probmpi.LOC_Y == 2)& (dfn4_vect_fused_probmpi.GLB_Z/dfn4_vect_fused_probmpi.LOC_Z == 2)]


## 2) Fused_VECT_NonProbmpi

### Adding Columns

In [54]:
dfn4_vect_fused_nonprobmpi["N_Nodes"] = 4
dfn4_vect_fused_nonprobmpi['NPX'] = (dfn4_vect_fused_nonprobmpi.GLB_X//dfn4_vect_fused_nonprobmpi.LOC_X)
dfn4_vect_fused_nonprobmpi['NPY'] = (dfn4_vect_fused_nonprobmpi.GLB_Y//dfn4_vect_fused_nonprobmpi.LOC_Y)
dfn4_vect_fused_nonprobmpi['NPZ'] = (dfn4_vect_fused_nonprobmpi.GLB_Z//dfn4_vect_fused_nonprobmpi.LOC_Z)
dfn4_vect_fused_nonprobmpi['NPT'] = (dfn4_vect_fused_nonprobmpi.GLB_T//dfn4_vect_fused_nonprobmpi.LOC_T)
dfn4_vect_fused_nonprobmpi["Total_proc"] = dfn4_vect_fused_nonprobmpi["omp_thrd"]* dfn4_vect_fused_nonprobmpi.MPI_size
# dfn4_vect_fused_nonprobmpi.columns

### Data Summary

In [55]:
dfn4_vect_fused_nonprobmpi_summary = dfn4_vect_fused_nonprobmpi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn4_vect_fused_nonprobmpi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,32,16,4,2,2,2,512,139
1,56,9,7,2,2,2,504,40
2,64,8,8,2,2,2,512,70
3,72,7,9,2,2,2,504,30
4,128,4,16,2,2,2,512,34
5,168,3,21,2,2,2,504,12
6,248,2,31,2,2,2,496,9
7,256,2,32,2,2,2,512,17
8,488,1,61,2,2,2,488,4
9,496,1,62,2,2,2,496,6


### Cases Setup

In [56]:
dfn4VectFusNonProb_32mpi_16omp = dfn4_vect_fused_nonprobmpi[(dfn4_vect_fused_nonprobmpi.omp_thrd == 16) & (dfn4_vect_fused_nonprobmpi.GLB_T/dfn4_vect_fused_nonprobmpi.LOC_T == 4) & (dfn4_vect_fused_nonprobmpi.GLB_X/dfn4_vect_fused_nonprobmpi.LOC_X == 2) & (dfn4_vect_fused_nonprobmpi.GLB_Y/dfn4_vect_fused_nonprobmpi.LOC_Y == 2)& (dfn4_vect_fused_nonprobmpi.GLB_Z/dfn4_vect_fused_nonprobmpi.LOC_Z == 2)]
dfn4VectFusNonProb_64mpi_8omp = dfn4_vect_fused_nonprobmpi[(dfn4_vect_fused_nonprobmpi.omp_thrd == 8) & (dfn4_vect_fused_nonprobmpi.GLB_T/dfn4_vect_fused_nonprobmpi.LOC_T == 8) & (dfn4_vect_fused_nonprobmpi.GLB_X/dfn4_vect_fused_nonprobmpi.LOC_X == 2) & (dfn4_vect_fused_nonprobmpi.GLB_Y/dfn4_vect_fused_nonprobmpi.LOC_Y == 2)& (dfn4_vect_fused_nonprobmpi.GLB_Z/dfn4_vect_fused_nonprobmpi.LOC_Z == 2)]
dfn4VectFusNonProb_128mpi_4omp = dfn4_vect_fused_nonprobmpi[(dfn4_vect_fused_nonprobmpi.omp_thrd == 4) & (dfn4_vect_fused_nonprobmpi.GLB_T/dfn4_vect_fused_nonprobmpi.LOC_T == 16) & (dfn4_vect_fused_nonprobmpi.GLB_X/dfn4_vect_fused_nonprobmpi.LOC_X == 2) & (dfn4_vect_fused_nonprobmpi.GLB_Y/dfn4_vect_fused_nonprobmpi.LOC_Y == 2)& (dfn4_vect_fused_nonprobmpi.GLB_Z/dfn4_vect_fused_nonprobmpi.LOC_Z == 2)]
dfn4VectFusNonProb_256mpi_2omp = dfn4_vect_fused_nonprobmpi[(dfn4_vect_fused_nonprobmpi.omp_thrd == 2) & (dfn4_vect_fused_nonprobmpi.GLB_T/dfn4_vect_fused_nonprobmpi.LOC_T == 32) & (dfn4_vect_fused_nonprobmpi.GLB_X/dfn4_vect_fused_nonprobmpi.LOC_X == 2) & (dfn4_vect_fused_nonprobmpi.GLB_Y/dfn4_vect_fused_nonprobmpi.LOC_Y == 2)& (dfn4_vect_fused_nonprobmpi.GLB_Z/dfn4_vect_fused_nonprobmpi.LOC_Z == 2)]
dfn4VectFusNonProb_512mpi_1omp = dfn4_vect_fused_nonprobmpi[(dfn4_vect_fused_nonprobmpi.omp_thrd == 1) & (dfn4_vect_fused_nonprobmpi.GLB_T/dfn4_vect_fused_nonprobmpi.LOC_T == 64) & (dfn4_vect_fused_nonprobmpi.GLB_X/dfn4_vect_fused_nonprobmpi.LOC_X == 2) & (dfn4_vect_fused_nonprobmpi.GLB_Y/dfn4_vect_fused_nonprobmpi.LOC_Y == 2)& (dfn4_vect_fused_nonprobmpi.GLB_Z/dfn4_vect_fused_nonprobmpi.LOC_Z == 2)]
dfn4VectFusNonProb_56mpi_9omp = dfn4_vect_fused_nonprobmpi[(dfn4_vect_fused_nonprobmpi.omp_thrd == 9) & (dfn4_vect_fused_nonprobmpi.GLB_T/dfn4_vect_fused_nonprobmpi.LOC_T == 7) & (dfn4_vect_fused_nonprobmpi.GLB_X/dfn4_vect_fused_nonprobmpi.LOC_X == 2) & (dfn4_vect_fused_nonprobmpi.GLB_Y/dfn4_vect_fused_nonprobmpi.LOC_Y == 2)& (dfn4_vect_fused_nonprobmpi.GLB_Z/dfn4_vect_fused_nonprobmpi.LOC_Z == 2)]
dfn4VectFusNonProb_248mpi_2omp = dfn4_vect_fused_nonprobmpi[(dfn4_vect_fused_nonprobmpi.omp_thrd == 2) & (dfn4_vect_fused_nonprobmpi.GLB_T/dfn4_vect_fused_nonprobmpi.LOC_T == 31) & (dfn4_vect_fused_nonprobmpi.GLB_X/dfn4_vect_fused_nonprobmpi.LOC_X == 2) & (dfn4_vect_fused_nonprobmpi.GLB_Y/dfn4_vect_fused_nonprobmpi.LOC_Y == 2)& (dfn4_vect_fused_nonprobmpi.GLB_Z/dfn4_vect_fused_nonprobmpi.LOC_Z == 2)]


## GFLOPS - Fused VECT: probempi vs nonprobempi

In [57]:
fig = go.Figure() 

# Case 1
fig.add_trace(go.Scatter(x=(dfn4VectFusProb_512mpi_1omp.Dirc_Data_Mov*(dfn4VectFusProb_512mpi_1omp.MPI_size/4)), 
                         y=(dfn4VectFusProb_512mpi_1omp.GFLOPS*(dfn4VectFusProb_512mpi_1omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='dfn4VectFusProb_512mpi_1omp',
                         line=dict(color='darkcyan',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))

fig.add_trace(go.Scatter(x=(dfn4VectFusNonProb_512mpi_1omp.Dirc_Data_Mov*(dfn4VectFusNonProb_512mpi_1omp.MPI_size/4)), 
                         y=(dfn4VectFusNonProb_512mpi_1omp.GFLOPS*(dfn4VectFusNonProb_512mpi_1omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectFusNonProb_512mpi_1omp',
                         line=dict(color='magenta',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))


# Case 2
fig.add_trace(go.Scatter(x=(dfn4VectFusProb_256mpi_2omp.Dirc_Data_Mov*(dfn4VectFusProb_256mpi_2omp.MPI_size/4)), 
                         y=(dfn4VectFusProb_256mpi_2omp.GFLOPS*(dfn4VectFusProb_256mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectFusProb_256mpi_2omp',
                         line=dict(color='royalblue',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))

fig.add_trace(go.Scatter(x=(dfn4VectFusNonProb_256mpi_2omp.Dirc_Data_Mov*(dfn4VectFusNonProb_256mpi_2omp.MPI_size/4)), 
                         y=(dfn4VectFusNonProb_256mpi_2omp.GFLOPS*(dfn4VectFusNonProb_256mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectFusNonProb_256mpi_2omp',
                         line=dict(color='sandybrown',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))


# Case 3
fig.add_trace(go.Scatter(x=(dfn4VectFusProb_128mpi_4omp.Dirc_Data_Mov*(dfn4VectFusProb_128mpi_4omp.MPI_size/4)), 
                         y=(dfn4VectFusProb_128mpi_4omp.GFLOPS*(dfn4VectFusProb_128mpi_4omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectFusProb_128mpi_4omp',
                         line=dict(color='darkblue',width=3),
                         marker=dict(symbol='circle',size=7, line=dict(color='tomato',width=2))))

fig.add_trace(go.Scatter(x=(dfn4VectFusNonProb_128mpi_4omp.Dirc_Data_Mov*(dfn4VectFusNonProb_128mpi_4omp.MPI_size/4)), 
                         y=(dfn4VectFusNonProb_128mpi_4omp.GFLOPS*(dfn4VectFusNonProb_128mpi_4omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectFusNonProb_128mpi_4omp',
                         line=dict(color='cyan',width=3),
                         marker=dict(symbol='circle',size=7,line=dict(color='Purple',width=2))))



# Case 4
fig.add_trace(go.Scatter(x=(dfn4VectFusProb_64mpi_8omp.Dirc_Data_Mov*(dfn4VectFusProb_64mpi_8omp.MPI_size/4)), 
                         y=(dfn4VectFusProb_64mpi_8omp.GFLOPS*(dfn4VectFusProb_64mpi_8omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectFusProb_64mpi_8omp',
                         line=dict(color='blue',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))

fig.add_trace(go.Scatter(x=(dfn4VectFusNonProb_64mpi_8omp.Dirc_Data_Mov*(dfn4VectFusNonProb_64mpi_8omp.MPI_size/4)), 
                         y=(dfn4VectFusNonProb_64mpi_8omp.GFLOPS*(dfn4VectFusNonProb_64mpi_8omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectFusNonProb_64mpi_8omp',
                         line=dict(color='red',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))


# Case 5
fig.add_trace(go.Scatter(x=(dfn4VectFusProb_32mpi_16omp.Dirc_Data_Mov*(dfn4VectFusProb_32mpi_16omp.MPI_size/4)), 
                         y=(dfn4VectFusProb_32mpi_16omp.GFLOPS*(dfn4VectFusProb_32mpi_16omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectFusProb_32mpi_16omp',
                         line=dict(color='chartreuse',width=3),
                         marker=dict(symbol='square-dot',size=6, line=dict(color='mediumvioletred',width=1))))

fig.add_trace(go.Scatter(x=(dfn4VectFusNonProb_32mpi_16omp.Dirc_Data_Mov*(dfn4VectFusNonProb_32mpi_16omp.MPI_size/4)), 
                         y=(dfn4VectFusNonProb_32mpi_16omp.GFLOPS*(dfn4VectFusNonProb_32mpi_16omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectFusNonProb_32mpi_16omp',
                         line=dict(color='black',width=3),
                         marker=dict(symbol='square-dot',size=6, line=dict(color='mediumvioletred',width=1))))


# Case 6
fig.add_trace(go.Scatter(x=(dfn4VectFusProb_56mpi_9omp.Dirc_Data_Mov*(dfn4VectFusProb_56mpi_9omp.MPI_size/4)), 
                         y=(dfn4VectFusProb_56mpi_9omp.GFLOPS*(dfn4VectFusProb_56mpi_9omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectFusProb_56mpi_9omp',
                         line=dict(color='gold',width=3),
                         marker=dict(symbol='triangle-up',size=6, line=dict(color='black',width=1))))

fig.add_trace(go.Scatter(x=(dfn4VectFusNonProb_56mpi_9omp.Dirc_Data_Mov*(dfn4VectFusNonProb_56mpi_9omp.MPI_size/4)), 
                         y=(dfn4VectFusNonProb_56mpi_9omp.GFLOPS*(dfn4VectFusNonProb_56mpi_9omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectFusNonProb_56mpi_9omp',
                         line=dict(color='dimgray',width=3),
                         marker=dict(symbol='triangle-up',size=6, line=dict(color='black',width=1))))



# Case 7
fig.add_trace(go.Scatter(x=(dfn4VectFusProb_248mpi_2omp.Dirc_Data_Mov*(dfn4VectFusProb_248mpi_2omp.MPI_size/4)), 
                         y=(dfn4VectFusProb_248mpi_2omp.GFLOPS*(dfn4VectFusProb_248mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectFusProb_248mpi_2omp',
                         line=dict(color='olive',width=3),
                         marker=dict(symbol='circle',size=6, line=dict(color='olive',width=2))))

fig.add_trace(go.Scatter(x=(dfn4VectFusNonProb_248mpi_2omp.Dirc_Data_Mov*(dfn4VectFusNonProb_248mpi_2omp.MPI_size/4)), 
                         y=(dfn4VectFusNonProb_248mpi_2omp.GFLOPS*(dfn4VectFusNonProb_248mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectFusNonProb_248mpi_2omp',
                         line=dict(color='Brown',width=3),
                         marker=dict(symbol='circle',size=6, line=dict(color='darkred',width=2))))




# Adding a vertical line
fig.add_vline(x=512e6, line_width=3, line_dash="dash", line_color="green")
# fig.add_hline(y=1024, line_width=3, line_dash="dash", line_color="red")

# Add annotation
fig.add_annotation(
    x='512e6'
    , y=130
    , text=f'LLC Size Per Node'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="purple", family="Sans Serif")
    , align="left"
    ,)

fig.update_layout(title='"Fused-VECT (4-Nodes): probempi vs nonprobempi" GFlops Measurement',
                  xaxis_title='Dirc_Data_Mov', yaxis_title='GFLOPs/Node', 
                  xaxis=dict(ticks="outside"),  
                  yaxis=dict(ticks="outside"))

## BANDWIDTH - Fused VECT: probempi vs nonprobempi

In [58]:
fig = go.Figure() 

# Case 1
fig.add_trace(go.Scatter(x=(dfn4VectFusProb_512mpi_1omp.Dirc_Data_Mov*(dfn4VectFusProb_512mpi_1omp.MPI_size/4)), 
                         y=(dfn4VectFusProb_512mpi_1omp.BAND*(dfn4VectFusProb_512mpi_1omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='dfn4VectFusProb_512mpi_1omp',
                         line=dict(color='darkcyan',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))

fig.add_trace(go.Scatter(x=(dfn4VectFusNonProb_512mpi_1omp.Dirc_Data_Mov*(dfn4VectFusNonProb_512mpi_1omp.MPI_size/4)), 
                         y=(dfn4VectFusNonProb_512mpi_1omp.BAND*(dfn4VectFusNonProb_512mpi_1omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectFusNonProb_512mpi_1omp',
                         line=dict(color='magenta',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))


# Case 2
fig.add_trace(go.Scatter(x=(dfn4VectFusProb_256mpi_2omp.Dirc_Data_Mov*(dfn4VectFusProb_256mpi_2omp.MPI_size/4)), 
                         y=(dfn4VectFusProb_256mpi_2omp.BAND*(dfn4VectFusProb_256mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectFusProb_256mpi_2omp',
                         line=dict(color='royalblue',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))

fig.add_trace(go.Scatter(x=(dfn4VectFusNonProb_256mpi_2omp.Dirc_Data_Mov*(dfn4VectFusNonProb_256mpi_2omp.MPI_size/4)), 
                         y=(dfn4VectFusNonProb_256mpi_2omp.BAND*(dfn4VectFusNonProb_256mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectFusNonProb_256mpi_2omp',
                         line=dict(color='sandybrown',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))


# Case 3
fig.add_trace(go.Scatter(x=(dfn4VectFusProb_128mpi_4omp.Dirc_Data_Mov*(dfn4VectFusProb_128mpi_4omp.MPI_size/4)), 
                         y=(dfn4VectFusProb_128mpi_4omp.BAND*(dfn4VectFusProb_128mpi_4omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectFusProb_128mpi_4omp',
                         line=dict(color='darkblue',width=3),
                         marker=dict(symbol='circle',size=7, line=dict(color='tomato',width=2))))

fig.add_trace(go.Scatter(x=(dfn4VectFusNonProb_128mpi_4omp.Dirc_Data_Mov*(dfn4VectFusNonProb_128mpi_4omp.MPI_size/4)), 
                         y=(dfn4VectFusNonProb_128mpi_4omp.BAND*(dfn4VectFusNonProb_128mpi_4omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectFusNonProb_128mpi_4omp',
                         line=dict(color='cyan',width=3),
                         marker=dict(symbol='circle',size=7,line=dict(color='Purple',width=2))))



# Case 4
fig.add_trace(go.Scatter(x=(dfn4VectFusProb_64mpi_8omp.Dirc_Data_Mov*(dfn4VectFusProb_64mpi_8omp.MPI_size/4)), 
                         y=(dfn4VectFusProb_64mpi_8omp.BAND*(dfn4VectFusProb_64mpi_8omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectFusProb_64mpi_8omp',
                         line=dict(color='blue',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))

fig.add_trace(go.Scatter(x=(dfn4VectFusNonProb_64mpi_8omp.Dirc_Data_Mov*(dfn4VectFusNonProb_64mpi_8omp.MPI_size/4)), 
                         y=(dfn4VectFusNonProb_64mpi_8omp.BAND*(dfn4VectFusNonProb_64mpi_8omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectFusNonProb_64mpi_8omp',
                         line=dict(color='red',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))


# Case 5
fig.add_trace(go.Scatter(x=(dfn4VectFusProb_32mpi_16omp.Dirc_Data_Mov*(dfn4VectFusProb_32mpi_16omp.MPI_size/4)), 
                         y=(dfn4VectFusProb_32mpi_16omp.BAND*(dfn4VectFusProb_32mpi_16omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectFusProb_32mpi_16omp',
                         line=dict(color='chartreuse',width=3),
                         marker=dict(symbol='square-dot',size=6, line=dict(color='mediumvioletred',width=1))))

fig.add_trace(go.Scatter(x=(dfn4VectFusNonProb_32mpi_16omp.Dirc_Data_Mov*(dfn4VectFusNonProb_32mpi_16omp.MPI_size/4)), 
                         y=(dfn4VectFusNonProb_32mpi_16omp.BAND*(dfn4VectFusNonProb_32mpi_16omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectFusNonProb_32mpi_16omp',
                         line=dict(color='black',width=3),
                         marker=dict(symbol='square-dot',size=6, line=dict(color='mediumvioletred',width=1))))


# Case 6
fig.add_trace(go.Scatter(x=(dfn4VectFusProb_56mpi_9omp.Dirc_Data_Mov*(dfn4VectFusProb_56mpi_9omp.MPI_size/4)), 
                         y=(dfn4VectFusProb_56mpi_9omp.BAND*(dfn4VectFusProb_56mpi_9omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectFusProb_56mpi_9omp',
                         line=dict(color='gold',width=3),
                         marker=dict(symbol='triangle-up',size=6, line=dict(color='black',width=1))))

fig.add_trace(go.Scatter(x=(dfn4VectFusNonProb_56mpi_9omp.Dirc_Data_Mov*(dfn4VectFusNonProb_56mpi_9omp.MPI_size/4)), 
                         y=(dfn4VectFusNonProb_56mpi_9omp.BAND*(dfn4VectFusNonProb_56mpi_9omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectFusNonProb_56mpi_9omp',
                         line=dict(color='dimgray',width=3),
                         marker=dict(symbol='triangle-up',size=6, line=dict(color='black',width=1))))



# Case 7
fig.add_trace(go.Scatter(x=(dfn4VectFusProb_248mpi_2omp.Dirc_Data_Mov*(dfn4VectFusProb_248mpi_2omp.MPI_size/4)), 
                         y=(dfn4VectFusProb_248mpi_2omp.BAND*(dfn4VectFusProb_248mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectFusProb_248mpi_2omp',
                         line=dict(color='olive',width=3),
                         marker=dict(symbol='circle',size=6, line=dict(color='olive',width=2))))

fig.add_trace(go.Scatter(x=(dfn4VectFusNonProb_248mpi_2omp.Dirc_Data_Mov*(dfn4VectFusNonProb_248mpi_2omp.MPI_size/4)), 
                         y=(dfn4VectFusNonProb_248mpi_2omp.BAND*(dfn4VectFusNonProb_248mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectFusNonProb_248mpi_2omp',
                         line=dict(color='Brown',width=3),
                         marker=dict(symbol='circle',size=6, line=dict(color='darkred',width=2))))




# Adding vertical and horizontal lines 
fig.add_vline(x=512e6, line_width=3, line_dash="dot", line_color="green")
fig.add_hline(y=409, line_width=3, line_dash="dash", line_color="red")

# Add annotation with arrow
fig.add_annotation(
    x='512e6'
    , y=280
    , text=f'LLC Size Per Node: 512M'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="purple", family="Sans Serif")
    , align="left"
    ,)

fig.add_annotation(
    x='130e6'
    , y=370
    , text=f'Max Bandwidth Per Node'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="orange", family="Sans Serif")
    , align="left"
    ,)


fig.update_layout(title='"Fused VECT: Probmpi vs Non-Probempi" Bandwidth Measurement',
                  xaxis_title='Dirc_Data_Mov', yaxis_title='Bandwidth/Node', 
                  xaxis=dict(ticks="outside"),  
                  yaxis=dict(ticks="outside"),
                  legend=go.layout.Legend(bordercolor="black",borderwidth=1)) 





## 3) NonFused_VECT_Probmpi

### Adding Columns

In [59]:
dfn4_vect_nonfused_probempi["N_Nodes"] = 4
dfn4_vect_nonfused_probempi['NPX'] = (dfn4_vect_nonfused_probempi.GLB_X//dfn4_vect_nonfused_probempi.LOC_X)
dfn4_vect_nonfused_probempi['NPY'] = (dfn4_vect_nonfused_probempi.GLB_Y//dfn4_vect_nonfused_probempi.LOC_Y)
dfn4_vect_nonfused_probempi['NPZ'] = (dfn4_vect_nonfused_probempi.GLB_Z//dfn4_vect_nonfused_probempi.LOC_Z)
dfn4_vect_nonfused_probempi['NPT'] = (dfn4_vect_nonfused_probempi.GLB_T//dfn4_vect_nonfused_probempi.LOC_T)
dfn4_vect_nonfused_probempi["Total_proc"] = dfn4_vect_nonfused_probempi["omp_thrd"]* dfn4_vect_nonfused_probempi.MPI_size
# dfn4_vect_nonfused_probempi.columns

### Data Summary

In [60]:
dfn4_vect_nonfused_probempi_summary = dfn4_vect_nonfused_probempi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn4_vect_nonfused_probempi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,32,16,4,2,2,2,512,111
1,56,9,7,2,2,2,504,39
2,64,8,8,2,2,2,512,70
3,72,7,9,2,2,2,504,31
4,128,4,16,2,2,2,512,33
5,168,3,21,2,2,2,504,14
6,248,2,31,2,2,2,496,8
7,256,2,32,2,2,2,512,17
8,488,1,61,2,2,2,488,4
9,496,1,62,2,2,2,496,7


### Cases Setup

In [61]:
dfn4VectNonFusProb_32mpi_16omp = dfn4_vect_nonfused_probempi[(dfn4_vect_nonfused_probempi.omp_thrd == 16) & (dfn4_vect_nonfused_probempi.GLB_T/dfn4_vect_nonfused_probempi.LOC_T == 4) & (dfn4_vect_nonfused_probempi.GLB_X/dfn4_vect_nonfused_probempi.LOC_X == 2) & (dfn4_vect_nonfused_probempi.GLB_Y/dfn4_vect_nonfused_probempi.LOC_Y == 2)& (dfn4_vect_nonfused_probempi.GLB_Z/dfn4_vect_nonfused_probempi.LOC_Z == 2)]
dfn4VectNonFusProb_64mpi_8omp = dfn4_vect_nonfused_probempi[(dfn4_vect_nonfused_probempi.omp_thrd == 8) & (dfn4_vect_nonfused_probempi.GLB_T/dfn4_vect_nonfused_probempi.LOC_T == 8) & (dfn4_vect_nonfused_probempi.GLB_X/dfn4_vect_nonfused_probempi.LOC_X == 2) & (dfn4_vect_nonfused_probempi.GLB_Y/dfn4_vect_nonfused_probempi.LOC_Y == 2)& (dfn4_vect_nonfused_probempi.GLB_Z/dfn4_vect_nonfused_probempi.LOC_Z == 2)]
dfn4VectNonFusProb_128mpi_4omp = dfn4_vect_nonfused_probempi[(dfn4_vect_nonfused_probempi.omp_thrd == 4) & (dfn4_vect_nonfused_probempi.GLB_T/dfn4_vect_nonfused_probempi.LOC_T == 16) & (dfn4_vect_nonfused_probempi.GLB_X/dfn4_vect_nonfused_probempi.LOC_X == 2) & (dfn4_vect_nonfused_probempi.GLB_Y/dfn4_vect_nonfused_probempi.LOC_Y == 2)& (dfn4_vect_nonfused_probempi.GLB_Z/dfn4_vect_nonfused_probempi.LOC_Z == 2)]
dfn4VectNonFusProb_256mpi_2omp = dfn4_vect_nonfused_probempi[(dfn4_vect_nonfused_probempi.omp_thrd == 2) & (dfn4_vect_nonfused_probempi.GLB_T/dfn4_vect_nonfused_probempi.LOC_T == 32) & (dfn4_vect_nonfused_probempi.GLB_X/dfn4_vect_nonfused_probempi.LOC_X == 2) & (dfn4_vect_nonfused_probempi.GLB_Y/dfn4_vect_nonfused_probempi.LOC_Y == 2)& (dfn4_vect_nonfused_probempi.GLB_Z/dfn4_vect_nonfused_probempi.LOC_Z == 2)]
dfn4VectNonFusProb_512mpi_1omp = dfn4_vect_nonfused_probempi[(dfn4_vect_nonfused_probempi.omp_thrd == 1) & (dfn4_vect_nonfused_probempi.GLB_T/dfn4_vect_nonfused_probempi.LOC_T == 64) & (dfn4_vect_nonfused_probempi.GLB_X/dfn4_vect_nonfused_probempi.LOC_X == 2) & (dfn4_vect_nonfused_probempi.GLB_Y/dfn4_vect_nonfused_probempi.LOC_Y == 2)& (dfn4_vect_nonfused_probempi.GLB_Z/dfn4_vect_nonfused_probempi.LOC_Z == 2)]
dfn4VectNonFusProb_56mpi_9omp = dfn4_vect_nonfused_probempi[(dfn4_vect_nonfused_probempi.omp_thrd == 9) & (dfn4_vect_nonfused_probempi.GLB_T/dfn4_vect_nonfused_probempi.LOC_T == 7) & (dfn4_vect_nonfused_probempi.GLB_X/dfn4_vect_nonfused_probempi.LOC_X == 2) & (dfn4_vect_nonfused_probempi.GLB_Y/dfn4_vect_nonfused_probempi.LOC_Y == 2)& (dfn4_vect_nonfused_probempi.GLB_Z/dfn4_vect_nonfused_probempi.LOC_Z == 2)]
dfn4VectNonFusProb_248mpi_2omp = dfn4_vect_nonfused_probempi[(dfn4_vect_nonfused_probempi.omp_thrd == 2) & (dfn4_vect_nonfused_probempi.GLB_T/dfn4_vect_nonfused_probempi.LOC_T == 31) & (dfn4_vect_nonfused_probempi.GLB_X/dfn4_vect_nonfused_probempi.LOC_X == 2) & (dfn4_vect_nonfused_probempi.GLB_Y/dfn4_vect_nonfused_probempi.LOC_Y == 2)& (dfn4_vect_nonfused_probempi.GLB_Z/dfn4_vect_nonfused_probempi.LOC_Z == 2)]


## 4) NonFused_VECT_NonProbmpi

### Adding Columns

In [62]:
dfn4_vect_nonfused_nonprobempi["N_Nodes"] = 4
dfn4_vect_nonfused_nonprobempi['NPX'] = (dfn4_vect_nonfused_nonprobempi.GLB_X//dfn4_vect_nonfused_nonprobempi.LOC_X)
dfn4_vect_nonfused_nonprobempi['NPY'] = (dfn4_vect_nonfused_nonprobempi.GLB_Y//dfn4_vect_nonfused_nonprobempi.LOC_Y)
dfn4_vect_nonfused_nonprobempi['NPZ'] = (dfn4_vect_nonfused_nonprobempi.GLB_Z//dfn4_vect_nonfused_nonprobempi.LOC_Z)
dfn4_vect_nonfused_nonprobempi['NPT'] = (dfn4_vect_nonfused_nonprobempi.GLB_T//dfn4_vect_nonfused_nonprobempi.LOC_T)
dfn4_vect_nonfused_nonprobempi["Total_proc"] = dfn4_vect_nonfused_nonprobempi["omp_thrd"]* dfn4_vect_nonfused_nonprobempi.MPI_size
# dfn4_vect_nonfused_nonprobempi.columns

### Data Summary

In [63]:
dfn4_vect_nonfused_nonprobempi_summary = dfn4_vect_nonfused_nonprobempi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn4_vect_nonfused_nonprobempi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,32,16,4,2,2,2,512,109
1,56,9,7,2,2,2,504,40
2,64,8,8,2,2,2,512,70
3,72,7,9,2,2,2,504,31
4,128,4,16,2,2,2,512,35
5,168,3,21,2,2,2,504,14
6,248,2,31,2,2,2,496,9
7,256,2,32,2,2,2,512,16
8,488,1,61,2,2,2,488,4
9,496,1,62,2,2,2,496,7


### Cases Setup

In [64]:
dfn4VectNonFusNonProb_32mpi_16omp = dfn4_vect_nonfused_nonprobempi[(dfn4_vect_nonfused_nonprobempi.omp_thrd == 16) & (dfn4_vect_nonfused_nonprobempi.GLB_T/dfn4_vect_nonfused_nonprobempi.LOC_T == 4) & (dfn4_vect_nonfused_nonprobempi.GLB_X/dfn4_vect_nonfused_nonprobempi.LOC_X == 2) & (dfn4_vect_nonfused_nonprobempi.GLB_Y/dfn4_vect_nonfused_nonprobempi.LOC_Y == 2)& (dfn4_vect_nonfused_nonprobempi.GLB_Z/dfn4_vect_nonfused_nonprobempi.LOC_Z == 2)]
dfn4VectNonFusNonProb_64mpi_8omp = dfn4_vect_nonfused_nonprobempi[(dfn4_vect_nonfused_nonprobempi.omp_thrd == 8) & (dfn4_vect_nonfused_nonprobempi.GLB_T/dfn4_vect_nonfused_nonprobempi.LOC_T == 8) & (dfn4_vect_nonfused_nonprobempi.GLB_X/dfn4_vect_nonfused_nonprobempi.LOC_X == 2) & (dfn4_vect_nonfused_nonprobempi.GLB_Y/dfn4_vect_nonfused_nonprobempi.LOC_Y == 2)& (dfn4_vect_nonfused_nonprobempi.GLB_Z/dfn4_vect_nonfused_nonprobempi.LOC_Z == 2)]
dfn4VectNonFusNonProb_128mpi_4omp = dfn4_vect_nonfused_nonprobempi[(dfn4_vect_nonfused_nonprobempi.omp_thrd == 4) & (dfn4_vect_nonfused_nonprobempi.GLB_T/dfn4_vect_nonfused_nonprobempi.LOC_T == 16) & (dfn4_vect_nonfused_nonprobempi.GLB_X/dfn4_vect_nonfused_nonprobempi.LOC_X == 2) & (dfn4_vect_nonfused_nonprobempi.GLB_Y/dfn4_vect_nonfused_nonprobempi.LOC_Y == 2)& (dfn4_vect_nonfused_nonprobempi.GLB_Z/dfn4_vect_nonfused_nonprobempi.LOC_Z == 2)]
dfn4VectNonFusNonProb_256mpi_2omp = dfn4_vect_nonfused_nonprobempi[(dfn4_vect_nonfused_nonprobempi.omp_thrd == 2) & (dfn4_vect_nonfused_nonprobempi.GLB_T/dfn4_vect_nonfused_nonprobempi.LOC_T == 32) & (dfn4_vect_nonfused_nonprobempi.GLB_X/dfn4_vect_nonfused_nonprobempi.LOC_X == 2) & (dfn4_vect_nonfused_nonprobempi.GLB_Y/dfn4_vect_nonfused_nonprobempi.LOC_Y == 2)& (dfn4_vect_nonfused_nonprobempi.GLB_Z/dfn4_vect_nonfused_nonprobempi.LOC_Z == 2)]
dfn4VectNonFusNonProb_512mpi_1omp = dfn4_vect_nonfused_nonprobempi[(dfn4_vect_nonfused_nonprobempi.omp_thrd == 1) & (dfn4_vect_nonfused_nonprobempi.GLB_T/dfn4_vect_nonfused_nonprobempi.LOC_T == 64) & (dfn4_vect_nonfused_nonprobempi.GLB_X/dfn4_vect_nonfused_nonprobempi.LOC_X == 2) & (dfn4_vect_nonfused_nonprobempi.GLB_Y/dfn4_vect_nonfused_nonprobempi.LOC_Y == 2)& (dfn4_vect_nonfused_nonprobempi.GLB_Z/dfn4_vect_nonfused_nonprobempi.LOC_Z == 2)]
dfn4VectNonFusNonProb_56mpi_9omp = dfn4_vect_nonfused_nonprobempi[(dfn4_vect_nonfused_nonprobempi.omp_thrd == 9) & (dfn4_vect_nonfused_nonprobempi.GLB_T/dfn4_vect_nonfused_nonprobempi.LOC_T == 7) & (dfn4_vect_nonfused_nonprobempi.GLB_X/dfn4_vect_nonfused_nonprobempi.LOC_X == 2) & (dfn4_vect_nonfused_nonprobempi.GLB_Y/dfn4_vect_nonfused_nonprobempi.LOC_Y == 2)& (dfn4_vect_nonfused_nonprobempi.GLB_Z/dfn4_vect_nonfused_nonprobempi.LOC_Z == 2)]
dfn4VectNonFusNonProb_248mpi_2omp = dfn4_vect_nonfused_nonprobempi[(dfn4_vect_nonfused_nonprobempi.omp_thrd == 2) & (dfn4_vect_nonfused_nonprobempi.GLB_T/dfn4_vect_nonfused_nonprobempi.LOC_T == 31) & (dfn4_vect_nonfused_nonprobempi.GLB_X/dfn4_vect_nonfused_nonprobempi.LOC_X == 2) & (dfn4_vect_nonfused_nonprobempi.GLB_Y/dfn4_vect_nonfused_nonprobempi.LOC_Y == 2)& (dfn4_vect_nonfused_nonprobempi.GLB_Z/dfn4_vect_nonfused_nonprobempi.LOC_Z == 2)]


## GFLOPS - NonFused-VECT: Probempi vs NonProbempi

In [65]:
fig = go.Figure() 

# Case 1
fig.add_trace(go.Scatter(x=(dfn4VectNonFusProb_512mpi_1omp.Dirc_Data_Mov*(dfn4VectNonFusProb_512mpi_1omp.MPI_size/4)), 
                         y=(dfn4VectNonFusProb_512mpi_1omp.GFLOPS*(dfn4VectNonFusProb_512mpi_1omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectNonFusProb_512mpi_1omp',
                         line=dict(color='darkcyan',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))

fig.add_trace(go.Scatter(x=(dfn4VectNonFusNonProb_512mpi_1omp.Dirc_Data_Mov*(dfn4VectNonFusNonProb_512mpi_1omp.MPI_size/4)), 
                         y=(dfn4VectNonFusNonProb_512mpi_1omp.GFLOPS*(dfn4VectNonFusNonProb_512mpi_1omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectNonFusNonProb_512mpi_1omp',
                         line=dict(color='magenta',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))


# Case 2
fig.add_trace(go.Scatter(x=(dfn4VectNonFusProb_256mpi_2omp.Dirc_Data_Mov*(dfn4VectNonFusProb_256mpi_2omp.MPI_size/4)), 
                         y=(dfn4VectNonFusProb_256mpi_2omp.GFLOPS*(dfn4VectNonFusProb_256mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectNonFusProb_256mpi_2omp',
                         line=dict(color='royalblue',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))

fig.add_trace(go.Scatter(x=(dfn4VectNonFusNonProb_256mpi_2omp.Dirc_Data_Mov*(dfn4VectNonFusNonProb_256mpi_2omp.MPI_size/4)), 
                         y=(dfn4VectNonFusNonProb_256mpi_2omp.GFLOPS*(dfn4VectNonFusNonProb_256mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectNonFusNonProb_256mpi_2omp',
                         line=dict(color='sandybrown',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))


# Case 3
fig.add_trace(go.Scatter(x=(dfn4VectNonFusProb_128mpi_4omp.Dirc_Data_Mov*(dfn4VectNonFusProb_128mpi_4omp.MPI_size/4)), 
                         y=(dfn4VectNonFusProb_128mpi_4omp.GFLOPS*(dfn4VectNonFusProb_128mpi_4omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectNonFusProb_128mpi_4omp',
                         line=dict(color='darkblue',width=3),
                         marker=dict(symbol='circle',size=7, line=dict(color='tomato',width=2))))

fig.add_trace(go.Scatter(x=(dfn4VectNonFusNonProb_128mpi_4omp.Dirc_Data_Mov*(dfn4VectNonFusNonProb_128mpi_4omp.MPI_size/4)), 
                         y=(dfn4VectNonFusNonProb_128mpi_4omp.GFLOPS*(dfn4VectNonFusNonProb_128mpi_4omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectNonFusNonProb_128mpi_4omp',
                         line=dict(color='cyan',width=3),
                         marker=dict(symbol='circle',size=7,line=dict(color='Purple',width=2))))



# Case 4
fig.add_trace(go.Scatter(x=(dfn4VectNonFusProb_64mpi_8omp.Dirc_Data_Mov*(dfn4VectNonFusProb_64mpi_8omp.MPI_size/4)), 
                         y=(dfn4VectNonFusProb_64mpi_8omp.GFLOPS*(dfn4VectNonFusProb_64mpi_8omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectNonFusProb_64mpi_8omp',
                         line=dict(color='blue',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))

fig.add_trace(go.Scatter(x=(dfn4VectNonFusNonProb_64mpi_8omp.Dirc_Data_Mov*(dfn4VectNonFusNonProb_64mpi_8omp.MPI_size/4)), 
                         y=(dfn4VectNonFusNonProb_64mpi_8omp.GFLOPS*(dfn4VectNonFusNonProb_64mpi_8omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectNonFusNonProb_64mpi_8omp',
                         line=dict(color='red',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))


# Case 5
fig.add_trace(go.Scatter(x=(dfn4VectNonFusProb_32mpi_16omp.Dirc_Data_Mov*(dfn4VectNonFusProb_32mpi_16omp.MPI_size/4)), 
                         y=(dfn4VectNonFusProb_32mpi_16omp.GFLOPS*(dfn4VectNonFusProb_32mpi_16omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectNonFusProb_32mpi_16omp',
                         line=dict(color='chartreuse',width=3),
                         marker=dict(symbol='square-dot',size=6, line=dict(color='mediumvioletred',width=1))))

fig.add_trace(go.Scatter(x=(dfn4VectNonFusNonProb_32mpi_16omp.Dirc_Data_Mov*(dfn4VectNonFusNonProb_32mpi_16omp.MPI_size/4)), 
                         y=(dfn4VectNonFusNonProb_32mpi_16omp.GFLOPS*(dfn4VectNonFusNonProb_32mpi_16omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectNonFusNonProb_32mpi_16omp',
                         line=dict(color='black',width=3),
                         marker=dict(symbol='square-dot',size=6, line=dict(color='mediumvioletred',width=1))))


# Case 6
fig.add_trace(go.Scatter(x=(dfn4VectNonFusProb_56mpi_9omp.Dirc_Data_Mov*(dfn4VectNonFusProb_56mpi_9omp.MPI_size/4)), 
                         y=(dfn4VectNonFusProb_56mpi_9omp.GFLOPS*(dfn4VectNonFusProb_56mpi_9omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectNonFusProb_56mpi_9omp',
                         line=dict(color='gold',width=3),
                         marker=dict(symbol='triangle-up',size=6, line=dict(color='black',width=1))))

fig.add_trace(go.Scatter(x=(dfn4VectNonFusNonProb_56mpi_9omp.Dirc_Data_Mov*(dfn4VectNonFusNonProb_56mpi_9omp.MPI_size/4)), 
                         y=(dfn4VectNonFusNonProb_56mpi_9omp.GFLOPS*(dfn4VectNonFusNonProb_56mpi_9omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectNonFusNonProb_56mpi_9omp',
                         line=dict(color='dimgray',width=3),
                         marker=dict(symbol='triangle-up',size=6, line=dict(color='black',width=1))))



# Case 7
fig.add_trace(go.Scatter(x=(dfn4VectNonFusProb_248mpi_2omp.Dirc_Data_Mov*(dfn4VectNonFusProb_248mpi_2omp.MPI_size/4)), 
                         y=(dfn4VectNonFusProb_248mpi_2omp.GFLOPS*(dfn4VectNonFusProb_248mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectNonFusProb_248mpi_2omp',
                         line=dict(color='olive',width=3),
                         marker=dict(symbol='circle',size=6, line=dict(color='olive',width=2))))

fig.add_trace(go.Scatter(x=(dfn4VectNonFusNonProb_248mpi_2omp.Dirc_Data_Mov*(dfn4VectNonFusNonProb_248mpi_2omp.MPI_size/4)), 
                         y=(dfn4VectNonFusNonProb_248mpi_2omp.GFLOPS*(dfn4VectNonFusNonProb_248mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectNonFusNonProb_248mpi_2omp',
                         line=dict(color='Brown',width=3),
                         marker=dict(symbol='circle',size=6, line=dict(color='darkred',width=2))))




# Adding a vertical line
fig.add_vline(x=512e6, line_width=3, line_dash="dash", line_color="green")
# fig.add_hline(y=1024, line_width=3, line_dash="dash", line_color="red")

# Add annotation
fig.add_annotation(
    x='512e6'
    , y=130
    , text=f'LLC Size Per Node'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="purple", family="Sans Serif")
    , align="left"
    ,)

fig.update_layout(title='"NonFused-VECT (4-Nodes): probempi vs nonprobempi" GFlops Measurement',
                  xaxis_title='Dirc_Data_Mov', yaxis_title='GFLOPs/Node', 
                  xaxis=dict(ticks="outside"),  
                  yaxis=dict(ticks="outside"))

## BANDWIDTH - NonFused-VECT: Probempi vs NonProbempi

In [66]:
fig = go.Figure() 

# Case 1
fig.add_trace(go.Scatter(x=(dfn4VectNonFusProb_512mpi_1omp.Dirc_Data_Mov*(dfn4VectNonFusProb_512mpi_1omp.MPI_size/4)), 
                         y=(dfn4VectNonFusProb_512mpi_1omp.BAND*(dfn4VectNonFusProb_512mpi_1omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectNonFusProb_512mpi_1omp',
                         line=dict(color='darkcyan',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))

fig.add_trace(go.Scatter(x=(dfn4VectNonFusNonProb_512mpi_1omp.Dirc_Data_Mov*(dfn4VectNonFusNonProb_512mpi_1omp.MPI_size/4)), 
                         y=(dfn4VectNonFusNonProb_512mpi_1omp.BAND*(dfn4VectNonFusNonProb_512mpi_1omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectNonFusNonProb_512mpi_1omp',
                         line=dict(color='magenta',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))


# Case 2
fig.add_trace(go.Scatter(x=(dfn4VectNonFusProb_256mpi_2omp.Dirc_Data_Mov*(dfn4VectNonFusProb_256mpi_2omp.MPI_size/4)), 
                         y=(dfn4VectNonFusProb_256mpi_2omp.BAND*(dfn4VectNonFusProb_256mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectNonFusProb_256mpi_2omp',
                         line=dict(color='royalblue',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))

fig.add_trace(go.Scatter(x=(dfn4VectNonFusNonProb_256mpi_2omp.Dirc_Data_Mov*(dfn4VectNonFusNonProb_256mpi_2omp.MPI_size/4)), 
                         y=(dfn4VectNonFusNonProb_256mpi_2omp.BAND*(dfn4VectNonFusNonProb_256mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectNonFusNonProb_256mpi_2omp',
                         line=dict(color='sandybrown',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))


# Case 3
fig.add_trace(go.Scatter(x=(dfn4VectNonFusProb_128mpi_4omp.Dirc_Data_Mov*(dfn4VectNonFusProb_128mpi_4omp.MPI_size/4)), 
                         y=(dfn4VectNonFusProb_128mpi_4omp.BAND*(dfn4VectNonFusProb_128mpi_4omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectNonFusProb_128mpi_4omp',
                         line=dict(color='darkblue',width=3),
                         marker=dict(symbol='circle',size=7, line=dict(color='tomato',width=2))))

fig.add_trace(go.Scatter(x=(dfn4VectNonFusNonProb_128mpi_4omp.Dirc_Data_Mov*(dfn4VectNonFusNonProb_128mpi_4omp.MPI_size/4)), 
                         y=(dfn4VectNonFusNonProb_128mpi_4omp.BAND*(dfn4VectNonFusNonProb_128mpi_4omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectNonFusNonProb_128mpi_4omp',
                         line=dict(color='cyan',width=3),
                         marker=dict(symbol='circle',size=7,line=dict(color='Purple',width=2))))



# Case 4
fig.add_trace(go.Scatter(x=(dfn4VectNonFusProb_64mpi_8omp.Dirc_Data_Mov*(dfn4VectNonFusProb_64mpi_8omp.MPI_size/4)), 
                         y=(dfn4VectNonFusProb_64mpi_8omp.BAND*(dfn4VectNonFusProb_64mpi_8omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectNonFusProb_64mpi_8omp',
                         line=dict(color='blue',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))

fig.add_trace(go.Scatter(x=(dfn4VectNonFusNonProb_64mpi_8omp.Dirc_Data_Mov*(dfn4VectNonFusNonProb_64mpi_8omp.MPI_size/4)), 
                         y=(dfn4VectNonFusNonProb_64mpi_8omp.BAND*(dfn4VectNonFusNonProb_64mpi_8omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectNonFusNonProb_64mpi_8omp',
                         line=dict(color='red',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))


# Case 5
fig.add_trace(go.Scatter(x=(dfn4VectNonFusProb_32mpi_16omp.Dirc_Data_Mov*(dfn4VectNonFusProb_32mpi_16omp.MPI_size/4)), 
                         y=(dfn4VectNonFusProb_32mpi_16omp.BAND*(dfn4VectNonFusProb_32mpi_16omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectNonFusProb_32mpi_16omp',
                         line=dict(color='chartreuse',width=3),
                         marker=dict(symbol='square-dot',size=6, line=dict(color='mediumvioletred',width=1))))

fig.add_trace(go.Scatter(x=(dfn4VectNonFusNonProb_32mpi_16omp.Dirc_Data_Mov*(dfn4VectNonFusNonProb_32mpi_16omp.MPI_size/4)), 
                         y=(dfn4VectNonFusNonProb_32mpi_16omp.BAND*(dfn4VectNonFusNonProb_32mpi_16omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectNonFusNonProb_32mpi_16omp',
                         line=dict(color='black',width=3),
                         marker=dict(symbol='square-dot',size=6, line=dict(color='mediumvioletred',width=1))))


# Case 6
fig.add_trace(go.Scatter(x=(dfn4VectNonFusProb_56mpi_9omp.Dirc_Data_Mov*(dfn4VectNonFusProb_56mpi_9omp.MPI_size/4)), 
                         y=(dfn4VectNonFusProb_56mpi_9omp.BAND*(dfn4VectNonFusProb_56mpi_9omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectNonFusProb_56mpi_9omp',
                         line=dict(color='gold',width=3),
                         marker=dict(symbol='triangle-up',size=6, line=dict(color='black',width=1))))

fig.add_trace(go.Scatter(x=(dfn4VectNonFusNonProb_56mpi_9omp.Dirc_Data_Mov*(dfn4VectNonFusNonProb_56mpi_9omp.MPI_size/4)), 
                         y=(dfn4VectNonFusNonProb_56mpi_9omp.BAND*(dfn4VectNonFusNonProb_56mpi_9omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectNonFusNonProb_56mpi_9omp',
                         line=dict(color='dimgray',width=3),
                         marker=dict(symbol='triangle-up',size=6, line=dict(color='black',width=1))))



# Case 7
fig.add_trace(go.Scatter(x=(dfn4VectNonFusProb_248mpi_2omp.Dirc_Data_Mov*(dfn4VectNonFusProb_248mpi_2omp.MPI_size/4)), 
                         y=(dfn4VectNonFusProb_248mpi_2omp.BAND*(dfn4VectNonFusProb_248mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectNonFusProb_248mpi_2omp',
                         line=dict(color='olive',width=3),
                         marker=dict(symbol='circle',size=6, line=dict(color='olive',width=2))))

fig.add_trace(go.Scatter(x=(dfn4VectNonFusNonProb_248mpi_2omp.Dirc_Data_Mov*(dfn4VectNonFusNonProb_248mpi_2omp.MPI_size/4)), 
                         y=(dfn4VectNonFusNonProb_248mpi_2omp.BAND*(dfn4VectNonFusNonProb_248mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectNonFusNonProb_248mpi_2omp',
                         line=dict(color='Brown',width=3),
                         marker=dict(symbol='circle',size=6, line=dict(color='darkred',width=2))))



# Adding vertical and horizontal lines 
fig.add_vline(x=512e6, line_width=3, line_dash="dot", line_color="green")
fig.add_hline(y=409, line_width=3, line_dash="dash", line_color="red")

# Add annotation with arrow
fig.add_annotation(
    x='512e6'
    , y=280
    , text=f'LLC Size Per Node: 512M'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="purple", family="Sans Serif")
    , align="left"
    ,)

fig.add_annotation(
    x='130e6'
    , y=370
    , text=f'Max Bandwidth Per Node'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="orange", family="Sans Serif")
    , align="left"
    ,)


fig.update_layout(title='"NonFused-VECT: Probmpi vs Non-Probempi" Bandwidth Measurement',
                  xaxis_title='Dirc_Data_Mov', yaxis_title='Bandwidth/Node', 
                  xaxis=dict(ticks="outside"),  
                  yaxis=dict(ticks="outside"),
                  legend=go.layout.Legend(bordercolor="black",borderwidth=1)) 





## 5) Newgeo_VECT_Probmpi

### Adding Columns

In [67]:
dfn4_vect_newgeo_probempi["N_Nodes"] = 4
dfn4_vect_newgeo_probempi['NPX'] = (dfn4_vect_newgeo_probempi.GLB_X//dfn4_vect_newgeo_probempi.LOC_X)
dfn4_vect_newgeo_probempi['NPY'] = (dfn4_vect_newgeo_probempi.GLB_Y//dfn4_vect_newgeo_probempi.LOC_Y)
dfn4_vect_newgeo_probempi['NPZ'] = (dfn4_vect_newgeo_probempi.GLB_Z//dfn4_vect_newgeo_probempi.LOC_Z)
dfn4_vect_newgeo_probempi['NPT'] = (dfn4_vect_newgeo_probempi.GLB_T//dfn4_vect_newgeo_probempi.LOC_T)
dfn4_vect_newgeo_probempi["Total_proc"] = dfn4_vect_newgeo_probempi["omp_thrd"]* dfn4_vect_newgeo_probempi.MPI_size
# dfn4_vect_newgeo_probempi.columns

### Data Summary

In [68]:
dfn4_vect_newgeo_probempi_summary = dfn4_vect_newgeo_probempi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn4_vect_newgeo_probempi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,32,16,4,2,2,2,512,139
1,56,9,7,2,2,2,504,40
2,64,8,8,2,2,2,512,70
3,72,7,9,2,2,2,504,31
4,128,4,16,2,2,2,512,34
5,168,3,21,2,2,2,504,14
6,248,2,31,2,2,2,496,9
7,256,2,32,2,2,2,512,16
8,488,1,61,2,2,2,488,4
9,496,1,62,2,2,2,496,6


### Cases Setup

In [69]:
dfn4VectNewgeoProb_32mpi_16omp = dfn4_vect_newgeo_probempi[(dfn4_vect_newgeo_probempi.omp_thrd == 16) & (dfn4_vect_newgeo_probempi.GLB_T/dfn4_vect_newgeo_probempi.LOC_T == 4) & (dfn4_vect_newgeo_probempi.GLB_X/dfn4_vect_newgeo_probempi.LOC_X == 2) & (dfn4_vect_newgeo_probempi.GLB_Y/dfn4_vect_newgeo_probempi.LOC_Y == 2)& (dfn4_vect_newgeo_probempi.GLB_Z/dfn4_vect_newgeo_probempi.LOC_Z == 2)]
dfn4VectNewgeoProb_64mpi_8omp = dfn4_vect_newgeo_probempi[(dfn4_vect_newgeo_probempi.omp_thrd == 8) & (dfn4_vect_newgeo_probempi.GLB_T/dfn4_vect_newgeo_probempi.LOC_T == 8) & (dfn4_vect_newgeo_probempi.GLB_X/dfn4_vect_newgeo_probempi.LOC_X == 2) & (dfn4_vect_newgeo_probempi.GLB_Y/dfn4_vect_newgeo_probempi.LOC_Y == 2)& (dfn4_vect_newgeo_probempi.GLB_Z/dfn4_vect_newgeo_probempi.LOC_Z == 2)]
dfn4VectNewgeoProb_128mpi_4omp = dfn4_vect_newgeo_probempi[(dfn4_vect_newgeo_probempi.omp_thrd == 4) & (dfn4_vect_newgeo_probempi.GLB_T/dfn4_vect_newgeo_probempi.LOC_T == 16) & (dfn4_vect_newgeo_probempi.GLB_X/dfn4_vect_newgeo_probempi.LOC_X == 2) & (dfn4_vect_newgeo_probempi.GLB_Y/dfn4_vect_newgeo_probempi.LOC_Y == 2)& (dfn4_vect_newgeo_probempi.GLB_Z/dfn4_vect_newgeo_probempi.LOC_Z == 2)]
dfn4VectNewgeoProb_256mpi_2omp = dfn4_vect_newgeo_probempi[(dfn4_vect_newgeo_probempi.omp_thrd == 2) & (dfn4_vect_newgeo_probempi.GLB_T/dfn4_vect_newgeo_probempi.LOC_T == 32) & (dfn4_vect_newgeo_probempi.GLB_X/dfn4_vect_newgeo_probempi.LOC_X == 2) & (dfn4_vect_newgeo_probempi.GLB_Y/dfn4_vect_newgeo_probempi.LOC_Y == 2)& (dfn4_vect_newgeo_probempi.GLB_Z/dfn4_vect_newgeo_probempi.LOC_Z == 2)]
dfn4VectNewgeoProb_512mpi_1omp = dfn4_vect_newgeo_probempi[(dfn4_vect_newgeo_probempi.omp_thrd == 1) & (dfn4_vect_newgeo_probempi.GLB_T/dfn4_vect_newgeo_probempi.LOC_T == 64) & (dfn4_vect_newgeo_probempi.GLB_X/dfn4_vect_newgeo_probempi.LOC_X == 2) & (dfn4_vect_newgeo_probempi.GLB_Y/dfn4_vect_newgeo_probempi.LOC_Y == 2)& (dfn4_vect_newgeo_probempi.GLB_Z/dfn4_vect_newgeo_probempi.LOC_Z == 2)]
dfn4VectNewgeoProb_56mpi_9omp = dfn4_vect_newgeo_probempi[(dfn4_vect_newgeo_probempi.omp_thrd == 9) & (dfn4_vect_newgeo_probempi.GLB_T/dfn4_vect_newgeo_probempi.LOC_T == 7) & (dfn4_vect_newgeo_probempi.GLB_X/dfn4_vect_newgeo_probempi.LOC_X == 2) & (dfn4_vect_newgeo_probempi.GLB_Y/dfn4_vect_newgeo_probempi.LOC_Y == 2)& (dfn4_vect_newgeo_probempi.GLB_Z/dfn4_vect_newgeo_probempi.LOC_Z == 2)]
dfn4VectNewgeoProb_248mpi_2omp = dfn4_vect_newgeo_probempi[(dfn4_vect_newgeo_probempi.omp_thrd == 2) & (dfn4_vect_newgeo_probempi.GLB_T/dfn4_vect_newgeo_probempi.LOC_T == 31) & (dfn4_vect_newgeo_probempi.GLB_X/dfn4_vect_newgeo_probempi.LOC_X == 2) & (dfn4_vect_newgeo_probempi.GLB_Y/dfn4_vect_newgeo_probempi.LOC_Y == 2)& (dfn4_vect_newgeo_probempi.GLB_Z/dfn4_vect_newgeo_probempi.LOC_Z == 2)]


## 6) Newgeo_VECT_NonProbmpi

### Adding Columns

In [70]:
dfn4_vect_newgeo_nonprobempi["N_Nodes"] = 4
dfn4_vect_newgeo_nonprobempi['NPX'] = (dfn4_vect_newgeo_nonprobempi.GLB_X//dfn4_vect_newgeo_nonprobempi.LOC_X)
dfn4_vect_newgeo_nonprobempi['NPY'] = (dfn4_vect_newgeo_nonprobempi.GLB_Y//dfn4_vect_newgeo_nonprobempi.LOC_Y)
dfn4_vect_newgeo_nonprobempi['NPZ'] = (dfn4_vect_newgeo_nonprobempi.GLB_Z//dfn4_vect_newgeo_nonprobempi.LOC_Z)
dfn4_vect_newgeo_nonprobempi['NPT'] = (dfn4_vect_newgeo_nonprobempi.GLB_T//dfn4_vect_newgeo_nonprobempi.LOC_T)
dfn4_vect_newgeo_nonprobempi["Total_proc"] = dfn4_vect_newgeo_nonprobempi["omp_thrd"]* dfn4_vect_newgeo_nonprobempi.MPI_size
# dfn4_vect_newgeo_nonprobempi.columns

### Data Summary

In [71]:
dfn4_vect_newgeo_nonprobempi_summary = dfn4_vect_newgeo_nonprobempi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn4_vect_newgeo_nonprobempi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,32,16,4,2,2,2,512,137
1,56,9,7,2,2,2,504,40
2,64,8,8,2,2,2,512,69
3,72,7,9,2,2,2,504,31
4,128,4,16,2,2,2,512,34
5,168,3,21,2,2,2,504,14
6,248,2,31,2,2,2,496,9
7,256,2,32,2,2,2,512,16
8,488,1,61,2,2,2,488,4
9,496,1,62,2,2,2,496,7


### Cases Setup

In [72]:
dfn4VectNewgeoNonProb_32mpi_16omp = dfn4_vect_newgeo_nonprobempi[(dfn4_vect_newgeo_nonprobempi.omp_thrd == 16) & (dfn4_vect_newgeo_nonprobempi.GLB_T/dfn4_vect_newgeo_nonprobempi.LOC_T == 4) & (dfn4_vect_newgeo_nonprobempi.GLB_X/dfn4_vect_newgeo_nonprobempi.LOC_X == 2) & (dfn4_vect_newgeo_nonprobempi.GLB_Y/dfn4_vect_newgeo_nonprobempi.LOC_Y == 2)& (dfn4_vect_newgeo_nonprobempi.GLB_Z/dfn4_vect_newgeo_nonprobempi.LOC_Z == 2)]
dfn4VectNewgeoNonProb_64mpi_8omp = dfn4_vect_newgeo_nonprobempi[(dfn4_vect_newgeo_nonprobempi.omp_thrd == 8) & (dfn4_vect_newgeo_nonprobempi.GLB_T/dfn4_vect_newgeo_nonprobempi.LOC_T == 8) & (dfn4_vect_newgeo_nonprobempi.GLB_X/dfn4_vect_newgeo_nonprobempi.LOC_X == 2) & (dfn4_vect_newgeo_nonprobempi.GLB_Y/dfn4_vect_newgeo_nonprobempi.LOC_Y == 2)& (dfn4_vect_newgeo_nonprobempi.GLB_Z/dfn4_vect_newgeo_nonprobempi.LOC_Z == 2)]
dfn4VectNewgeoNonProb_128mpi_4omp = dfn4_vect_newgeo_nonprobempi[(dfn4_vect_newgeo_nonprobempi.omp_thrd == 4) & (dfn4_vect_newgeo_nonprobempi.GLB_T/dfn4_vect_newgeo_nonprobempi.LOC_T == 16) & (dfn4_vect_newgeo_nonprobempi.GLB_X/dfn4_vect_newgeo_nonprobempi.LOC_X == 2) & (dfn4_vect_newgeo_nonprobempi.GLB_Y/dfn4_vect_newgeo_nonprobempi.LOC_Y == 2)& (dfn4_vect_newgeo_nonprobempi.GLB_Z/dfn4_vect_newgeo_nonprobempi.LOC_Z == 2)]
dfn4VectNewgeoNonProb_256mpi_2omp = dfn4_vect_newgeo_nonprobempi[(dfn4_vect_newgeo_nonprobempi.omp_thrd == 2) & (dfn4_vect_newgeo_nonprobempi.GLB_T/dfn4_vect_newgeo_nonprobempi.LOC_T == 32) & (dfn4_vect_newgeo_nonprobempi.GLB_X/dfn4_vect_newgeo_nonprobempi.LOC_X == 2) & (dfn4_vect_newgeo_nonprobempi.GLB_Y/dfn4_vect_newgeo_nonprobempi.LOC_Y == 2)& (dfn4_vect_newgeo_nonprobempi.GLB_Z/dfn4_vect_newgeo_nonprobempi.LOC_Z == 2)]
dfn4VectNewgeoNonProb_512mpi_1omp = dfn4_vect_newgeo_nonprobempi[(dfn4_vect_newgeo_nonprobempi.omp_thrd == 1) & (dfn4_vect_newgeo_nonprobempi.GLB_T/dfn4_vect_newgeo_nonprobempi.LOC_T == 64) & (dfn4_vect_newgeo_nonprobempi.GLB_X/dfn4_vect_newgeo_nonprobempi.LOC_X == 2) & (dfn4_vect_newgeo_nonprobempi.GLB_Y/dfn4_vect_newgeo_nonprobempi.LOC_Y == 2)& (dfn4_vect_newgeo_nonprobempi.GLB_Z/dfn4_vect_newgeo_nonprobempi.LOC_Z == 2)]
dfn4VectNewgeoNonProb_56mpi_9omp = dfn4_vect_newgeo_nonprobempi[(dfn4_vect_newgeo_nonprobempi.omp_thrd == 9) & (dfn4_vect_newgeo_nonprobempi.GLB_T/dfn4_vect_newgeo_nonprobempi.LOC_T == 7) & (dfn4_vect_newgeo_nonprobempi.GLB_X/dfn4_vect_newgeo_nonprobempi.LOC_X == 2) & (dfn4_vect_newgeo_nonprobempi.GLB_Y/dfn4_vect_newgeo_nonprobempi.LOC_Y == 2)& (dfn4_vect_newgeo_nonprobempi.GLB_Z/dfn4_vect_newgeo_nonprobempi.LOC_Z == 2)]
dfn4VectNewgeoNonProb_248mpi_2omp = dfn4_vect_newgeo_nonprobempi[(dfn4_vect_newgeo_nonprobempi.omp_thrd == 2) & (dfn4_vect_newgeo_nonprobempi.GLB_T/dfn4_vect_newgeo_nonprobempi.LOC_T == 31) & (dfn4_vect_newgeo_nonprobempi.GLB_X/dfn4_vect_newgeo_nonprobempi.LOC_X == 2) & (dfn4_vect_newgeo_nonprobempi.GLB_Y/dfn4_vect_newgeo_nonprobempi.LOC_Y == 2)& (dfn4_vect_newgeo_nonprobempi.GLB_Z/dfn4_vect_newgeo_nonprobempi.LOC_Z == 2)]


## GFLOPS - Newgeo VECT: Probempi vs NonProbempi

In [73]:
fig = go.Figure() 

# Case 1
fig.add_trace(go.Scatter(x=(dfn4VectNewgeoProb_512mpi_1omp.Dirc_Data_Mov*(dfn4VectNewgeoProb_512mpi_1omp.MPI_size/4)), 
                         y=(dfn4VectNewgeoProb_512mpi_1omp.GFLOPS*(dfn4VectNewgeoProb_512mpi_1omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectNewgeoProb_512mpi_1omp',
                         line=dict(color='darkcyan',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))

fig.add_trace(go.Scatter(x=(dfn4VectNewgeoNonProb_512mpi_1omp.Dirc_Data_Mov*(dfn4VectNewgeoNonProb_512mpi_1omp.MPI_size/4)), 
                         y=(dfn4VectNewgeoNonProb_512mpi_1omp.GFLOPS*(dfn4VectNewgeoNonProb_512mpi_1omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectNewgeoNonProb_512mpi_1omp',
                         line=dict(color='magenta',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))


# Case 2
fig.add_trace(go.Scatter(x=(dfn4VectNewgeoProb_256mpi_2omp.Dirc_Data_Mov*(dfn4VectNewgeoProb_256mpi_2omp.MPI_size/4)), 
                         y=(dfn4VectNewgeoProb_256mpi_2omp.GFLOPS*(dfn4VectNewgeoProb_256mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectNewgeoProb_256mpi_2omp',
                         line=dict(color='royalblue',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))

fig.add_trace(go.Scatter(x=(dfn4VectNewgeoNonProb_256mpi_2omp.Dirc_Data_Mov*(dfn4VectNewgeoNonProb_256mpi_2omp.MPI_size/4)), 
                         y=(dfn4VectNewgeoNonProb_256mpi_2omp.GFLOPS*(dfn4VectNewgeoNonProb_256mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectNewgeoNonProb_256mpi_2omp',
                         line=dict(color='sandybrown',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))


# Case 3
fig.add_trace(go.Scatter(x=(dfn4VectNewgeoProb_128mpi_4omp.Dirc_Data_Mov*(dfn4VectNewgeoProb_128mpi_4omp.MPI_size/4)), 
                         y=(dfn4VectNewgeoProb_128mpi_4omp.GFLOPS*(dfn4VectNewgeoProb_128mpi_4omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectNewgeoProb_128mpi_4omp',
                         line=dict(color='darkblue',width=3),
                         marker=dict(symbol='circle',size=7, line=dict(color='tomato',width=2))))

fig.add_trace(go.Scatter(x=(dfn4VectNewgeoNonProb_128mpi_4omp.Dirc_Data_Mov*(dfn4VectNewgeoNonProb_128mpi_4omp.MPI_size/4)), 
                         y=(dfn4VectNewgeoNonProb_128mpi_4omp.GFLOPS*(dfn4VectNewgeoNonProb_128mpi_4omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectNewgeoNonProb_128mpi_4omp',
                         line=dict(color='cyan',width=3),
                         marker=dict(symbol='circle',size=7,line=dict(color='Purple',width=2))))



# Case 4
fig.add_trace(go.Scatter(x=(dfn4VectNewgeoProb_64mpi_8omp.Dirc_Data_Mov*(dfn4VectNewgeoProb_64mpi_8omp.MPI_size/4)), 
                         y=(dfn4VectNewgeoProb_64mpi_8omp.GFLOPS*(dfn4VectNewgeoProb_64mpi_8omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectNewgeoProb_64mpi_8omp',
                         line=dict(color='blue',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))

fig.add_trace(go.Scatter(x=(dfn4VectNewgeoNonProb_64mpi_8omp.Dirc_Data_Mov*(dfn4VectNewgeoNonProb_64mpi_8omp.MPI_size/4)), 
                         y=(dfn4VectNewgeoNonProb_64mpi_8omp.GFLOPS*(dfn4VectNewgeoNonProb_64mpi_8omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectNewgeoNonProb_64mpi_8omp',
                         line=dict(color='red',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))


# Case 5
fig.add_trace(go.Scatter(x=(dfn4VectNewgeoProb_32mpi_16omp.Dirc_Data_Mov*(dfn4VectNewgeoProb_32mpi_16omp.MPI_size/4)), 
                         y=(dfn4VectNewgeoProb_32mpi_16omp.GFLOPS*(dfn4VectNewgeoProb_32mpi_16omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectNewgeoProb_32mpi_16omp',
                         line=dict(color='chartreuse',width=3),
                         marker=dict(symbol='square-dot',size=6, line=dict(color='mediumvioletred',width=1))))

fig.add_trace(go.Scatter(x=(dfn4VectNewgeoNonProb_32mpi_16omp.Dirc_Data_Mov*(dfn4VectNewgeoNonProb_32mpi_16omp.MPI_size/4)), 
                         y=(dfn4VectNewgeoNonProb_32mpi_16omp.GFLOPS*(dfn4VectNewgeoNonProb_32mpi_16omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectNewgeoNonProb_32mpi_16omp',
                         line=dict(color='black',width=3),
                         marker=dict(symbol='square-dot',size=6, line=dict(color='mediumvioletred',width=1))))


# Case 6
fig.add_trace(go.Scatter(x=(dfn4VectNewgeoProb_56mpi_9omp.Dirc_Data_Mov*(dfn4VectNewgeoProb_56mpi_9omp.MPI_size/4)), 
                         y=(dfn4VectNewgeoProb_56mpi_9omp.GFLOPS*(dfn4VectNewgeoProb_56mpi_9omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectNewgeoProb_56mpi_9omp',
                         line=dict(color='gold',width=3),
                         marker=dict(symbol='triangle-up',size=6, line=dict(color='black',width=1))))

fig.add_trace(go.Scatter(x=(dfn4VectNewgeoNonProb_56mpi_9omp.Dirc_Data_Mov*(dfn4VectNewgeoNonProb_56mpi_9omp.MPI_size/4)), 
                         y=(dfn4VectNewgeoNonProb_56mpi_9omp.GFLOPS*(dfn4VectNewgeoNonProb_56mpi_9omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectNewgeoNonProb_56mpi_9omp',
                         line=dict(color='dimgray',width=3),
                         marker=dict(symbol='triangle-up',size=6, line=dict(color='black',width=1))))



# Case 7
fig.add_trace(go.Scatter(x=(dfn4VectNewgeoProb_248mpi_2omp.Dirc_Data_Mov*(dfn4VectNewgeoProb_248mpi_2omp.MPI_size/4)), 
                         y=(dfn4VectNewgeoProb_248mpi_2omp.GFLOPS*(dfn4VectNewgeoProb_248mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectNewgeoProb_248mpi_2omp',
                         line=dict(color='olive',width=3),
                         marker=dict(symbol='circle',size=6, line=dict(color='olive',width=2))))

fig.add_trace(go.Scatter(x=(dfn4VectNewgeoNonProb_248mpi_2omp.Dirc_Data_Mov*(dfn4VectNewgeoNonProb_248mpi_2omp.MPI_size/4)), 
                         y=(dfn4VectNewgeoNonProb_248mpi_2omp.GFLOPS*(dfn4VectNewgeoNonProb_248mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectNewgeoNonProb_248mpi_2omp',
                         line=dict(color='Brown',width=3),
                         marker=dict(symbol='circle',size=6, line=dict(color='darkred',width=2))))




# Adding a vertical line
fig.add_vline(x=512e6, line_width=3, line_dash="dash", line_color="green")
# fig.add_hline(y=1024, line_width=3, line_dash="dash", line_color="red")

# Add annotation
fig.add_annotation(
    x='512e6'
    , y=130
    , text=f'LLC Size Per Node'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="purple", family="Sans Serif")
    , align="left"
    ,)

fig.update_layout(title='"Newgeo-VECT (4-Nodes): probempi vs nonprobempi" GFlops Measurement',
                  xaxis_title='Dirc_Data_Mov', yaxis_title='GFLOPs/Node', 
                  xaxis=dict(ticks="outside"),  
                  yaxis=dict(ticks="outside"))

## BANDWIDTH - Newgeo VECT: Probempi vs NonProbempi

In [74]:
fig = go.Figure() 

# Case 1
fig.add_trace(go.Scatter(x=(dfn4VectNewgeoProb_512mpi_1omp.Dirc_Data_Mov*(dfn4VectNewgeoProb_512mpi_1omp.MPI_size/4)), 
                         y=(dfn4VectNewgeoProb_512mpi_1omp.BAND*(dfn4VectNewgeoProb_512mpi_1omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectNewgeoProb_512mpi_1omp',
                         line=dict(color='darkcyan',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))

fig.add_trace(go.Scatter(x=(dfn4VectNewgeoNonProb_512mpi_1omp.Dirc_Data_Mov*(dfn4VectNewgeoNonProb_512mpi_1omp.MPI_size/4)), 
                         y=(dfn4VectNewgeoNonProb_512mpi_1omp.BAND*(dfn4VectNewgeoNonProb_512mpi_1omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectNewgeoNonProb_512mpi_1omp',
                         line=dict(color='magenta',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))


# Case 2
fig.add_trace(go.Scatter(x=(dfn4VectNewgeoProb_256mpi_2omp.Dirc_Data_Mov*(dfn4VectNewgeoProb_256mpi_2omp.MPI_size/4)), 
                         y=(dfn4VectNewgeoProb_256mpi_2omp.BAND*(dfn4VectNewgeoProb_256mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectNewgeoProb_256mpi_2omp',
                         line=dict(color='royalblue',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))

fig.add_trace(go.Scatter(x=(dfn4VectNewgeoNonProb_256mpi_2omp.Dirc_Data_Mov*(dfn4VectNewgeoNonProb_256mpi_2omp.MPI_size/4)), 
                         y=(dfn4VectNewgeoNonProb_256mpi_2omp.BAND*(dfn4VectNewgeoNonProb_256mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectNewgeoNonProb_256mpi_2omp',
                         line=dict(color='sandybrown',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))


# Case 3
fig.add_trace(go.Scatter(x=(dfn4VectNewgeoProb_128mpi_4omp.Dirc_Data_Mov*(dfn4VectNewgeoProb_128mpi_4omp.MPI_size/4)), 
                         y=(dfn4VectNewgeoProb_128mpi_4omp.BAND*(dfn4VectNewgeoProb_128mpi_4omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectNewgeoProb_128mpi_4omp',
                         line=dict(color='darkblue',width=3),
                         marker=dict(symbol='circle',size=7, line=dict(color='tomato',width=2))))

fig.add_trace(go.Scatter(x=(dfn4VectNewgeoNonProb_128mpi_4omp.Dirc_Data_Mov*(dfn4VectNewgeoNonProb_128mpi_4omp.MPI_size/4)), 
                         y=(dfn4VectNewgeoNonProb_128mpi_4omp.BAND*(dfn4VectNewgeoNonProb_128mpi_4omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectNewgeoNonProb_128mpi_4omp',
                         line=dict(color='cyan',width=3),
                         marker=dict(symbol='circle',size=7,line=dict(color='Purple',width=2))))



# Case 4
fig.add_trace(go.Scatter(x=(dfn4VectNewgeoProb_64mpi_8omp.Dirc_Data_Mov*(dfn4VectNewgeoProb_64mpi_8omp.MPI_size/4)), 
                         y=(dfn4VectNewgeoProb_64mpi_8omp.BAND*(dfn4VectNewgeoProb_64mpi_8omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectNewgeoProb_64mpi_8omp',
                         line=dict(color='blue',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))

fig.add_trace(go.Scatter(x=(dfn4VectNewgeoNonProb_64mpi_8omp.Dirc_Data_Mov*(dfn4VectNewgeoNonProb_64mpi_8omp.MPI_size/4)), 
                         y=(dfn4VectNewgeoNonProb_64mpi_8omp.BAND*(dfn4VectNewgeoNonProb_64mpi_8omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectNewgeoNonProb_64mpi_8omp',
                         line=dict(color='red',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))


# Case 5
fig.add_trace(go.Scatter(x=(dfn4VectNewgeoProb_32mpi_16omp.Dirc_Data_Mov*(dfn4VectNewgeoProb_32mpi_16omp.MPI_size/4)), 
                         y=(dfn4VectNewgeoProb_32mpi_16omp.BAND*(dfn4VectNewgeoProb_32mpi_16omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectNewgeoProb_32mpi_16omp',
                         line=dict(color='chartreuse',width=3),
                         marker=dict(symbol='square-dot',size=6, line=dict(color='mediumvioletred',width=1))))

fig.add_trace(go.Scatter(x=(dfn4VectNewgeoNonProb_32mpi_16omp.Dirc_Data_Mov*(dfn4VectNewgeoNonProb_32mpi_16omp.MPI_size/4)), 
                         y=(dfn4VectNewgeoNonProb_32mpi_16omp.BAND*(dfn4VectNewgeoNonProb_32mpi_16omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectNewgeoNonProb_32mpi_16omp',
                         line=dict(color='black',width=3),
                         marker=dict(symbol='square-dot',size=6, line=dict(color='mediumvioletred',width=1))))


# Case 6
fig.add_trace(go.Scatter(x=(dfn4VectNewgeoProb_56mpi_9omp.Dirc_Data_Mov*(dfn4VectNewgeoProb_56mpi_9omp.MPI_size/4)), 
                         y=(dfn4VectNewgeoProb_56mpi_9omp.BAND*(dfn4VectNewgeoProb_56mpi_9omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectNewgeoProb_56mpi_9omp',
                         line=dict(color='gold',width=3),
                         marker=dict(symbol='triangle-up',size=6, line=dict(color='black',width=1))))

fig.add_trace(go.Scatter(x=(dfn4VectNewgeoNonProb_56mpi_9omp.Dirc_Data_Mov*(dfn4VectNewgeoNonProb_56mpi_9omp.MPI_size/4)), 
                         y=(dfn4VectNewgeoNonProb_56mpi_9omp.BAND*(dfn4VectNewgeoNonProb_56mpi_9omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectNewgeoNonProb_56mpi_9omp',
                         line=dict(color='dimgray',width=3),
                         marker=dict(symbol='triangle-up',size=6, line=dict(color='black',width=1))))



# Case 7
fig.add_trace(go.Scatter(x=(dfn4VectNewgeoProb_248mpi_2omp.Dirc_Data_Mov*(dfn4VectNewgeoProb_248mpi_2omp.MPI_size/4)), 
                         y=(dfn4VectNewgeoProb_248mpi_2omp.BAND*(dfn4VectNewgeoProb_248mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectNewgeoProb_248mpi_2omp',
                         line=dict(color='olive',width=3),
                         marker=dict(symbol='circle',size=6, line=dict(color='olive',width=2))))

fig.add_trace(go.Scatter(x=(dfn4VectNewgeoNonProb_248mpi_2omp.Dirc_Data_Mov*(dfn4VectNewgeoNonProb_248mpi_2omp.MPI_size/4)), 
                         y=(dfn4VectNewgeoNonProb_248mpi_2omp.BAND*(dfn4VectNewgeoNonProb_248mpi_2omp.MPI_size/4)), 
                         mode='lines+markers', 
                         name='VectNewgeoNonProb_248mpi_2omp',
                         line=dict(color='Brown',width=3),
                         marker=dict(symbol='circle',size=6, line=dict(color='darkred',width=2))))

# Adding vertical and horizontal lines 
fig.add_vline(x=512e6, line_width=3, line_dash="dot", line_color="green")
fig.add_hline(y=409, line_width=3, line_dash="dash", line_color="red")

# Add annotation with arrow
fig.add_annotation(
    x='512e6'
    , y=280
    , text=f'LLC Size Per Node: 512M'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="purple", family="Sans Serif")
    , align="left"
    ,)

fig.add_annotation(
    x='130e6'
    , y=370
    , text=f'Max Bandwidth Per Node'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="orange", family="Sans Serif")
    , align="left"
    ,)


fig.update_layout(title='"Newgeo VECT: Probempi vs NonProbempi" Bandwidth Measurement',
                  xaxis_title='Dirc_Data_Mov', yaxis_title='Bandwidth/Node', 
                  xaxis=dict(ticks="outside"),  
                  yaxis=dict(ticks="outside"),
                  legend=go.layout.Legend(bordercolor="black",borderwidth=1)) 



